In [1]:
import sequence_graph
import submatrix
from itertools import combinations
import re
import requests
from bs4 import BeautifulSoup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def align_graphs(x, y, substitution_matrix, s):
    """Computes a global pairwise alignment of two sequence graphs
    
    Uses a linear gap scoring function with space score s.
    The graphs are assumed to be in topological order, i.e., if vertex j is a parent of vertex i, then j < i
    In the case of ties during the traceback, the algorithm will traceback to the cell (i, j) in
    the dynamic programming matrix with the largest coordinates.
    
    Args:
        x: a sequence graph (a SequenceGraph object).
        y: a sequence graph (a SequenceGraph object).
        substitution_matrix: the substitution matrix, represented as a dictionary 
        s: the space score (a numeric value)
    Returns:
        A tuple (score, alignment), where alignment is represented
        as a list two aligned strings of the same length.
    """
    assert x.is_valid(), "sequence graph x is not valid"
    assert y.is_valid(), "sequence graph y is not valid"

    ###
    ### Your solution to PROBLEM 1
    ###
    
    n = x.num_vertices()
    m = y.num_vertices()  

    M = matrix(n, m)
    t = matrix(n, m)
    
    M[0][0]=0
    
    for i in range(1, n):
        parents = [(M[k][0]+s, (k,0)) for k in sorted(x.parents(i), reverse=True)]
        M[i][0], t[i][0] = max(parents)
        
    for j in range(1, m):
        parents = [(M[0][l]+s, (0,l)) for l in sorted(y.parents(j), reverse=True)]
        M[0][j], t[0][j] = max(parents)
        
    
    for i in range(1, n-1):
        for j in range(1, m-1):
            vals1 = []
            for k in sorted(x.parents(i), reverse=True):
                for l in sorted(y.parents(j), reverse=True):
                    vals1.append((M[k][l] + substitution_matrix[x.vertex_label(i), y.vertex_label(j)], (k,l)))
                    
            val1 = max(vals1)
            vals2 = [(M[k][j]+s, (k, j)) for k in sorted(x.parents(i), reverse=True)]
            val2 = max(vals2)
            vals3 = [(M[i][l]+s, (i, l)) for l in sorted(y.parents(j), reverse=True)]
            val3 = max(vals3)
            vals = sorted([val1, val2, val3], key=lambda item:item[1], reverse=True)
            M[i][j], t[i][j] = max(vals)
    
    vals = []
    for k in x.parents(n-1):
        for l in y.parents(m-1):
            vals.append((M[k][l], (k,l)))
    M[n-1][m-1], t[n-1][m-1] = max(vals)
    
    x_result = ""
    y_result = ""
    i, j = n-1,m-1
    
    pairs = []
    while i or j:
        traceback = t[i][j]
        p, q = traceback
                    
        if p == i:
            pairs.append(("-", y.vertex_label(j)))
        
        elif q == j:
            pairs.append((x.vertex_label(i), "-"))
            
        else:
            pairs.append((x.vertex_label(i) or "-", y.vertex_label(j) or "-"))
            
        while j-q > 1:
            j -= 1
            # pairs.append((" ", y.vertex_label(j)))
        
        while i-p > 1:
            i -= 1
            # pairs.append((x.vertex_label(i), " "))

                
        i = p
        j = q
    
    pairs.reverse()
    pairs = pairs[:-1]    
    
    return (M[n-1][m-1], pairs) #transpose_alignment(pairs))
    
# Some helper functions that may be of use to you in your implementation
def matrix(num_rows, num_cols, initial_value=None):
    """Returns a matrix (a list of rows, each of which is a list) 
    with num_rows and num_cols and with initial_value in each entry"""
    return [[initial_value] * num_cols for i in range(num_rows)]

def transpose_alignment(alignment):
    """Returns a column-based alignment from a row-based alignment or vice versa"""
    return list(map(''.join, zip(*alignment)))

In [4]:
DNA = 'COMPSCI300|COMPSCI220|MATH217|MATH221|MATH222|STAT240|COMPSCI320|MATH320|STAT340|COMPSCI576|COMPSCI513|COMPSCI471|COMPSCI412|COMPSCI400|STAT309|ECON410|ECON400|COMPSCI533|COMPSCI525|COMPSCI425|ECE570|LIS461|COMPSCI539|MATH535|ISYE521|GENBUS656|COMPSCI540|COMPSCI539|COMPSCI532|CHEM115|CHEM116|CHEM109|CHEM103CHEM104|PHYSICS207|PHYSICS208|PHYSICS201|PHYSICS202|PHYSICS103PHYSICS104|BIOCORE381|BIOCORE383|BOTANY130|ZOOLOGY101|ZOOLOGY102|BIOLOGY151BIOLOGY152|MATH171|MATH217|MATH221|CHEM343CHEM345|CHEM341|STAT240|STAT301|STAT371|GENETICS466|GENETICS467GENETICS468|BIOCHEM507|BIOCHEM501|BIOCHEM570|COMPSCI576|BIOCHEM560|BIOCHEM508|GENETICS528|GENETICS525|GENETICS520|GENETICS527|GENETICS522|GENETICS299|GENETICS545'.split("|")
basic_dna_submatrix = submatrix.match_mismatch_matrix(1, -1, DNA)

In [5]:
# ds = "|".join("""COMPSCI220
# MATH221
# MATH222
# STAT240
# COMPSCI320
# MATH320
# STAT340
# COMPSCI400
# ECON400
# COMPSCI425
# LIS461
# COMPSCI532""".split("\n"))
# ds

# variants = [
#     (2, "MATH217"),
#     (1, "COMPSCI300"),
#     (11, "ECE570"),
#     (12, "COMPSCI539"),
#     (12, "COMPSCI540"),
#     (12, "GENBUS656"),
#     (12, "ISYE521"),
#     (12, "MATH535"),
#     (12, "COMPSCI539"),
#     (8, "COMPSCI412"),
#     (8, "COMPSCI471"),
#     (8, "COMPSCI513"),
#     (8, "COMPSCI576"),
#     (9, "ECON410"),
#     (9, "STAT309"),
#     (10, "COMPSCI525"),
#     (10, "COMPSCI533"),
# ]
 
# ds_g = sequence_graph.VariantGraph(ds)
# for var in variants:
#     ds_g.add_substitution_variant(*var)
# ds_g.reorder_topological()
# ds_g.plot()
# ds_g.is_in_topological_order()

In [6]:
# gg = """CHEM103CHEM104
# MATH221
# STAT371
# CHEM341
# PHYSICS103PHYSICS104
# BIOLOGY151BIOLOGY152
# BIOCHEM501
# GENETICS467GENETICS468
# GENETICS545
# GENETICS520
# GENETICS522
# BIOCHEM508""".split("\n")

# l=sorted(gg, key=lambda item: int(re.findall(r"\d{3}", item)[0]))
# [(l[i], i+1) for i in range(len(l))]

# gg = "|".join(sorted(gg, key=lambda item: int(re.findall(r"\d{3}", item)[0])))
# gg

# variants = [
#     (4, "MATH171|MATH217"),
#     (6, "STAT301"),
#     (6, "STAT240"),
#     (1, "CHEM109"),
#     (1, "CHEM115|CHEM116"),
#     (5, "CHEM343CHEM345"),
#     (2, "PHYSICS201|PHYSICS202"),
#     (2, "PHYSICS207|PHYSICS208"),
#     (3, "BOTANY130|ZOOLOGY101|ZOOLOGY102"),
#     (3, "BIOCORE381|BIOCORE383"),
#     (8, "BIOCHEM507"),
#     (7, "GENETICS466"),
#     (12, "GENETICS299"),
#     (10, "GENETICS525"),
#     (11, "GENETICS527"),
#     (10, "GENETICS528"),
#     (9, "BIOCHEM560"),
#     (9, "COMPSCI576"),
#     (9, "BIOCHEM570")
# ]
 
# gg_g = sequence_graph.VariantGraph(gg)
# for var in variants:
#     gg_g.add_substitution_variant(*var)
    
# gg_g.reorder_topological()
# gg_g.plot()

In [7]:
major_alphabet = {}
major_graph = {}

In [8]:
with requests.get("https://guide.wisc.edu/undergraduate/letters-science/astronomy/astronomy-physics-bs/#requirementstext") as r:
    soup = BeautifulSoup(r.text, 'html.parser')

    courses = []

    for element in soup.find("div", {"id":"requirementstextcontainer"}).find_all("a", {"class":"bubblelink code"}):
        course = element.text.encode('ascii', 'ignore').decode('ascii')
        if re.findall(r"[A-Z]+[0-9]+", course):
            courses.append(course)
            
courses

major_alphabet["astronomy-physics-bs"] = {}
for i in range(len(courses)):
    if courses[i] not in major_alphabet["astronomy-physics-bs"]:
        major_alphabet["astronomy-physics-bs"][courses[i]] = i

In [9]:
astro_ref = "ASTRON310,ASTRON320,PHYSICS247,PHYSICS248,PHYSICS249,PHYSICS311,PHYSICS322,PHYSICS415,PHYSICS448,PHYSICS449,ASTRON465"
astro_l = astro_ref.split(",")
astro_break = [(astro_l[i], i+1) for i in range(len(astro_l))]
astro_break

[('ASTRON310', 1),
 ('ASTRON320', 2),
 ('PHYSICS247', 3),
 ('PHYSICS248', 4),
 ('PHYSICS249', 5),
 ('PHYSICS311', 6),
 ('PHYSICS322', 7),
 ('PHYSICS415', 8),
 ('PHYSICS448', 9),
 ('PHYSICS449', 10),
 ('ASTRON465', 11)]

In [10]:
for i in combinations(['ASTRON310',
 'ASTRON320',
 'ASTRON330',
 'ASTRON335',
 'ASTRON340',
 'ASTRON500',], 2):
    j = (1,2,",".join(i)) 
    print(j)

(1, 2, 'ASTRON310,ASTRON320')
(1, 2, 'ASTRON310,ASTRON330')
(1, 2, 'ASTRON310,ASTRON335')
(1, 2, 'ASTRON310,ASTRON340')
(1, 2, 'ASTRON310,ASTRON500')
(1, 2, 'ASTRON320,ASTRON330')
(1, 2, 'ASTRON320,ASTRON335')
(1, 2, 'ASTRON320,ASTRON340')
(1, 2, 'ASTRON320,ASTRON500')
(1, 2, 'ASTRON330,ASTRON335')
(1, 2, 'ASTRON330,ASTRON340')
(1, 2, 'ASTRON330,ASTRON500')
(1, 2, 'ASTRON335,ASTRON340')
(1, 2, 'ASTRON335,ASTRON500')
(1, 2, 'ASTRON340,ASTRON500')


In [11]:
astro_variants = [
    (1, 2, 'ASTRON310,ASTRON320'),
(1, 2, 'ASTRON310,ASTRON330'),
(1, 2, 'ASTRON310,ASTRON335'),
(1, 2, 'ASTRON310,ASTRON340'),
(1, 2, 'ASTRON310,ASTRON500'),
(1, 2, 'ASTRON320,ASTRON330'),
(1, 2, 'ASTRON320,ASTRON335'),
(1, 2, 'ASTRON320,ASTRON340'),
(1, 2, 'ASTRON320,ASTRON500'),
(1, 2, 'ASTRON330,ASTRON335'),
(1, 2, 'ASTRON330,ASTRON340'),
(1, 2, 'ASTRON330,ASTRON500'),
(1, 2, 'ASTRON335,ASTRON340'),
(1, 2, 'ASTRON335,ASTRON500'),
(1, 2, 'ASTRON340,ASTRON500'),
    (3,3,'PHYSICS201,PHYSICS202,PHYSICS205'),
    (3,3,'PHYSICS207,PHYSICS208,PHYSICS241'),
    (9,2, 'PHYSICS531'),
    (11,1, 'PHYSICS307')]

In [12]:
astro_g = sequence_graph.VariantGraph(astro_ref)
for var in astro_variants:
    astro_g.add_complex_variant(*var)
    
astro_g.reorder_topological()
astro_g.plot()
major_graph["astronomy-physics-bs"] = astro_g

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1060.0px" height="1380.0px" viewBox="0 0 1060.0 1380.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8126b2ee2c1e4665a85b37e1bc7345b4"> ASTRON340 ASTRON500 ASTRON335 ASTRON500 ASTRON335 ASTRON340 ASTRON330 ASTRON500 ASTRON330 ASTRON340 ASTRON330 ASTRON335 ASTRON320 ASTRON500 ASTRON320 ASTRON340 ASTRON320 ASTRON335 ASTRON320 ASTRON330 ASTRON310 ASTRON500 ASTRON310 ASTRON340 ASTRON310 ASTRON335 ASTRON310 ASTRON330 ASTRON310 ASTRON320 ASTRON310 ASTRON320 PHYSICS207 PHYSICS208 PHYSICS241 PHYSICS201 PHYSICS202 PHYSICS205 PHYSICS247 PHYSICS248 PHYSICS249 PHYSICS311 PHYSICS322 PHYSICS415 PHYSICS531 PHYSICS448 PHYSICS449 PHYSICS307 ASTRON465

In [13]:
with requests.get("https://guide.wisc.edu/undergraduate/agricultural-life-sciences/animal-dairy-sciences/dairy-science-bs/#requirementstext") as r:
    soup = BeautifulSoup(r.text, 'html.parser')

    courses = []

    for element in soup.find("div", {"id":"requirementstextcontainer"}).find_all("a", {"class":"bubblelink code"}):
        course = element.text.encode('ascii', 'ignore').decode('ascii')
        if re.findall(r"[A-Z]+[0-9]+", course):
            dep,num = re.findall(r"(\D+)([0-9]+)", course)[0]
            codes = sorted(dep.split("/"))[0]+num
            codes = re.sub(r"\W", "", codes)
            courses.append(codes)
            
courses


major_alphabet["dairy-science-bs"] = {}
for i in range(len(courses)):
    if courses[i] not in major_alphabet["dairy-science-bs"]:
        major_alphabet["dairy-science-bs"][courses[i]] = i

In [14]:
dairy_ref = "MATH112,STAT301,CHEM103,CHEM104,ZOOLOGY101,ZOOLOGY102,AGRONOMY100,GENETICS466,BIOCHEM301,AAE101,ANSCI101,ANSCI102,DYSCI233,DYSCI233,DYSCI234,ANSCI311,ANSCI361,ANSCI361,ANSCI362,ANSCI373,DYSCI378,ANSCI414,ANSCI434,DYSCI399,DYSCI535,ANSCI135"
dairy_l = dairy_ref.split(",")
dairy_break = [(dairy_l[i], i+1) for i in range(len(dairy_l))]
dairy_break

[('MATH112', 1),
 ('STAT301', 2),
 ('CHEM103', 3),
 ('CHEM104', 4),
 ('ZOOLOGY101', 5),
 ('ZOOLOGY102', 6),
 ('AGRONOMY100', 7),
 ('GENETICS466', 8),
 ('BIOCHEM301', 9),
 ('AAE101', 10),
 ('ANSCI101', 11),
 ('ANSCI102', 12),
 ('DYSCI233', 13),
 ('DYSCI233', 14),
 ('DYSCI234', 15),
 ('ANSCI311', 16),
 ('ANSCI361', 17),
 ('ANSCI361', 18),
 ('ANSCI362', 19),
 ('ANSCI373', 20),
 ('DYSCI378', 21),
 ('ANSCI414', 22),
 ('ANSCI434', 23),
 ('DYSCI399', 24),
 ('DYSCI535', 25),
 ('ANSCI135', 26)]

In [15]:
for i in [ 'ANSCI135',
 'DYSCI205',
 'DYSCI289',
 'DYSCI299',
 'ANSCI370',
 'DYSCI375',
 'AGRONOMY471',
 'ANSCI472',
 'ANSCI473',
 'DYSCI534',
 'DYSCI681',
 'DYSCI682',
 'DYSCI699',]:
    j = (26,1,i) 
    print(j, ",")

# (26, 1, ",".join(['ANSCI135',
#  'DYSCI205',
#  'DYSCI289',
#  'DYSCI299',
#  'ANSCI370',
#  'DYSCI375',
#  'AGRONOMY471',
#  'ANSCI472',
#  'ANSCI473',
#  'DYSCI534',
#  'DYSCI681',
#  'DYSCI682',
#  'DYSCI699']))

(26, 1, 'ANSCI135') ,
(26, 1, 'DYSCI205') ,
(26, 1, 'DYSCI289') ,
(26, 1, 'DYSCI299') ,
(26, 1, 'ANSCI370') ,
(26, 1, 'DYSCI375') ,
(26, 1, 'AGRONOMY471') ,
(26, 1, 'ANSCI472') ,
(26, 1, 'ANSCI473') ,
(26, 1, 'DYSCI534') ,
(26, 1, 'DYSCI681') ,
(26, 1, 'DYSCI682') ,
(26, 1, 'DYSCI699') ,


In [16]:
dairy_variants = [
    (1, 1, "MATH114"),
    (1, 1, "MATH171"),
    (2, 1, 'STAT371'),
    (3, 2, 'CHEM109'),
    (5, 3, 'ZOOLOGY101,ZOOLOGY102,BOTANY130'),
    (5, 3, 'BIOLOGY151,BIOLOGY152'),
    (8, 1, 'CHEM341') ,
(8, 1, 'CHEM343') ,
(8, 1, 'MICROBIO101') ,
(8, 1, 'MICROBIO303') ,
(8, 1, 'MMI341') ,
    (9, 1, 'BIOCHEM501'),
    (9,1,'BIOCHEM507,BIOCHEM508'),
    (10,1,'ECON101'),
    (19, 1,'ANSCI363'),
(26, 1, 'DYSCI205') ,
(26, 1, 'DYSCI289') ,
(26, 1, 'DYSCI299') ,
(26, 1, 'ANSCI370') ,
(26, 1, 'DYSCI375') ,
(26, 1, 'AGRONOMY471') ,
(26, 1, 'ANSCI472') ,
(26, 1, 'ANSCI473') ,
(26, 1, 'DYSCI534') ,
(26, 1, 'DYSCI681') ,
(26, 1, 'DYSCI682') ,
(26, 1, 'DYSCI699') ,
]

In [17]:
dairy_g = sequence_graph.VariantGraph(dairy_ref)
for var in dairy_variants:
    dairy_g.add_complex_variant(*var)
    
dairy_g.reorder_topological()
dairy_g.plot()
major_graph["dairy-science-bs"] = dairy_g

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="2340.0px" height="1140.0px" viewBox="0 0 2340.0 1140.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7e4b78ada9394626896f3c6d05d13826"> MATH171 MATH114 MATH112 STAT371 STAT301 CHEM109 CHEM103 CHEM104 BIOLOGY151 BIOLOGY152 ZOOLOGY101 ZOOLOGY102 BOTANY130 ZOOLOGY101 ZOOLOGY102 AGRONOMY100 MMI341 MICROBIO303 MICROBIO101 CHEM343 CHEM341 GENETICS466 BIOCHEM507 BIOCHEM508 BIOCHEM501 BIOCHEM301 ECON101 AAE101 ANSCI101 ANSCI102 DYSCI233 DYSCI233 DYSCI234 ANSCI311 ANSCI361 ANSCI361 ANSCI363 ANSCI362 ANSCI373 DYSCI378 ANSCI414 ANSCI434 DYSCI399 DYSCI535 DYSCI699 DYSCI682 DYSCI681 DYSCI534 ANSCI473 ANSCI472 AGRONOMY471 DYSCI375 ANSCI370 DYSCI299 DYSCI289 DYSCI205 ANSCI135

In [18]:
with requests.get("https://guide.wisc.edu/undergraduate/agricultural-life-sciences/genetics/genetics-genomics-bs/#requirementstext") as r:
    soup = BeautifulSoup(r.text, 'html.parser')

    courses = []

    for element in soup.find("div", {"id":"requirementstextcontainer"}).find_all("a", {"class":"bubblelink code"}):
        course = element.text.encode('ascii', 'ignore').decode('ascii')
        if re.findall(r"[A-Z]+[0-9]+", course):
            dep,num = re.findall(r"(\D+)([0-9]+)", course)[0]
            codes = sorted(dep.split("/"))[0]+num
            codes = re.sub(r"\W", "", codes)
            courses.append(codes)
            
courses

major_alphabet["genetics-genomics-bs"] = {}
for i in range(len(courses)):
    if courses[i] not in major_alphabet["genetics-genomics-bs"]:
        major_alphabet["genetics-genomics-bs"][courses[i]] = i

In [19]:
gene_ref = "MATH221,STAT371,CHEM103,CHEM104,CHEM341,PHYSICS103,PHYSICS104,BIOLOGY151,BIOLOGY152,BIOCHEM501,GENETICS467,GENETICS468,GENETICS545,BIOLOGY522,GENETICS520,BIOLOGY522,GENETICS525,BIOCHEM508,BIOCHEM560"
gene_l = gene_ref.split(",")
gene_break = [(gene_l[i], i+1) for i in range(len(gene_l))]
gene_break

[('MATH221', 1),
 ('STAT371', 2),
 ('CHEM103', 3),
 ('CHEM104', 4),
 ('CHEM341', 5),
 ('PHYSICS103', 6),
 ('PHYSICS104', 7),
 ('BIOLOGY151', 8),
 ('BIOLOGY152', 9),
 ('BIOCHEM501', 10),
 ('GENETICS467', 11),
 ('GENETICS468', 12),
 ('GENETICS545', 13),
 ('BIOLOGY522', 14),
 ('GENETICS520', 15),
 ('BIOLOGY522', 16),
 ('GENETICS525', 17),
 ('BIOCHEM508', 18),
 ('BIOCHEM560', 19)]

In [20]:
for i in combinations(['BIOCHEM508',
 'BIOCHEM560',
 'CHEM344',
 'CHEM345',
 'BIOCHEM575',
 'BIOCORE485',
 'BIOCORE587',
 'ANTHRO410',
 'BOTANY563',
 'MICROBIO303',
 'MICROBIO304',
 'MICROBIO470',
 'MICROBIO545',
 'MICROBIO632',
 'MMI341',
 'MMI528',
 'PHMSCI490',
 'PLPATH622',
 'MMI640',
 'ENVIRST360',
 'ZOOLOGY425',
 'ZOOLOGY470',
 'ZOOLOGY555',
 'ZOOLOGY570',
 'AGRONOMY338',
 'AGRONOMY340',
 'AGRONOMY360',
 'AGRONOMY501',
 'AGRONOMY502',
 'ANSCI361',
 'ANSCI362',
 'ANSCI363',
 'BOTANY505',
 'BMI576',],2):
    j = (18,2,",".join(i)) 
    print(j, ",")

# for i in ['GENETICS299',
 # 'GENETICS699',
 # 'GENETICS681',
 # 'GENETICS682',
 # 'GENETICS399',]:
 #    j = (13,1,i) 
 #    print(j, ",")

(18, 2, 'BIOCHEM508,BIOCHEM560') ,
(18, 2, 'BIOCHEM508,CHEM344') ,
(18, 2, 'BIOCHEM508,CHEM345') ,
(18, 2, 'BIOCHEM508,BIOCHEM575') ,
(18, 2, 'BIOCHEM508,BIOCORE485') ,
(18, 2, 'BIOCHEM508,BIOCORE587') ,
(18, 2, 'BIOCHEM508,ANTHRO410') ,
(18, 2, 'BIOCHEM508,BOTANY563') ,
(18, 2, 'BIOCHEM508,MICROBIO303') ,
(18, 2, 'BIOCHEM508,MICROBIO304') ,
(18, 2, 'BIOCHEM508,MICROBIO470') ,
(18, 2, 'BIOCHEM508,MICROBIO545') ,
(18, 2, 'BIOCHEM508,MICROBIO632') ,
(18, 2, 'BIOCHEM508,MMI341') ,
(18, 2, 'BIOCHEM508,MMI528') ,
(18, 2, 'BIOCHEM508,PHMSCI490') ,
(18, 2, 'BIOCHEM508,PLPATH622') ,
(18, 2, 'BIOCHEM508,MMI640') ,
(18, 2, 'BIOCHEM508,ENVIRST360') ,
(18, 2, 'BIOCHEM508,ZOOLOGY425') ,
(18, 2, 'BIOCHEM508,ZOOLOGY470') ,
(18, 2, 'BIOCHEM508,ZOOLOGY555') ,
(18, 2, 'BIOCHEM508,ZOOLOGY570') ,
(18, 2, 'BIOCHEM508,AGRONOMY338') ,
(18, 2, 'BIOCHEM508,AGRONOMY340') ,
(18, 2, 'BIOCHEM508,AGRONOMY360') ,
(18, 2, 'BIOCHEM508,AGRONOMY501') ,
(18, 2, 'BIOCHEM508,AGRONOMY502') ,
(18, 2, 'BIOCHEM508,ANSCI361') ,

In [21]:
gene_variants = [
    (1,1,'MATH171,MATH217'),
    (2,1,'STAT301'),
    (2,1,'STAT240'),
    (3,2,'CHEM109'),
    (3,2,'CHEM115,CHEM116'),
    (5,1,'CHEM343,CHEM345'),
    (6,2,'PHYSICS201,PHYSICS202'),
    (6,2,'PHYSICS207,PHYSICS208'),
    (8,2,'BIOLOGY130,BIOLOGY101,BIOLOGY102'),
    (8, 2, 'BIOCORE381,BIOCORE383,BIOCORE382,BIOCORE384') ,
(8, 2, 'BIOCORE381,BIOCORE383,BIOCORE382,BIOCORE486') ,
(8, 2, 'BIOCORE381,BIOCORE383,BIOCORE384,BIOCORE486') ,
    (10, 1, 'BIOCHEM507'),
    (11,2,'GENETICS466'),
    (13, 1, 'GENETICS299') ,
(13, 1, 'GENETICS699') ,
(13, 1, 'GENETICS681') ,
(13, 1, 'GENETICS682') ,
(13, 1, 'GENETICS399') ,
    (14,1,'GENETICS527'),
    (14,1,'GENETICS566'),
    (14,1,'GENETICS564'),
    (14,1,'GENETICS699,GENETICS567'),
    (14,1,'GENETICS681,GENETICS682,GENETICS567'),
    (15, 3, 'GENETICS520,BIOLOGY522,GENETICS525') ,
(15, 3, 'GENETICS520,BIOLOGY522,GENETICS527') ,
(15, 3, 'GENETICS520,BIOLOGY522,GENETICS528') ,
(15, 3, 'GENETICS520,BIOLOGY522,GENETICS548') ,
(15, 3, 'GENETICS520,BIOLOGY522,GENETICS564') ,
(15, 3, 'GENETICS520,BIOLOGY522,GENETICS565') ,
(15, 3, 'GENETICS520,BIOLOGY522,GENETICS566') ,
(15, 3, 'GENETICS520,BIOLOGY522,GENETICS588') ,
(15, 3, 'GENETICS520,BIOLOGY522,GENETICS605') ,
(15, 3, 'GENETICS520,BIOLOGY522,BIOCHEM612') ,
(15, 3, 'GENETICS520,BIOLOGY522,BIOCHEM620') ,
(15, 3, 'GENETICS520,BIOLOGY522,CHEM626') ,
(15, 3, 'GENETICS520,BIOLOGY522,GENETICS627') ,
(15, 3, 'GENETICS520,GENETICS525,GENETICS527') ,
(15, 3, 'GENETICS520,GENETICS525,GENETICS528') ,
(15, 3, 'GENETICS520,GENETICS525,GENETICS548') ,
(15, 3, 'GENETICS520,GENETICS525,GENETICS564') ,
(15, 3, 'GENETICS520,GENETICS525,GENETICS565') ,
(15, 3, 'GENETICS520,GENETICS525,GENETICS566') ,
(15, 3, 'GENETICS520,GENETICS525,GENETICS588') ,
(15, 3, 'GENETICS520,GENETICS525,GENETICS605') ,
(15, 3, 'GENETICS520,GENETICS525,BIOCHEM612') ,
(15, 3, 'GENETICS520,GENETICS525,BIOCHEM620') ,
(15, 3, 'GENETICS520,GENETICS525,CHEM626') ,
(15, 3, 'GENETICS520,GENETICS525,GENETICS627') ,
(15, 3, 'GENETICS520,GENETICS527,GENETICS528') ,
(15, 3, 'GENETICS520,GENETICS527,GENETICS548') ,
(15, 3, 'GENETICS520,GENETICS527,GENETICS564') ,
(15, 3, 'GENETICS520,GENETICS527,GENETICS565') ,
(15, 3, 'GENETICS520,GENETICS527,GENETICS566') ,
(15, 3, 'GENETICS520,GENETICS527,GENETICS588') ,
(15, 3, 'GENETICS520,GENETICS527,GENETICS605') ,
(15, 3, 'GENETICS520,GENETICS527,BIOCHEM612') ,
(15, 3, 'GENETICS520,GENETICS527,BIOCHEM620') ,
(15, 3, 'GENETICS520,GENETICS527,CHEM626') ,
(15, 3, 'GENETICS520,GENETICS527,GENETICS627') ,
(15, 3, 'GENETICS520,GENETICS528,GENETICS548') ,
(15, 3, 'GENETICS520,GENETICS528,GENETICS564') ,
(15, 3, 'GENETICS520,GENETICS528,GENETICS565') ,
(15, 3, 'GENETICS520,GENETICS528,GENETICS566') ,
(15, 3, 'GENETICS520,GENETICS528,GENETICS588') ,
(15, 3, 'GENETICS520,GENETICS528,GENETICS605') ,
(15, 3, 'GENETICS520,GENETICS528,BIOCHEM612') ,
(15, 3, 'GENETICS520,GENETICS528,BIOCHEM620') ,
(15, 3, 'GENETICS520,GENETICS528,CHEM626') ,
(15, 3, 'GENETICS520,GENETICS528,GENETICS627') ,
(15, 3, 'GENETICS520,GENETICS548,GENETICS564') ,
(15, 3, 'GENETICS520,GENETICS548,GENETICS565') ,
(15, 3, 'GENETICS520,GENETICS548,GENETICS566') ,
(15, 3, 'GENETICS520,GENETICS548,GENETICS588') ,
(15, 3, 'GENETICS520,GENETICS548,GENETICS605') ,
(15, 3, 'GENETICS520,GENETICS548,BIOCHEM612') ,
(15, 3, 'GENETICS520,GENETICS548,BIOCHEM620') ,
(15, 3, 'GENETICS520,GENETICS548,CHEM626') ,
(15, 3, 'GENETICS520,GENETICS548,GENETICS627') ,
(15, 3, 'GENETICS520,GENETICS564,GENETICS565') ,
(15, 3, 'GENETICS520,GENETICS564,GENETICS566') ,
(15, 3, 'GENETICS520,GENETICS564,GENETICS588') ,
(15, 3, 'GENETICS520,GENETICS564,GENETICS605') ,
(15, 3, 'GENETICS520,GENETICS564,BIOCHEM612') ,
(15, 3, 'GENETICS520,GENETICS564,BIOCHEM620') ,
(15, 3, 'GENETICS520,GENETICS564,CHEM626') ,
(15, 3, 'GENETICS520,GENETICS564,GENETICS627') ,
(15, 3, 'GENETICS520,GENETICS565,GENETICS566') ,
(15, 3, 'GENETICS520,GENETICS565,GENETICS588') ,
(15, 3, 'GENETICS520,GENETICS565,GENETICS605') ,
(15, 3, 'GENETICS520,GENETICS565,BIOCHEM612') ,
(15, 3, 'GENETICS520,GENETICS565,BIOCHEM620') ,
(15, 3, 'GENETICS520,GENETICS565,CHEM626') ,
(15, 3, 'GENETICS520,GENETICS565,GENETICS627') ,
(15, 3, 'GENETICS520,GENETICS566,GENETICS588') ,
(15, 3, 'GENETICS520,GENETICS566,GENETICS605') ,
(15, 3, 'GENETICS520,GENETICS566,BIOCHEM612') ,
(15, 3, 'GENETICS520,GENETICS566,BIOCHEM620') ,
(15, 3, 'GENETICS520,GENETICS566,CHEM626') ,
(15, 3, 'GENETICS520,GENETICS566,GENETICS627') ,
(15, 3, 'GENETICS520,GENETICS588,GENETICS605') ,
(15, 3, 'GENETICS520,GENETICS588,BIOCHEM612') ,
(15, 3, 'GENETICS520,GENETICS588,BIOCHEM620') ,
(15, 3, 'GENETICS520,GENETICS588,CHEM626') ,
(15, 3, 'GENETICS520,GENETICS588,GENETICS627') ,
(15, 3, 'GENETICS520,GENETICS605,BIOCHEM612') ,
(15, 3, 'GENETICS520,GENETICS605,BIOCHEM620') ,
(15, 3, 'GENETICS520,GENETICS605,CHEM626') ,
(15, 3, 'GENETICS520,GENETICS605,GENETICS627') ,
(15, 3, 'GENETICS520,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'GENETICS520,BIOCHEM612,CHEM626') ,
(15, 3, 'GENETICS520,BIOCHEM612,GENETICS627') ,
(15, 3, 'GENETICS520,BIOCHEM620,CHEM626') ,
(15, 3, 'GENETICS520,BIOCHEM620,GENETICS627') ,
(15, 3, 'GENETICS520,CHEM626,GENETICS627') ,
(15, 3, 'BIOLOGY522,GENETICS525,GENETICS527') ,
(15, 3, 'BIOLOGY522,GENETICS525,GENETICS528') ,
(15, 3, 'BIOLOGY522,GENETICS525,GENETICS548') ,
(15, 3, 'BIOLOGY522,GENETICS525,GENETICS564') ,
(15, 3, 'BIOLOGY522,GENETICS525,GENETICS565') ,
(15, 3, 'BIOLOGY522,GENETICS525,GENETICS566') ,
(15, 3, 'BIOLOGY522,GENETICS525,GENETICS588') ,
(15, 3, 'BIOLOGY522,GENETICS525,GENETICS605') ,
(15, 3, 'BIOLOGY522,GENETICS525,BIOCHEM612') ,
(15, 3, 'BIOLOGY522,GENETICS525,BIOCHEM620') ,
(15, 3, 'BIOLOGY522,GENETICS525,CHEM626') ,
(15, 3, 'BIOLOGY522,GENETICS525,GENETICS627') ,
(15, 3, 'BIOLOGY522,GENETICS527,GENETICS528') ,
(15, 3, 'BIOLOGY522,GENETICS527,GENETICS548') ,
(15, 3, 'BIOLOGY522,GENETICS527,GENETICS564') ,
(15, 3, 'BIOLOGY522,GENETICS527,GENETICS565') ,
(15, 3, 'BIOLOGY522,GENETICS527,GENETICS566') ,
(15, 3, 'BIOLOGY522,GENETICS527,GENETICS588') ,
(15, 3, 'BIOLOGY522,GENETICS527,GENETICS605') ,
(15, 3, 'BIOLOGY522,GENETICS527,BIOCHEM612') ,
(15, 3, 'BIOLOGY522,GENETICS527,BIOCHEM620') ,
(15, 3, 'BIOLOGY522,GENETICS527,CHEM626') ,
(15, 3, 'BIOLOGY522,GENETICS527,GENETICS627') ,
(15, 3, 'BIOLOGY522,GENETICS528,GENETICS548') ,
(15, 3, 'BIOLOGY522,GENETICS528,GENETICS564') ,
(15, 3, 'BIOLOGY522,GENETICS528,GENETICS565') ,
(15, 3, 'BIOLOGY522,GENETICS528,GENETICS566') ,
(15, 3, 'BIOLOGY522,GENETICS528,GENETICS588') ,
(15, 3, 'BIOLOGY522,GENETICS528,GENETICS605') ,
(15, 3, 'BIOLOGY522,GENETICS528,BIOCHEM612') ,
(15, 3, 'BIOLOGY522,GENETICS528,BIOCHEM620') ,
(15, 3, 'BIOLOGY522,GENETICS528,CHEM626') ,
(15, 3, 'BIOLOGY522,GENETICS528,GENETICS627') ,
(15, 3, 'BIOLOGY522,GENETICS548,GENETICS564') ,
(15, 3, 'BIOLOGY522,GENETICS548,GENETICS565') ,
(15, 3, 'BIOLOGY522,GENETICS548,GENETICS566') ,
(15, 3, 'BIOLOGY522,GENETICS548,GENETICS588') ,
(15, 3, 'BIOLOGY522,GENETICS548,GENETICS605') ,
(15, 3, 'BIOLOGY522,GENETICS548,BIOCHEM612') ,
(15, 3, 'BIOLOGY522,GENETICS548,BIOCHEM620') ,
(15, 3, 'BIOLOGY522,GENETICS548,CHEM626') ,
(15, 3, 'BIOLOGY522,GENETICS548,GENETICS627') ,
(15, 3, 'BIOLOGY522,GENETICS564,GENETICS565') ,
(15, 3, 'BIOLOGY522,GENETICS564,GENETICS566') ,
(15, 3, 'BIOLOGY522,GENETICS564,GENETICS588') ,
(15, 3, 'BIOLOGY522,GENETICS564,GENETICS605') ,
(15, 3, 'BIOLOGY522,GENETICS564,BIOCHEM612') ,
(15, 3, 'BIOLOGY522,GENETICS564,BIOCHEM620') ,
(15, 3, 'BIOLOGY522,GENETICS564,CHEM626') ,
(15, 3, 'BIOLOGY522,GENETICS564,GENETICS627') ,
(15, 3, 'BIOLOGY522,GENETICS565,GENETICS566') ,
(15, 3, 'BIOLOGY522,GENETICS565,GENETICS588') ,
(15, 3, 'BIOLOGY522,GENETICS565,GENETICS605') ,
(15, 3, 'BIOLOGY522,GENETICS565,BIOCHEM612') ,
(15, 3, 'BIOLOGY522,GENETICS565,BIOCHEM620') ,
(15, 3, 'BIOLOGY522,GENETICS565,CHEM626') ,
(15, 3, 'BIOLOGY522,GENETICS565,GENETICS627') ,
(15, 3, 'BIOLOGY522,GENETICS566,GENETICS588') ,
(15, 3, 'BIOLOGY522,GENETICS566,GENETICS605') ,
(15, 3, 'BIOLOGY522,GENETICS566,BIOCHEM612') ,
(15, 3, 'BIOLOGY522,GENETICS566,BIOCHEM620') ,
(15, 3, 'BIOLOGY522,GENETICS566,CHEM626') ,
(15, 3, 'BIOLOGY522,GENETICS566,GENETICS627') ,
(15, 3, 'BIOLOGY522,GENETICS588,GENETICS605') ,
(15, 3, 'BIOLOGY522,GENETICS588,BIOCHEM612') ,
(15, 3, 'BIOLOGY522,GENETICS588,BIOCHEM620') ,
(15, 3, 'BIOLOGY522,GENETICS588,CHEM626') ,
(15, 3, 'BIOLOGY522,GENETICS588,GENETICS627') ,
(15, 3, 'BIOLOGY522,GENETICS605,BIOCHEM612') ,
(15, 3, 'BIOLOGY522,GENETICS605,BIOCHEM620') ,
(15, 3, 'BIOLOGY522,GENETICS605,CHEM626') ,
(15, 3, 'BIOLOGY522,GENETICS605,GENETICS627') ,
(15, 3, 'BIOLOGY522,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'BIOLOGY522,BIOCHEM612,CHEM626') ,
(15, 3, 'BIOLOGY522,BIOCHEM612,GENETICS627') ,
(15, 3, 'BIOLOGY522,BIOCHEM620,CHEM626') ,
(15, 3, 'BIOLOGY522,BIOCHEM620,GENETICS627') ,
(15, 3, 'BIOLOGY522,CHEM626,GENETICS627') ,
(15, 3, 'GENETICS525,GENETICS527,GENETICS528') ,
(15, 3, 'GENETICS525,GENETICS527,GENETICS548') ,
(15, 3, 'GENETICS525,GENETICS527,GENETICS564') ,
(15, 3, 'GENETICS525,GENETICS527,GENETICS565') ,
(15, 3, 'GENETICS525,GENETICS527,GENETICS566') ,
(15, 3, 'GENETICS525,GENETICS527,GENETICS588') ,
(15, 3, 'GENETICS525,GENETICS527,GENETICS605') ,
(15, 3, 'GENETICS525,GENETICS527,BIOCHEM612') ,
(15, 3, 'GENETICS525,GENETICS527,BIOCHEM620') ,
(15, 3, 'GENETICS525,GENETICS527,CHEM626') ,
(15, 3, 'GENETICS525,GENETICS527,GENETICS627') ,
(15, 3, 'GENETICS525,GENETICS528,GENETICS548') ,
(15, 3, 'GENETICS525,GENETICS528,GENETICS564') ,
(15, 3, 'GENETICS525,GENETICS528,GENETICS565') ,
(15, 3, 'GENETICS525,GENETICS528,GENETICS566') ,
(15, 3, 'GENETICS525,GENETICS528,GENETICS588') ,
(15, 3, 'GENETICS525,GENETICS528,GENETICS605') ,
(15, 3, 'GENETICS525,GENETICS528,BIOCHEM612') ,
(15, 3, 'GENETICS525,GENETICS528,BIOCHEM620') ,
(15, 3, 'GENETICS525,GENETICS528,CHEM626') ,
(15, 3, 'GENETICS525,GENETICS528,GENETICS627') ,
(15, 3, 'GENETICS525,GENETICS548,GENETICS564') ,
(15, 3, 'GENETICS525,GENETICS548,GENETICS565') ,
(15, 3, 'GENETICS525,GENETICS548,GENETICS566') ,
(15, 3, 'GENETICS525,GENETICS548,GENETICS588') ,
(15, 3, 'GENETICS525,GENETICS548,GENETICS605') ,
(15, 3, 'GENETICS525,GENETICS548,BIOCHEM612') ,
(15, 3, 'GENETICS525,GENETICS548,BIOCHEM620') ,
(15, 3, 'GENETICS525,GENETICS548,CHEM626') ,
(15, 3, 'GENETICS525,GENETICS548,GENETICS627') ,
(15, 3, 'GENETICS525,GENETICS564,GENETICS565') ,
(15, 3, 'GENETICS525,GENETICS564,GENETICS566') ,
(15, 3, 'GENETICS525,GENETICS564,GENETICS588') ,
(15, 3, 'GENETICS525,GENETICS564,GENETICS605') ,
(15, 3, 'GENETICS525,GENETICS564,BIOCHEM612') ,
(15, 3, 'GENETICS525,GENETICS564,BIOCHEM620') ,
(15, 3, 'GENETICS525,GENETICS564,CHEM626') ,
(15, 3, 'GENETICS525,GENETICS564,GENETICS627') ,
(15, 3, 'GENETICS525,GENETICS565,GENETICS566') ,
(15, 3, 'GENETICS525,GENETICS565,GENETICS588') ,
(15, 3, 'GENETICS525,GENETICS565,GENETICS605') ,
(15, 3, 'GENETICS525,GENETICS565,BIOCHEM612') ,
(15, 3, 'GENETICS525,GENETICS565,BIOCHEM620') ,
(15, 3, 'GENETICS525,GENETICS565,CHEM626') ,
(15, 3, 'GENETICS525,GENETICS565,GENETICS627') ,
(15, 3, 'GENETICS525,GENETICS566,GENETICS588') ,
(15, 3, 'GENETICS525,GENETICS566,GENETICS605') ,
(15, 3, 'GENETICS525,GENETICS566,BIOCHEM612') ,
(15, 3, 'GENETICS525,GENETICS566,BIOCHEM620') ,
(15, 3, 'GENETICS525,GENETICS566,CHEM626') ,
(15, 3, 'GENETICS525,GENETICS566,GENETICS627') ,
(15, 3, 'GENETICS525,GENETICS588,GENETICS605') ,
(15, 3, 'GENETICS525,GENETICS588,BIOCHEM612') ,
(15, 3, 'GENETICS525,GENETICS588,BIOCHEM620') ,
(15, 3, 'GENETICS525,GENETICS588,CHEM626') ,
(15, 3, 'GENETICS525,GENETICS588,GENETICS627') ,
(15, 3, 'GENETICS525,GENETICS605,BIOCHEM612') ,
(15, 3, 'GENETICS525,GENETICS605,BIOCHEM620') ,
(15, 3, 'GENETICS525,GENETICS605,CHEM626') ,
(15, 3, 'GENETICS525,GENETICS605,GENETICS627') ,
(15, 3, 'GENETICS525,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'GENETICS525,BIOCHEM612,CHEM626') ,
(15, 3, 'GENETICS525,BIOCHEM612,GENETICS627') ,
(15, 3, 'GENETICS525,BIOCHEM620,CHEM626') ,
(15, 3, 'GENETICS525,BIOCHEM620,GENETICS627') ,
(15, 3, 'GENETICS525,CHEM626,GENETICS627') ,
(15, 3, 'GENETICS527,GENETICS528,GENETICS548') ,
(15, 3, 'GENETICS527,GENETICS528,GENETICS564') ,
(15, 3, 'GENETICS527,GENETICS528,GENETICS565') ,
(15, 3, 'GENETICS527,GENETICS528,GENETICS566') ,
(15, 3, 'GENETICS527,GENETICS528,GENETICS588') ,
(15, 3, 'GENETICS527,GENETICS528,GENETICS605') ,
(15, 3, 'GENETICS527,GENETICS528,BIOCHEM612') ,
(15, 3, 'GENETICS527,GENETICS528,BIOCHEM620') ,
(15, 3, 'GENETICS527,GENETICS528,CHEM626') ,
(15, 3, 'GENETICS527,GENETICS528,GENETICS627') ,
(15, 3, 'GENETICS527,GENETICS548,GENETICS564') ,
(15, 3, 'GENETICS527,GENETICS548,GENETICS565') ,
(15, 3, 'GENETICS527,GENETICS548,GENETICS566') ,
(15, 3, 'GENETICS527,GENETICS548,GENETICS588') ,
(15, 3, 'GENETICS527,GENETICS548,GENETICS605') ,
(15, 3, 'GENETICS527,GENETICS548,BIOCHEM612') ,
(15, 3, 'GENETICS527,GENETICS548,BIOCHEM620') ,
(15, 3, 'GENETICS527,GENETICS548,CHEM626') ,
(15, 3, 'GENETICS527,GENETICS548,GENETICS627') ,
(15, 3, 'GENETICS527,GENETICS564,GENETICS565') ,
(15, 3, 'GENETICS527,GENETICS564,GENETICS566') ,
(15, 3, 'GENETICS527,GENETICS564,GENETICS588') ,
(15, 3, 'GENETICS527,GENETICS564,GENETICS605') ,
(15, 3, 'GENETICS527,GENETICS564,BIOCHEM612') ,
(15, 3, 'GENETICS527,GENETICS564,BIOCHEM620') ,
(15, 3, 'GENETICS527,GENETICS564,CHEM626') ,
(15, 3, 'GENETICS527,GENETICS564,GENETICS627') ,
(15, 3, 'GENETICS527,GENETICS565,GENETICS566') ,
(15, 3, 'GENETICS527,GENETICS565,GENETICS588') ,
(15, 3, 'GENETICS527,GENETICS565,GENETICS605') ,
(15, 3, 'GENETICS527,GENETICS565,BIOCHEM612') ,
(15, 3, 'GENETICS527,GENETICS565,BIOCHEM620') ,
(15, 3, 'GENETICS527,GENETICS565,CHEM626') ,
(15, 3, 'GENETICS527,GENETICS565,GENETICS627') ,
(15, 3, 'GENETICS527,GENETICS566,GENETICS588') ,
(15, 3, 'GENETICS527,GENETICS566,GENETICS605') ,
(15, 3, 'GENETICS527,GENETICS566,BIOCHEM612') ,
(15, 3, 'GENETICS527,GENETICS566,BIOCHEM620') ,
(15, 3, 'GENETICS527,GENETICS566,CHEM626') ,
(15, 3, 'GENETICS527,GENETICS566,GENETICS627') ,
(15, 3, 'GENETICS527,GENETICS588,GENETICS605') ,
(15, 3, 'GENETICS527,GENETICS588,BIOCHEM612') ,
(15, 3, 'GENETICS527,GENETICS588,BIOCHEM620') ,
(15, 3, 'GENETICS527,GENETICS588,CHEM626') ,
(15, 3, 'GENETICS527,GENETICS588,GENETICS627') ,
(15, 3, 'GENETICS527,GENETICS605,BIOCHEM612') ,
(15, 3, 'GENETICS527,GENETICS605,BIOCHEM620') ,
(15, 3, 'GENETICS527,GENETICS605,CHEM626') ,
(15, 3, 'GENETICS527,GENETICS605,GENETICS627') ,
(15, 3, 'GENETICS527,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'GENETICS527,BIOCHEM612,CHEM626') ,
(15, 3, 'GENETICS527,BIOCHEM612,GENETICS627') ,
(15, 3, 'GENETICS527,BIOCHEM620,CHEM626') ,
(15, 3, 'GENETICS527,BIOCHEM620,GENETICS627') ,
(15, 3, 'GENETICS527,CHEM626,GENETICS627') ,
(15, 3, 'GENETICS528,GENETICS548,GENETICS564') ,
(15, 3, 'GENETICS528,GENETICS548,GENETICS565') ,
(15, 3, 'GENETICS528,GENETICS548,GENETICS566') ,
(15, 3, 'GENETICS528,GENETICS548,GENETICS588') ,
(15, 3, 'GENETICS528,GENETICS548,GENETICS605') ,
(15, 3, 'GENETICS528,GENETICS548,BIOCHEM612') ,
(15, 3, 'GENETICS528,GENETICS548,BIOCHEM620') ,
(15, 3, 'GENETICS528,GENETICS548,CHEM626') ,
(15, 3, 'GENETICS528,GENETICS548,GENETICS627') ,
(15, 3, 'GENETICS528,GENETICS564,GENETICS565') ,
(15, 3, 'GENETICS528,GENETICS564,GENETICS566') ,
(15, 3, 'GENETICS528,GENETICS564,GENETICS588') ,
(15, 3, 'GENETICS528,GENETICS564,GENETICS605') ,
(15, 3, 'GENETICS528,GENETICS564,BIOCHEM612') ,
(15, 3, 'GENETICS528,GENETICS564,BIOCHEM620') ,
(15, 3, 'GENETICS528,GENETICS564,CHEM626') ,
(15, 3, 'GENETICS528,GENETICS564,GENETICS627') ,
(15, 3, 'GENETICS528,GENETICS565,GENETICS566') ,
(15, 3, 'GENETICS528,GENETICS565,GENETICS588') ,
(15, 3, 'GENETICS528,GENETICS565,GENETICS605') ,
(15, 3, 'GENETICS528,GENETICS565,BIOCHEM612') ,
(15, 3, 'GENETICS528,GENETICS565,BIOCHEM620') ,
(15, 3, 'GENETICS528,GENETICS565,CHEM626') ,
(15, 3, 'GENETICS528,GENETICS565,GENETICS627') ,
(15, 3, 'GENETICS528,GENETICS566,GENETICS588') ,
(15, 3, 'GENETICS528,GENETICS566,GENETICS605') ,
(15, 3, 'GENETICS528,GENETICS566,BIOCHEM612') ,
(15, 3, 'GENETICS528,GENETICS566,BIOCHEM620') ,
(15, 3, 'GENETICS528,GENETICS566,CHEM626') ,
(15, 3, 'GENETICS528,GENETICS566,GENETICS627') ,
(15, 3, 'GENETICS528,GENETICS588,GENETICS605') ,
(15, 3, 'GENETICS528,GENETICS588,BIOCHEM612') ,
(15, 3, 'GENETICS528,GENETICS588,BIOCHEM620') ,
(15, 3, 'GENETICS528,GENETICS588,CHEM626') ,
(15, 3, 'GENETICS528,GENETICS588,GENETICS627') ,
(15, 3, 'GENETICS528,GENETICS605,BIOCHEM612') ,
(15, 3, 'GENETICS528,GENETICS605,BIOCHEM620') ,
(15, 3, 'GENETICS528,GENETICS605,CHEM626') ,
(15, 3, 'GENETICS528,GENETICS605,GENETICS627') ,
(15, 3, 'GENETICS528,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'GENETICS528,BIOCHEM612,CHEM626') ,
(15, 3, 'GENETICS528,BIOCHEM612,GENETICS627') ,
(15, 3, 'GENETICS528,BIOCHEM620,CHEM626') ,
(15, 3, 'GENETICS528,BIOCHEM620,GENETICS627') ,
(15, 3, 'GENETICS528,CHEM626,GENETICS627') ,
(15, 3, 'GENETICS548,GENETICS564,GENETICS565') ,
(15, 3, 'GENETICS548,GENETICS564,GENETICS566') ,
(15, 3, 'GENETICS548,GENETICS564,GENETICS588') ,
(15, 3, 'GENETICS548,GENETICS564,GENETICS605') ,
(15, 3, 'GENETICS548,GENETICS564,BIOCHEM612') ,
(15, 3, 'GENETICS548,GENETICS564,BIOCHEM620') ,
(15, 3, 'GENETICS548,GENETICS564,CHEM626') ,
(15, 3, 'GENETICS548,GENETICS564,GENETICS627') ,
(15, 3, 'GENETICS548,GENETICS565,GENETICS566') ,
(15, 3, 'GENETICS548,GENETICS565,GENETICS588') ,
(15, 3, 'GENETICS548,GENETICS565,GENETICS605') ,
(15, 3, 'GENETICS548,GENETICS565,BIOCHEM612') ,
(15, 3, 'GENETICS548,GENETICS565,BIOCHEM620') ,
(15, 3, 'GENETICS548,GENETICS565,CHEM626') ,
(15, 3, 'GENETICS548,GENETICS565,GENETICS627') ,
(15, 3, 'GENETICS548,GENETICS566,GENETICS588') ,
(15, 3, 'GENETICS548,GENETICS566,GENETICS605') ,
(15, 3, 'GENETICS548,GENETICS566,BIOCHEM612') ,
(15, 3, 'GENETICS548,GENETICS566,BIOCHEM620') ,
(15, 3, 'GENETICS548,GENETICS566,CHEM626') ,
(15, 3, 'GENETICS548,GENETICS566,GENETICS627') ,
(15, 3, 'GENETICS548,GENETICS588,GENETICS605') ,
(15, 3, 'GENETICS548,GENETICS588,BIOCHEM612') ,
(15, 3, 'GENETICS548,GENETICS588,BIOCHEM620') ,
(15, 3, 'GENETICS548,GENETICS588,CHEM626') ,
(15, 3, 'GENETICS548,GENETICS588,GENETICS627') ,
(15, 3, 'GENETICS548,GENETICS605,BIOCHEM612') ,
(15, 3, 'GENETICS548,GENETICS605,BIOCHEM620') ,
(15, 3, 'GENETICS548,GENETICS605,CHEM626') ,
(15, 3, 'GENETICS548,GENETICS605,GENETICS627') ,
(15, 3, 'GENETICS548,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'GENETICS548,BIOCHEM612,CHEM626') ,
(15, 3, 'GENETICS548,BIOCHEM612,GENETICS627') ,
(15, 3, 'GENETICS548,BIOCHEM620,CHEM626') ,
(15, 3, 'GENETICS548,BIOCHEM620,GENETICS627') ,
(15, 3, 'GENETICS548,CHEM626,GENETICS627') ,
(15, 3, 'GENETICS564,GENETICS565,GENETICS566') ,
(15, 3, 'GENETICS564,GENETICS565,GENETICS588') ,
(15, 3, 'GENETICS564,GENETICS565,GENETICS605') ,
(15, 3, 'GENETICS564,GENETICS565,BIOCHEM612') ,
(15, 3, 'GENETICS564,GENETICS565,BIOCHEM620') ,
(15, 3, 'GENETICS564,GENETICS565,CHEM626') ,
(15, 3, 'GENETICS564,GENETICS565,GENETICS627') ,
(15, 3, 'GENETICS564,GENETICS566,GENETICS588') ,
(15, 3, 'GENETICS564,GENETICS566,GENETICS605') ,
(15, 3, 'GENETICS564,GENETICS566,BIOCHEM612') ,
(15, 3, 'GENETICS564,GENETICS566,BIOCHEM620') ,
(15, 3, 'GENETICS564,GENETICS566,CHEM626') ,
(15, 3, 'GENETICS564,GENETICS566,GENETICS627') ,
(15, 3, 'GENETICS564,GENETICS588,GENETICS605') ,
(15, 3, 'GENETICS564,GENETICS588,BIOCHEM612') ,
(15, 3, 'GENETICS564,GENETICS588,BIOCHEM620') ,
(15, 3, 'GENETICS564,GENETICS588,CHEM626') ,
(15, 3, 'GENETICS564,GENETICS588,GENETICS627') ,
(15, 3, 'GENETICS564,GENETICS605,BIOCHEM612') ,
(15, 3, 'GENETICS564,GENETICS605,BIOCHEM620') ,
(15, 3, 'GENETICS564,GENETICS605,CHEM626') ,
(15, 3, 'GENETICS564,GENETICS605,GENETICS627') ,
(15, 3, 'GENETICS564,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'GENETICS564,BIOCHEM612,CHEM626') ,
(15, 3, 'GENETICS564,BIOCHEM612,GENETICS627') ,
(15, 3, 'GENETICS564,BIOCHEM620,CHEM626') ,
(15, 3, 'GENETICS564,BIOCHEM620,GENETICS627') ,
(15, 3, 'GENETICS564,CHEM626,GENETICS627') ,
(15, 3, 'GENETICS565,GENETICS566,GENETICS588') ,
(15, 3, 'GENETICS565,GENETICS566,GENETICS605') ,
(15, 3, 'GENETICS565,GENETICS566,BIOCHEM612') ,
(15, 3, 'GENETICS565,GENETICS566,BIOCHEM620') ,
(15, 3, 'GENETICS565,GENETICS566,CHEM626') ,
(15, 3, 'GENETICS565,GENETICS566,GENETICS627') ,
(15, 3, 'GENETICS565,GENETICS588,GENETICS605') ,
(15, 3, 'GENETICS565,GENETICS588,BIOCHEM612') ,
(15, 3, 'GENETICS565,GENETICS588,BIOCHEM620') ,
(15, 3, 'GENETICS565,GENETICS588,CHEM626') ,
(15, 3, 'GENETICS565,GENETICS588,GENETICS627') ,
(15, 3, 'GENETICS565,GENETICS605,BIOCHEM612') ,
(15, 3, 'GENETICS565,GENETICS605,BIOCHEM620') ,
(15, 3, 'GENETICS565,GENETICS605,CHEM626') ,
(15, 3, 'GENETICS565,GENETICS605,GENETICS627') ,
(15, 3, 'GENETICS565,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'GENETICS565,BIOCHEM612,CHEM626') ,
(15, 3, 'GENETICS565,BIOCHEM612,GENETICS627') ,
(15, 3, 'GENETICS565,BIOCHEM620,CHEM626') ,
(15, 3, 'GENETICS565,BIOCHEM620,GENETICS627') ,
(15, 3, 'GENETICS565,CHEM626,GENETICS627') ,
(15, 3, 'GENETICS566,GENETICS588,GENETICS605') ,
(15, 3, 'GENETICS566,GENETICS588,BIOCHEM612') ,
(15, 3, 'GENETICS566,GENETICS588,BIOCHEM620') ,
(15, 3, 'GENETICS566,GENETICS588,CHEM626') ,
(15, 3, 'GENETICS566,GENETICS588,GENETICS627') ,
(15, 3, 'GENETICS566,GENETICS605,BIOCHEM612') ,
(15, 3, 'GENETICS566,GENETICS605,BIOCHEM620') ,
(15, 3, 'GENETICS566,GENETICS605,CHEM626') ,
(15, 3, 'GENETICS566,GENETICS605,GENETICS627') ,
(15, 3, 'GENETICS566,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'GENETICS566,BIOCHEM612,CHEM626') ,
(15, 3, 'GENETICS566,BIOCHEM612,GENETICS627') ,
(15, 3, 'GENETICS566,BIOCHEM620,CHEM626') ,
(15, 3, 'GENETICS566,BIOCHEM620,GENETICS627') ,
(15, 3, 'GENETICS566,CHEM626,GENETICS627') ,
(15, 3, 'GENETICS588,GENETICS605,BIOCHEM612') ,
(15, 3, 'GENETICS588,GENETICS605,BIOCHEM620') ,
(15, 3, 'GENETICS588,GENETICS605,CHEM626') ,
(15, 3, 'GENETICS588,GENETICS605,GENETICS627') ,
(15, 3, 'GENETICS588,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'GENETICS588,BIOCHEM612,CHEM626') ,
(15, 3, 'GENETICS588,BIOCHEM612,GENETICS627') ,
(15, 3, 'GENETICS588,BIOCHEM620,CHEM626') ,
(15, 3, 'GENETICS588,BIOCHEM620,GENETICS627') ,
(15, 3, 'GENETICS588,CHEM626,GENETICS627') ,
(15, 3, 'GENETICS605,BIOCHEM612,BIOCHEM620') ,
(15, 3, 'GENETICS605,BIOCHEM612,CHEM626') ,
(15, 3, 'GENETICS605,BIOCHEM612,GENETICS627') ,
(15, 3, 'GENETICS605,BIOCHEM620,CHEM626') ,
(15, 3, 'GENETICS605,BIOCHEM620,GENETICS627') ,
(15, 3, 'GENETICS605,CHEM626,GENETICS627') ,
(15, 3, 'BIOCHEM612,BIOCHEM620,CHEM626') ,
(15, 3, 'BIOCHEM612,BIOCHEM620,GENETICS627') ,
(15, 3, 'BIOCHEM612,CHEM626,GENETICS627') ,
(15, 3, 'BIOCHEM620,CHEM626,GENETICS627') ,
    (18, 2, 'BIOCHEM508,BIOCHEM560') ,
(18, 2, 'BIOCHEM508,CHEM344') ,
(18, 2, 'BIOCHEM508,CHEM345') ,
(18, 2, 'BIOCHEM508,BIOCHEM575') ,
(18, 2, 'BIOCHEM508,BIOCORE485') ,
(18, 2, 'BIOCHEM508,BIOCORE587') ,
(18, 2, 'BIOCHEM508,ANTHRO410') ,
(18, 2, 'BIOCHEM508,BOTANY563') ,
(18, 2, 'BIOCHEM508,MICROBIO303') ,
(18, 2, 'BIOCHEM508,MICROBIO304') ,
(18, 2, 'BIOCHEM508,MICROBIO470') ,
(18, 2, 'BIOCHEM508,MICROBIO545') ,
(18, 2, 'BIOCHEM508,MICROBIO632') ,
(18, 2, 'BIOCHEM508,MMI341') ,
(18, 2, 'BIOCHEM508,MMI528') ,
(18, 2, 'BIOCHEM508,PHMSCI490') ,
(18, 2, 'BIOCHEM508,PLPATH622') ,
(18, 2, 'BIOCHEM508,MMI640') ,
(18, 2, 'BIOCHEM508,ENVIRST360') ,
(18, 2, 'BIOCHEM508,ZOOLOGY425') ,
(18, 2, 'BIOCHEM508,ZOOLOGY470') ,
(18, 2, 'BIOCHEM508,ZOOLOGY555') ,
(18, 2, 'BIOCHEM508,ZOOLOGY570') ,
(18, 2, 'BIOCHEM508,AGRONOMY338') ,
(18, 2, 'BIOCHEM508,AGRONOMY340') ,
(18, 2, 'BIOCHEM508,AGRONOMY360') ,
(18, 2, 'BIOCHEM508,AGRONOMY501') ,
(18, 2, 'BIOCHEM508,AGRONOMY502') ,
(18, 2, 'BIOCHEM508,ANSCI361') ,
(18, 2, 'BIOCHEM508,ANSCI362') ,
(18, 2, 'BIOCHEM508,ANSCI363') ,
(18, 2, 'BIOCHEM508,BOTANY505') ,
(18, 2, 'BIOCHEM508,BMI576') ,
(18, 2, 'BIOCHEM560,CHEM344') ,
(18, 2, 'BIOCHEM560,CHEM345') ,
(18, 2, 'BIOCHEM560,BIOCHEM575') ,
(18, 2, 'BIOCHEM560,BIOCORE485') ,
(18, 2, 'BIOCHEM560,BIOCORE587') ,
(18, 2, 'BIOCHEM560,ANTHRO410') ,
(18, 2, 'BIOCHEM560,BOTANY563') ,
(18, 2, 'BIOCHEM560,MICROBIO303') ,
(18, 2, 'BIOCHEM560,MICROBIO304') ,
(18, 2, 'BIOCHEM560,MICROBIO470') ,
(18, 2, 'BIOCHEM560,MICROBIO545') ,
(18, 2, 'BIOCHEM560,MICROBIO632') ,
(18, 2, 'BIOCHEM560,MMI341') ,
(18, 2, 'BIOCHEM560,MMI528') ,
(18, 2, 'BIOCHEM560,PHMSCI490') ,
(18, 2, 'BIOCHEM560,PLPATH622') ,
(18, 2, 'BIOCHEM560,MMI640') ,
(18, 2, 'BIOCHEM560,ENVIRST360') ,
(18, 2, 'BIOCHEM560,ZOOLOGY425') ,
(18, 2, 'BIOCHEM560,ZOOLOGY470') ,
(18, 2, 'BIOCHEM560,ZOOLOGY555') ,
(18, 2, 'BIOCHEM560,ZOOLOGY570') ,
(18, 2, 'BIOCHEM560,AGRONOMY338') ,
(18, 2, 'BIOCHEM560,AGRONOMY340') ,
(18, 2, 'BIOCHEM560,AGRONOMY360') ,
(18, 2, 'BIOCHEM560,AGRONOMY501') ,
(18, 2, 'BIOCHEM560,AGRONOMY502') ,
(18, 2, 'BIOCHEM560,ANSCI361') ,
(18, 2, 'BIOCHEM560,ANSCI362') ,
(18, 2, 'BIOCHEM560,ANSCI363') ,
(18, 2, 'BIOCHEM560,BOTANY505') ,
(18, 2, 'BIOCHEM560,BMI576') ,
(18, 2, 'CHEM344,CHEM345') ,
(18, 2, 'CHEM344,BIOCHEM575') ,
(18, 2, 'CHEM344,BIOCORE485') ,
(18, 2, 'CHEM344,BIOCORE587') ,
(18, 2, 'CHEM344,ANTHRO410') ,
(18, 2, 'CHEM344,BOTANY563') ,
(18, 2, 'CHEM344,MICROBIO303') ,
(18, 2, 'CHEM344,MICROBIO304') ,
(18, 2, 'CHEM344,MICROBIO470') ,
(18, 2, 'CHEM344,MICROBIO545') ,
(18, 2, 'CHEM344,MICROBIO632') ,
(18, 2, 'CHEM344,MMI341') ,
(18, 2, 'CHEM344,MMI528') ,
(18, 2, 'CHEM344,PHMSCI490') ,
(18, 2, 'CHEM344,PLPATH622') ,
(18, 2, 'CHEM344,MMI640') ,
(18, 2, 'CHEM344,ENVIRST360') ,
(18, 2, 'CHEM344,ZOOLOGY425') ,
(18, 2, 'CHEM344,ZOOLOGY470') ,
(18, 2, 'CHEM344,ZOOLOGY555') ,
(18, 2, 'CHEM344,ZOOLOGY570') ,
(18, 2, 'CHEM344,AGRONOMY338') ,
(18, 2, 'CHEM344,AGRONOMY340') ,
(18, 2, 'CHEM344,AGRONOMY360') ,
(18, 2, 'CHEM344,AGRONOMY501') ,
(18, 2, 'CHEM344,AGRONOMY502') ,
(18, 2, 'CHEM344,ANSCI361') ,
(18, 2, 'CHEM344,ANSCI362') ,
(18, 2, 'CHEM344,ANSCI363') ,
(18, 2, 'CHEM344,BOTANY505') ,
(18, 2, 'CHEM344,BMI576') ,
(18, 2, 'CHEM345,BIOCHEM575') ,
(18, 2, 'CHEM345,BIOCORE485') ,
(18, 2, 'CHEM345,BIOCORE587') ,
(18, 2, 'CHEM345,ANTHRO410') ,
(18, 2, 'CHEM345,BOTANY563') ,
(18, 2, 'CHEM345,MICROBIO303') ,
(18, 2, 'CHEM345,MICROBIO304') ,
(18, 2, 'CHEM345,MICROBIO470') ,
(18, 2, 'CHEM345,MICROBIO545') ,
(18, 2, 'CHEM345,MICROBIO632') ,
(18, 2, 'CHEM345,MMI341') ,
(18, 2, 'CHEM345,MMI528') ,
(18, 2, 'CHEM345,PHMSCI490') ,
(18, 2, 'CHEM345,PLPATH622') ,
(18, 2, 'CHEM345,MMI640') ,
(18, 2, 'CHEM345,ENVIRST360') ,
(18, 2, 'CHEM345,ZOOLOGY425') ,
(18, 2, 'CHEM345,ZOOLOGY470') ,
(18, 2, 'CHEM345,ZOOLOGY555') ,
(18, 2, 'CHEM345,ZOOLOGY570') ,
(18, 2, 'CHEM345,AGRONOMY338') ,
(18, 2, 'CHEM345,AGRONOMY340') ,
(18, 2, 'CHEM345,AGRONOMY360') ,
(18, 2, 'CHEM345,AGRONOMY501') ,
(18, 2, 'CHEM345,AGRONOMY502') ,
(18, 2, 'CHEM345,ANSCI361') ,
(18, 2, 'CHEM345,ANSCI362') ,
(18, 2, 'CHEM345,ANSCI363') ,
(18, 2, 'CHEM345,BOTANY505') ,
(18, 2, 'CHEM345,BMI576') ,
(18, 2, 'BIOCHEM575,BIOCORE485') ,
(18, 2, 'BIOCHEM575,BIOCORE587') ,
(18, 2, 'BIOCHEM575,ANTHRO410') ,
(18, 2, 'BIOCHEM575,BOTANY563') ,
(18, 2, 'BIOCHEM575,MICROBIO303') ,
(18, 2, 'BIOCHEM575,MICROBIO304') ,
(18, 2, 'BIOCHEM575,MICROBIO470') ,
(18, 2, 'BIOCHEM575,MICROBIO545') ,
(18, 2, 'BIOCHEM575,MICROBIO632') ,
(18, 2, 'BIOCHEM575,MMI341') ,
(18, 2, 'BIOCHEM575,MMI528') ,
(18, 2, 'BIOCHEM575,PHMSCI490') ,
(18, 2, 'BIOCHEM575,PLPATH622') ,
(18, 2, 'BIOCHEM575,MMI640') ,
(18, 2, 'BIOCHEM575,ENVIRST360') ,
(18, 2, 'BIOCHEM575,ZOOLOGY425') ,
(18, 2, 'BIOCHEM575,ZOOLOGY470') ,
(18, 2, 'BIOCHEM575,ZOOLOGY555') ,
(18, 2, 'BIOCHEM575,ZOOLOGY570') ,
(18, 2, 'BIOCHEM575,AGRONOMY338') ,
(18, 2, 'BIOCHEM575,AGRONOMY340') ,
(18, 2, 'BIOCHEM575,AGRONOMY360') ,
(18, 2, 'BIOCHEM575,AGRONOMY501') ,
(18, 2, 'BIOCHEM575,AGRONOMY502') ,
(18, 2, 'BIOCHEM575,ANSCI361') ,
(18, 2, 'BIOCHEM575,ANSCI362') ,
(18, 2, 'BIOCHEM575,ANSCI363') ,
(18, 2, 'BIOCHEM575,BOTANY505') ,
(18, 2, 'BIOCHEM575,BMI576') ,
(18, 2, 'BIOCORE485,BIOCORE587') ,
(18, 2, 'BIOCORE485,ANTHRO410') ,
(18, 2, 'BIOCORE485,BOTANY563') ,
(18, 2, 'BIOCORE485,MICROBIO303') ,
(18, 2, 'BIOCORE485,MICROBIO304') ,
(18, 2, 'BIOCORE485,MICROBIO470') ,
(18, 2, 'BIOCORE485,MICROBIO545') ,
(18, 2, 'BIOCORE485,MICROBIO632') ,
(18, 2, 'BIOCORE485,MMI341') ,
(18, 2, 'BIOCORE485,MMI528') ,
(18, 2, 'BIOCORE485,PHMSCI490') ,
(18, 2, 'BIOCORE485,PLPATH622') ,
(18, 2, 'BIOCORE485,MMI640') ,
(18, 2, 'BIOCORE485,ENVIRST360') ,
(18, 2, 'BIOCORE485,ZOOLOGY425') ,
(18, 2, 'BIOCORE485,ZOOLOGY470') ,
(18, 2, 'BIOCORE485,ZOOLOGY555') ,
(18, 2, 'BIOCORE485,ZOOLOGY570') ,
(18, 2, 'BIOCORE485,AGRONOMY338') ,
(18, 2, 'BIOCORE485,AGRONOMY340') ,
(18, 2, 'BIOCORE485,AGRONOMY360') ,
(18, 2, 'BIOCORE485,AGRONOMY501') ,
(18, 2, 'BIOCORE485,AGRONOMY502') ,
(18, 2, 'BIOCORE485,ANSCI361') ,
(18, 2, 'BIOCORE485,ANSCI362') ,
(18, 2, 'BIOCORE485,ANSCI363') ,
(18, 2, 'BIOCORE485,BOTANY505') ,
(18, 2, 'BIOCORE485,BMI576') ,
(18, 2, 'BIOCORE587,ANTHRO410') ,
(18, 2, 'BIOCORE587,BOTANY563') ,
(18, 2, 'BIOCORE587,MICROBIO303') ,
(18, 2, 'BIOCORE587,MICROBIO304') ,
(18, 2, 'BIOCORE587,MICROBIO470') ,
(18, 2, 'BIOCORE587,MICROBIO545') ,
(18, 2, 'BIOCORE587,MICROBIO632') ,
(18, 2, 'BIOCORE587,MMI341') ,
(18, 2, 'BIOCORE587,MMI528') ,
(18, 2, 'BIOCORE587,PHMSCI490') ,
(18, 2, 'BIOCORE587,PLPATH622') ,
(18, 2, 'BIOCORE587,MMI640') ,
(18, 2, 'BIOCORE587,ENVIRST360') ,
(18, 2, 'BIOCORE587,ZOOLOGY425') ,
(18, 2, 'BIOCORE587,ZOOLOGY470') ,
(18, 2, 'BIOCORE587,ZOOLOGY555') ,
(18, 2, 'BIOCORE587,ZOOLOGY570') ,
(18, 2, 'BIOCORE587,AGRONOMY338') ,
(18, 2, 'BIOCORE587,AGRONOMY340') ,
(18, 2, 'BIOCORE587,AGRONOMY360') ,
(18, 2, 'BIOCORE587,AGRONOMY501') ,
(18, 2, 'BIOCORE587,AGRONOMY502') ,
(18, 2, 'BIOCORE587,ANSCI361') ,
(18, 2, 'BIOCORE587,ANSCI362') ,
(18, 2, 'BIOCORE587,ANSCI363') ,
(18, 2, 'BIOCORE587,BOTANY505') ,
(18, 2, 'BIOCORE587,BMI576') ,
(18, 2, 'ANTHRO410,BOTANY563') ,
(18, 2, 'ANTHRO410,MICROBIO303') ,
(18, 2, 'ANTHRO410,MICROBIO304') ,
(18, 2, 'ANTHRO410,MICROBIO470') ,
(18, 2, 'ANTHRO410,MICROBIO545') ,
(18, 2, 'ANTHRO410,MICROBIO632') ,
(18, 2, 'ANTHRO410,MMI341') ,
(18, 2, 'ANTHRO410,MMI528') ,
(18, 2, 'ANTHRO410,PHMSCI490') ,
(18, 2, 'ANTHRO410,PLPATH622') ,
(18, 2, 'ANTHRO410,MMI640') ,
(18, 2, 'ANTHRO410,ENVIRST360') ,
(18, 2, 'ANTHRO410,ZOOLOGY425') ,
(18, 2, 'ANTHRO410,ZOOLOGY470') ,
(18, 2, 'ANTHRO410,ZOOLOGY555') ,
(18, 2, 'ANTHRO410,ZOOLOGY570') ,
(18, 2, 'ANTHRO410,AGRONOMY338') ,
(18, 2, 'ANTHRO410,AGRONOMY340') ,
(18, 2, 'ANTHRO410,AGRONOMY360') ,
(18, 2, 'ANTHRO410,AGRONOMY501') ,
(18, 2, 'ANTHRO410,AGRONOMY502') ,
(18, 2, 'ANTHRO410,ANSCI361') ,
(18, 2, 'ANTHRO410,ANSCI362') ,
(18, 2, 'ANTHRO410,ANSCI363') ,
(18, 2, 'ANTHRO410,BOTANY505') ,
(18, 2, 'ANTHRO410,BMI576') ,
(18, 2, 'BOTANY563,MICROBIO303') ,
(18, 2, 'BOTANY563,MICROBIO304') ,
(18, 2, 'BOTANY563,MICROBIO470') ,
(18, 2, 'BOTANY563,MICROBIO545') ,
(18, 2, 'BOTANY563,MICROBIO632') ,
(18, 2, 'BOTANY563,MMI341') ,
(18, 2, 'BOTANY563,MMI528') ,
(18, 2, 'BOTANY563,PHMSCI490') ,
(18, 2, 'BOTANY563,PLPATH622') ,
(18, 2, 'BOTANY563,MMI640') ,
(18, 2, 'BOTANY563,ENVIRST360') ,
(18, 2, 'BOTANY563,ZOOLOGY425') ,
(18, 2, 'BOTANY563,ZOOLOGY470') ,
(18, 2, 'BOTANY563,ZOOLOGY555') ,
(18, 2, 'BOTANY563,ZOOLOGY570') ,
(18, 2, 'BOTANY563,AGRONOMY338') ,
(18, 2, 'BOTANY563,AGRONOMY340') ,
(18, 2, 'BOTANY563,AGRONOMY360') ,
(18, 2, 'BOTANY563,AGRONOMY501') ,
(18, 2, 'BOTANY563,AGRONOMY502') ,
(18, 2, 'BOTANY563,ANSCI361') ,
(18, 2, 'BOTANY563,ANSCI362') ,
(18, 2, 'BOTANY563,ANSCI363') ,
(18, 2, 'BOTANY563,BOTANY505') ,
(18, 2, 'BOTANY563,BMI576') ,
(18, 2, 'MICROBIO303,MICROBIO304') ,
(18, 2, 'MICROBIO303,MICROBIO470') ,
(18, 2, 'MICROBIO303,MICROBIO545') ,
(18, 2, 'MICROBIO303,MICROBIO632') ,
(18, 2, 'MICROBIO303,MMI341') ,
(18, 2, 'MICROBIO303,MMI528') ,
(18, 2, 'MICROBIO303,PHMSCI490') ,
(18, 2, 'MICROBIO303,PLPATH622') ,
(18, 2, 'MICROBIO303,MMI640') ,
(18, 2, 'MICROBIO303,ENVIRST360') ,
(18, 2, 'MICROBIO303,ZOOLOGY425') ,
(18, 2, 'MICROBIO303,ZOOLOGY470') ,
(18, 2, 'MICROBIO303,ZOOLOGY555') ,
(18, 2, 'MICROBIO303,ZOOLOGY570') ,
(18, 2, 'MICROBIO303,AGRONOMY338') ,
(18, 2, 'MICROBIO303,AGRONOMY340') ,
(18, 2, 'MICROBIO303,AGRONOMY360') ,
(18, 2, 'MICROBIO303,AGRONOMY501') ,
(18, 2, 'MICROBIO303,AGRONOMY502') ,
(18, 2, 'MICROBIO303,ANSCI361') ,
(18, 2, 'MICROBIO303,ANSCI362') ,
(18, 2, 'MICROBIO303,ANSCI363') ,
(18, 2, 'MICROBIO303,BOTANY505') ,
(18, 2, 'MICROBIO303,BMI576') ,
(18, 2, 'MICROBIO304,MICROBIO470') ,
(18, 2, 'MICROBIO304,MICROBIO545') ,
(18, 2, 'MICROBIO304,MICROBIO632') ,
(18, 2, 'MICROBIO304,MMI341') ,
(18, 2, 'MICROBIO304,MMI528') ,
(18, 2, 'MICROBIO304,PHMSCI490') ,
(18, 2, 'MICROBIO304,PLPATH622') ,
(18, 2, 'MICROBIO304,MMI640') ,
(18, 2, 'MICROBIO304,ENVIRST360') ,
(18, 2, 'MICROBIO304,ZOOLOGY425') ,
(18, 2, 'MICROBIO304,ZOOLOGY470') ,
(18, 2, 'MICROBIO304,ZOOLOGY555') ,
(18, 2, 'MICROBIO304,ZOOLOGY570') ,
(18, 2, 'MICROBIO304,AGRONOMY338') ,
(18, 2, 'MICROBIO304,AGRONOMY340') ,
(18, 2, 'MICROBIO304,AGRONOMY360') ,
(18, 2, 'MICROBIO304,AGRONOMY501') ,
(18, 2, 'MICROBIO304,AGRONOMY502') ,
(18, 2, 'MICROBIO304,ANSCI361') ,
(18, 2, 'MICROBIO304,ANSCI362') ,
(18, 2, 'MICROBIO304,ANSCI363') ,
(18, 2, 'MICROBIO304,BOTANY505') ,
(18, 2, 'MICROBIO304,BMI576') ,
(18, 2, 'MICROBIO470,MICROBIO545') ,
(18, 2, 'MICROBIO470,MICROBIO632') ,
(18, 2, 'MICROBIO470,MMI341') ,
(18, 2, 'MICROBIO470,MMI528') ,
(18, 2, 'MICROBIO470,PHMSCI490') ,
(18, 2, 'MICROBIO470,PLPATH622') ,
(18, 2, 'MICROBIO470,MMI640') ,
(18, 2, 'MICROBIO470,ENVIRST360') ,
(18, 2, 'MICROBIO470,ZOOLOGY425') ,
(18, 2, 'MICROBIO470,ZOOLOGY470') ,
(18, 2, 'MICROBIO470,ZOOLOGY555') ,
(18, 2, 'MICROBIO470,ZOOLOGY570') ,
(18, 2, 'MICROBIO470,AGRONOMY338') ,
(18, 2, 'MICROBIO470,AGRONOMY340') ,
(18, 2, 'MICROBIO470,AGRONOMY360') ,
(18, 2, 'MICROBIO470,AGRONOMY501') ,
(18, 2, 'MICROBIO470,AGRONOMY502') ,
(18, 2, 'MICROBIO470,ANSCI361') ,
(18, 2, 'MICROBIO470,ANSCI362') ,
(18, 2, 'MICROBIO470,ANSCI363') ,
(18, 2, 'MICROBIO470,BOTANY505') ,
(18, 2, 'MICROBIO470,BMI576') ,
(18, 2, 'MICROBIO545,MICROBIO632') ,
(18, 2, 'MICROBIO545,MMI341') ,
(18, 2, 'MICROBIO545,MMI528') ,
(18, 2, 'MICROBIO545,PHMSCI490') ,
(18, 2, 'MICROBIO545,PLPATH622') ,
(18, 2, 'MICROBIO545,MMI640') ,
(18, 2, 'MICROBIO545,ENVIRST360') ,
(18, 2, 'MICROBIO545,ZOOLOGY425') ,
(18, 2, 'MICROBIO545,ZOOLOGY470') ,
(18, 2, 'MICROBIO545,ZOOLOGY555') ,
(18, 2, 'MICROBIO545,ZOOLOGY570') ,
(18, 2, 'MICROBIO545,AGRONOMY338') ,
(18, 2, 'MICROBIO545,AGRONOMY340') ,
(18, 2, 'MICROBIO545,AGRONOMY360') ,
(18, 2, 'MICROBIO545,AGRONOMY501') ,
(18, 2, 'MICROBIO545,AGRONOMY502') ,
(18, 2, 'MICROBIO545,ANSCI361') ,
(18, 2, 'MICROBIO545,ANSCI362') ,
(18, 2, 'MICROBIO545,ANSCI363') ,
(18, 2, 'MICROBIO545,BOTANY505') ,
(18, 2, 'MICROBIO545,BMI576') ,
(18, 2, 'MICROBIO632,MMI341') ,
(18, 2, 'MICROBIO632,MMI528') ,
(18, 2, 'MICROBIO632,PHMSCI490') ,
(18, 2, 'MICROBIO632,PLPATH622') ,
(18, 2, 'MICROBIO632,MMI640') ,
(18, 2, 'MICROBIO632,ENVIRST360') ,
(18, 2, 'MICROBIO632,ZOOLOGY425') ,
(18, 2, 'MICROBIO632,ZOOLOGY470') ,
(18, 2, 'MICROBIO632,ZOOLOGY555') ,
(18, 2, 'MICROBIO632,ZOOLOGY570') ,
(18, 2, 'MICROBIO632,AGRONOMY338') ,
(18, 2, 'MICROBIO632,AGRONOMY340') ,
(18, 2, 'MICROBIO632,AGRONOMY360') ,
(18, 2, 'MICROBIO632,AGRONOMY501') ,
(18, 2, 'MICROBIO632,AGRONOMY502') ,
(18, 2, 'MICROBIO632,ANSCI361') ,
(18, 2, 'MICROBIO632,ANSCI362') ,
(18, 2, 'MICROBIO632,ANSCI363') ,
(18, 2, 'MICROBIO632,BOTANY505') ,
(18, 2, 'MICROBIO632,BMI576') ,
(18, 2, 'MMI341,MMI528') ,
(18, 2, 'MMI341,PHMSCI490') ,
(18, 2, 'MMI341,PLPATH622') ,
(18, 2, 'MMI341,MMI640') ,
(18, 2, 'MMI341,ENVIRST360') ,
(18, 2, 'MMI341,ZOOLOGY425') ,
(18, 2, 'MMI341,ZOOLOGY470') ,
(18, 2, 'MMI341,ZOOLOGY555') ,
(18, 2, 'MMI341,ZOOLOGY570') ,
(18, 2, 'MMI341,AGRONOMY338') ,
(18, 2, 'MMI341,AGRONOMY340') ,
(18, 2, 'MMI341,AGRONOMY360') ,
(18, 2, 'MMI341,AGRONOMY501') ,
(18, 2, 'MMI341,AGRONOMY502') ,
(18, 2, 'MMI341,ANSCI361') ,
(18, 2, 'MMI341,ANSCI362') ,
(18, 2, 'MMI341,ANSCI363') ,
(18, 2, 'MMI341,BOTANY505') ,
(18, 2, 'MMI341,BMI576') ,
(18, 2, 'MMI528,PHMSCI490') ,
(18, 2, 'MMI528,PLPATH622') ,
(18, 2, 'MMI528,MMI640') ,
(18, 2, 'MMI528,ENVIRST360') ,
(18, 2, 'MMI528,ZOOLOGY425') ,
(18, 2, 'MMI528,ZOOLOGY470') ,
(18, 2, 'MMI528,ZOOLOGY555') ,
(18, 2, 'MMI528,ZOOLOGY570') ,
(18, 2, 'MMI528,AGRONOMY338') ,
(18, 2, 'MMI528,AGRONOMY340') ,
(18, 2, 'MMI528,AGRONOMY360') ,
(18, 2, 'MMI528,AGRONOMY501') ,
(18, 2, 'MMI528,AGRONOMY502') ,
(18, 2, 'MMI528,ANSCI361') ,
(18, 2, 'MMI528,ANSCI362') ,
(18, 2, 'MMI528,ANSCI363') ,
(18, 2, 'MMI528,BOTANY505') ,
(18, 2, 'MMI528,BMI576') ,
(18, 2, 'PHMSCI490,PLPATH622') ,
(18, 2, 'PHMSCI490,MMI640') ,
(18, 2, 'PHMSCI490,ENVIRST360') ,
(18, 2, 'PHMSCI490,ZOOLOGY425') ,
(18, 2, 'PHMSCI490,ZOOLOGY470') ,
(18, 2, 'PHMSCI490,ZOOLOGY555') ,
(18, 2, 'PHMSCI490,ZOOLOGY570') ,
(18, 2, 'PHMSCI490,AGRONOMY338') ,
(18, 2, 'PHMSCI490,AGRONOMY340') ,
(18, 2, 'PHMSCI490,AGRONOMY360') ,
(18, 2, 'PHMSCI490,AGRONOMY501') ,
(18, 2, 'PHMSCI490,AGRONOMY502') ,
(18, 2, 'PHMSCI490,ANSCI361') ,
(18, 2, 'PHMSCI490,ANSCI362') ,
(18, 2, 'PHMSCI490,ANSCI363') ,
(18, 2, 'PHMSCI490,BOTANY505') ,
(18, 2, 'PHMSCI490,BMI576') ,
(18, 2, 'PLPATH622,MMI640') ,
(18, 2, 'PLPATH622,ENVIRST360') ,
(18, 2, 'PLPATH622,ZOOLOGY425') ,
(18, 2, 'PLPATH622,ZOOLOGY470') ,
(18, 2, 'PLPATH622,ZOOLOGY555') ,
(18, 2, 'PLPATH622,ZOOLOGY570') ,
(18, 2, 'PLPATH622,AGRONOMY338') ,
(18, 2, 'PLPATH622,AGRONOMY340') ,
(18, 2, 'PLPATH622,AGRONOMY360') ,
(18, 2, 'PLPATH622,AGRONOMY501') ,
(18, 2, 'PLPATH622,AGRONOMY502') ,
(18, 2, 'PLPATH622,ANSCI361') ,
(18, 2, 'PLPATH622,ANSCI362') ,
(18, 2, 'PLPATH622,ANSCI363') ,
(18, 2, 'PLPATH622,BOTANY505') ,
(18, 2, 'PLPATH622,BMI576') ,
(18, 2, 'MMI640,ENVIRST360') ,
(18, 2, 'MMI640,ZOOLOGY425') ,
(18, 2, 'MMI640,ZOOLOGY470') ,
(18, 2, 'MMI640,ZOOLOGY555') ,
(18, 2, 'MMI640,ZOOLOGY570') ,
(18, 2, 'MMI640,AGRONOMY338') ,
(18, 2, 'MMI640,AGRONOMY340') ,
(18, 2, 'MMI640,AGRONOMY360') ,
(18, 2, 'MMI640,AGRONOMY501') ,
(18, 2, 'MMI640,AGRONOMY502') ,
(18, 2, 'MMI640,ANSCI361') ,
(18, 2, 'MMI640,ANSCI362') ,
(18, 2, 'MMI640,ANSCI363') ,
(18, 2, 'MMI640,BOTANY505') ,
(18, 2, 'MMI640,BMI576') ,
(18, 2, 'ENVIRST360,ZOOLOGY425') ,
(18, 2, 'ENVIRST360,ZOOLOGY470') ,
(18, 2, 'ENVIRST360,ZOOLOGY555') ,
(18, 2, 'ENVIRST360,ZOOLOGY570') ,
(18, 2, 'ENVIRST360,AGRONOMY338') ,
(18, 2, 'ENVIRST360,AGRONOMY340') ,
(18, 2, 'ENVIRST360,AGRONOMY360') ,
(18, 2, 'ENVIRST360,AGRONOMY501') ,
(18, 2, 'ENVIRST360,AGRONOMY502') ,
(18, 2, 'ENVIRST360,ANSCI361') ,
(18, 2, 'ENVIRST360,ANSCI362') ,
(18, 2, 'ENVIRST360,ANSCI363') ,
(18, 2, 'ENVIRST360,BOTANY505') ,
(18, 2, 'ENVIRST360,BMI576') ,
(18, 2, 'ZOOLOGY425,ZOOLOGY470') ,
(18, 2, 'ZOOLOGY425,ZOOLOGY555') ,
(18, 2, 'ZOOLOGY425,ZOOLOGY570') ,
(18, 2, 'ZOOLOGY425,AGRONOMY338') ,
(18, 2, 'ZOOLOGY425,AGRONOMY340') ,
(18, 2, 'ZOOLOGY425,AGRONOMY360') ,
(18, 2, 'ZOOLOGY425,AGRONOMY501') ,
(18, 2, 'ZOOLOGY425,AGRONOMY502') ,
(18, 2, 'ZOOLOGY425,ANSCI361') ,
(18, 2, 'ZOOLOGY425,ANSCI362') ,
(18, 2, 'ZOOLOGY425,ANSCI363') ,
(18, 2, 'ZOOLOGY425,BOTANY505') ,
(18, 2, 'ZOOLOGY425,BMI576') ,
(18, 2, 'ZOOLOGY470,ZOOLOGY555') ,
(18, 2, 'ZOOLOGY470,ZOOLOGY570') ,
(18, 2, 'ZOOLOGY470,AGRONOMY338') ,
(18, 2, 'ZOOLOGY470,AGRONOMY340') ,
(18, 2, 'ZOOLOGY470,AGRONOMY360') ,
(18, 2, 'ZOOLOGY470,AGRONOMY501') ,
(18, 2, 'ZOOLOGY470,AGRONOMY502') ,
(18, 2, 'ZOOLOGY470,ANSCI361') ,
(18, 2, 'ZOOLOGY470,ANSCI362') ,
(18, 2, 'ZOOLOGY470,ANSCI363') ,
(18, 2, 'ZOOLOGY470,BOTANY505') ,
(18, 2, 'ZOOLOGY470,BMI576') ,
(18, 2, 'ZOOLOGY555,ZOOLOGY570') ,
(18, 2, 'ZOOLOGY555,AGRONOMY338') ,
(18, 2, 'ZOOLOGY555,AGRONOMY340') ,
(18, 2, 'ZOOLOGY555,AGRONOMY360') ,
(18, 2, 'ZOOLOGY555,AGRONOMY501') ,
(18, 2, 'ZOOLOGY555,AGRONOMY502') ,
(18, 2, 'ZOOLOGY555,ANSCI361') ,
(18, 2, 'ZOOLOGY555,ANSCI362') ,
(18, 2, 'ZOOLOGY555,ANSCI363') ,
(18, 2, 'ZOOLOGY555,BOTANY505') ,
(18, 2, 'ZOOLOGY555,BMI576') ,
(18, 2, 'ZOOLOGY570,AGRONOMY338') ,
(18, 2, 'ZOOLOGY570,AGRONOMY340') ,
(18, 2, 'ZOOLOGY570,AGRONOMY360') ,
(18, 2, 'ZOOLOGY570,AGRONOMY501') ,
(18, 2, 'ZOOLOGY570,AGRONOMY502') ,
(18, 2, 'ZOOLOGY570,ANSCI361') ,
(18, 2, 'ZOOLOGY570,ANSCI362') ,
(18, 2, 'ZOOLOGY570,ANSCI363') ,
(18, 2, 'ZOOLOGY570,BOTANY505') ,
(18, 2, 'ZOOLOGY570,BMI576') ,
(18, 2, 'AGRONOMY338,AGRONOMY340') ,
(18, 2, 'AGRONOMY338,AGRONOMY360') ,
(18, 2, 'AGRONOMY338,AGRONOMY501') ,
(18, 2, 'AGRONOMY338,AGRONOMY502') ,
(18, 2, 'AGRONOMY338,ANSCI361') ,
(18, 2, 'AGRONOMY338,ANSCI362') ,
(18, 2, 'AGRONOMY338,ANSCI363') ,
(18, 2, 'AGRONOMY338,BOTANY505') ,
(18, 2, 'AGRONOMY338,BMI576') ,
(18, 2, 'AGRONOMY340,AGRONOMY360') ,
(18, 2, 'AGRONOMY340,AGRONOMY501') ,
(18, 2, 'AGRONOMY340,AGRONOMY502') ,
(18, 2, 'AGRONOMY340,ANSCI361') ,
(18, 2, 'AGRONOMY340,ANSCI362') ,
(18, 2, 'AGRONOMY340,ANSCI363') ,
(18, 2, 'AGRONOMY340,BOTANY505') ,
(18, 2, 'AGRONOMY340,BMI576') ,
(18, 2, 'AGRONOMY360,AGRONOMY501') ,
(18, 2, 'AGRONOMY360,AGRONOMY502') ,
(18, 2, 'AGRONOMY360,ANSCI361') ,
(18, 2, 'AGRONOMY360,ANSCI362') ,
(18, 2, 'AGRONOMY360,ANSCI363') ,
(18, 2, 'AGRONOMY360,BOTANY505') ,
(18, 2, 'AGRONOMY360,BMI576') ,
(18, 2, 'AGRONOMY501,AGRONOMY502') ,
(18, 2, 'AGRONOMY501,ANSCI361') ,
(18, 2, 'AGRONOMY501,ANSCI362') ,
(18, 2, 'AGRONOMY501,ANSCI363') ,
(18, 2, 'AGRONOMY501,BOTANY505') ,
(18, 2, 'AGRONOMY501,BMI576') ,
(18, 2, 'AGRONOMY502,ANSCI361') ,
(18, 2, 'AGRONOMY502,ANSCI362') ,
(18, 2, 'AGRONOMY502,ANSCI363') ,
(18, 2, 'AGRONOMY502,BOTANY505') ,
(18, 2, 'AGRONOMY502,BMI576') ,
(18, 2, 'ANSCI361,ANSCI362') ,
(18, 2, 'ANSCI361,ANSCI363') ,
(18, 2, 'ANSCI361,BOTANY505') ,
(18, 2, 'ANSCI361,BMI576') ,
(18, 2, 'ANSCI362,ANSCI363') ,
(18, 2, 'ANSCI362,BOTANY505') ,
(18, 2, 'ANSCI362,BMI576') ,
(18, 2, 'ANSCI363,BOTANY505') ,
(18, 2, 'ANSCI363,BMI576') ,
(18, 2, 'BOTANY505,BMI576') ,]
    

In [22]:
gene_g = sequence_graph.VariantGraph(gene_ref)
for var in gene_variants:
    gene_g.add_complex_variant(*var)
    
gene_g.reorder_topological()
# gene_g.plot()
major_graph["genetics-genomics-bs"] = gene_g

In [23]:
with requests.get("https://guide.wisc.edu/undergraduate/letters-science/statistics/data-science-bs/#requirementstext") as r:
    soup = BeautifulSoup(r.text, 'html.parser')

    courses = []

    for element in soup.find("div", {"id":"requirementstextcontainer"}).find_all("a", {"class":"bubblelink code"}):
        course = element.text.encode('ascii', 'ignore').decode('ascii')
        if re.findall(r"[A-Z]+[0-9]+", course):
            dep,num = re.findall(r"(\D+)([0-9]+)", course)[0]
            codes = sorted(dep.split("/"))[0]+num
            codes = re.sub(r"\W", "", codes)
            courses.append(codes)
            
courses

major_alphabet["data-science-bs"] = {}
for i in range(len(courses)):
    if courses[i] not in major_alphabet["data-science-bs"]:
        major_alphabet["data-science-bs"][courses[i]] = i

In [24]:
ds_ref = "MATH221,MATH222,STAT240,STAT340,COMPSCI220,COMPSCI320,LIS461,COMPSCI532,COMPSCI400,ECON400,MATH320,COMPSCI425,COMPSCI525"
ds_l = ds_ref.split(",")
ds_break = [(ds_l[i], i+1) for i in range(len(ds_l))]
ds_break

[('MATH221', 1),
 ('MATH222', 2),
 ('STAT240', 3),
 ('STAT340', 4),
 ('COMPSCI220', 5),
 ('COMPSCI320', 6),
 ('LIS461', 7),
 ('COMPSCI532', 8),
 ('COMPSCI400', 9),
 ('ECON400', 10),
 ('MATH320', 11),
 ('COMPSCI425', 12),
 ('COMPSCI525', 13)]

In [25]:
# for i in combinations(['COMPSCI425',],2):
#     j = (12,2,",".join(i)) 
#     print(j, ",")

# for i in [
# ]:
#     j = (12,2,i) 
#     print(j, ",")

In [26]:
ds_variants = [
    (1,1,'MATH217'),
    (5,1,'COMPSCI300'),
    (7,1,'ECE570'),
    (8, 1, 'COMPSCI539') ,
(8, 1, 'COMPSCI540') ,
(8, 1, 'GENBUS656') ,
(8, 1, 'ISYE521') ,
(8, 1, 'MATH535') ,
(8, 1, 'PHYSICS361') ,
(8, 1, 'STAT451') ,
(8, 1, 'STAT453') ,
    (9, 1, 'COMPSCI412') ,
(9, 1, 'COMPSCI471') ,
(9, 1, 'COMPSCI513') ,
(9, 1, 'COMPSCI514') ,
(9, 1, 'COMPSCI524') ,
(9, 1, 'COMPSCI544') ,
(9, 1, 'COMPSCI564') ,
(9, 1, 'COMPSCI565') ,
(9, 1, 'BMI576') ,
(9, 1, 'GEOG573') ,
(9, 1, 'GEOG574') ,
(9, 1, 'MATH444') ,
(10, 1, 'ECON410') ,
(10, 1, 'ECON460') ,
(10, 1, 'GEOG579') ,
(10, 1, 'ISYE575') ,
(10, 1, 'MATH309') ,
(10, 1, 'STAT311') ,
(10, 1, 'MATH431') ,
(10, 1, 'MATH310') ,
(10, 1, 'STAT312') ,
(10, 1, 'STAT349') ,
(10, 1, 'STAT351') ,
(10, 1, 'STAT421') ,
(10, 1, 'ME424') ,
(10, 1, 'STAT436') ,
(10, 1, 'STAT443') ,
(10, 1, 'STAT456') ,
(10, 1, 'STAT461') ,
(10, 1, 'STAT575') ,
(10, 1, 'MATH531') ,
(10, 1, 'ISYE632') ,
(10, 1, 'MATH635') ,
(11, 1, 'MATH340') ,
(11, 1, 'MATH341') ,
(11, 1, 'MATH375') ,
    (12, 2, 'COMPSCI425,COMPSCI533') ,
(12, 2, 'COMPSCI425,COMPSCI559') ,
(12, 2, 'COMPSCI425,BMI567') ,
(12, 2, 'COMPSCI425,COMPSCI577') ,
(12, 2, 'COMPSCI425,ECE203') ,
(12, 2, 'COMPSCI425,ECON315') ,
(12, 2, 'COMPSCI425,ECON570') ,
(12, 2, 'COMPSCI425,ECON695') ,
(12, 2, 'COMPSCI425,GEOG378') ,
(12, 2, 'COMPSCI425,GEOG572') ,
(12, 2, 'COMPSCI425,GEOG575') ,
(12, 2, 'COMPSCI425,ISYE323') ,
(12, 2, 'COMPSCI425,ISYE412') ,
(12, 2, 'COMPSCI425,ISYE512') ,
(12, 2, 'COMPSCI425,ISYE612') ,
(12, 2, 'COMPSCI425,INFOSYS322') ,
(12, 2, 'COMPSCI425,LIS407') ,
(12, 2, 'COMPSCI425,LIS440') ,
(12, 2, 'COMPSCI425,LIS464') ,
(12, 2, 'COMPSCI425,LIS501') ,
(12, 2, 'COMPSCI425,LSC460') ,
(12, 2, 'COMPSCI425,LSC660') ,
(12, 2, 'COMPSCI425,MATH331') ,
(12, 2, 'COMPSCI425,SOC351') ,
(12, 2, 'COMPSCI425,CESOC618') ,
(12, 2, 'COMPSCI425,CESOC693') ,
(12, 2, 'COMPSCI425,SOILSCI585') ,
(12, 2, 'COMPSCI425,STAT405') ,
(12, 2, 'COMPSCI425,STAT433') ,
(12, 2, 'COMPSCI425,LIS461') ,
(12, 2, 'COMPSCI425,MATH331') ,
(12, 2, 'COMPSCI425,MATH309') ,
(12, 2, 'COMPSCI425,STAT311') ,
(12, 2, 'COMPSCI425,MATH431') ,
(12, 2, 'COMPSCI425,MATH310') ,
(12, 2, 'COMPSCI425,STAT312') ,
(12, 2, 'COMPSCI425,MATH320') ,
(12, 2, 'COMPSCI425,MATH340') ,
(12, 2, 'COMPSCI425,MATH341') ,
(12, 2, 'COMPSCI425,MATH375') ,
(12, 2, 'COMPSCI525,COMPSCI533') ,
(12, 2, 'COMPSCI525,COMPSCI559') ,
(12, 2, 'COMPSCI525,BMI567') ,
(12, 2, 'COMPSCI525,COMPSCI577') ,
(12, 2, 'COMPSCI525,ECE203') ,
(12, 2, 'COMPSCI525,ECON315') ,
(12, 2, 'COMPSCI525,ECON570') ,
(12, 2, 'COMPSCI525,ECON695') ,
(12, 2, 'COMPSCI525,GEOG378') ,
(12, 2, 'COMPSCI525,GEOG572') ,
(12, 2, 'COMPSCI525,GEOG575') ,
(12, 2, 'COMPSCI525,ISYE323') ,
(12, 2, 'COMPSCI525,ISYE412') ,
(12, 2, 'COMPSCI525,ISYE512') ,
(12, 2, 'COMPSCI525,ISYE612') ,
(12, 2, 'COMPSCI525,INFOSYS322') ,
(12, 2, 'COMPSCI525,LIS407') ,
(12, 2, 'COMPSCI525,LIS440') ,
(12, 2, 'COMPSCI525,LIS464') ,
(12, 2, 'COMPSCI525,LIS501') ,
(12, 2, 'COMPSCI525,LSC460') ,
(12, 2, 'COMPSCI525,LSC660') ,
(12, 2, 'COMPSCI525,MATH331') ,
(12, 2, 'COMPSCI525,SOC351') ,
(12, 2, 'COMPSCI525,CESOC618') ,
(12, 2, 'COMPSCI525,CESOC693') ,
(12, 2, 'COMPSCI525,SOILSCI585') ,
(12, 2, 'COMPSCI525,STAT405') ,
(12, 2, 'COMPSCI525,STAT433') ,
(12, 2, 'COMPSCI525,LIS461') ,
(12, 2, 'COMPSCI525,MATH331') ,
(12, 2, 'COMPSCI525,MATH309') ,
(12, 2, 'COMPSCI525,STAT311') ,
(12, 2, 'COMPSCI525,MATH431') ,
(12, 2, 'COMPSCI525,MATH310') ,
(12, 2, 'COMPSCI525,STAT312') ,
(12, 2, 'COMPSCI525,MATH320') ,
(12, 2, 'COMPSCI525,MATH340') ,
(12, 2, 'COMPSCI525,MATH341') ,
(12, 2, 'COMPSCI525,MATH375') ,
(12, 2, 'COMPSCI533,COMPSCI559') ,
(12, 2, 'COMPSCI533,BMI567') ,
(12, 2, 'COMPSCI533,COMPSCI577') ,
(12, 2, 'COMPSCI533,ECE203') ,
(12, 2, 'COMPSCI533,ECON315') ,
(12, 2, 'COMPSCI533,ECON570') ,
(12, 2, 'COMPSCI533,ECON695') ,
(12, 2, 'COMPSCI533,GEOG378') ,
(12, 2, 'COMPSCI533,GEOG572') ,
(12, 2, 'COMPSCI533,GEOG575') ,
(12, 2, 'COMPSCI533,ISYE323') ,
(12, 2, 'COMPSCI533,ISYE412') ,
(12, 2, 'COMPSCI533,ISYE512') ,
(12, 2, 'COMPSCI533,ISYE612') ,
(12, 2, 'COMPSCI533,INFOSYS322') ,
(12, 2, 'COMPSCI533,LIS407') ,
(12, 2, 'COMPSCI533,LIS440') ,
(12, 2, 'COMPSCI533,LIS464') ,
(12, 2, 'COMPSCI533,LIS501') ,
(12, 2, 'COMPSCI533,LSC460') ,
(12, 2, 'COMPSCI533,LSC660') ,
(12, 2, 'COMPSCI533,MATH331') ,
(12, 2, 'COMPSCI533,SOC351') ,
(12, 2, 'COMPSCI533,CESOC618') ,
(12, 2, 'COMPSCI533,CESOC693') ,
(12, 2, 'COMPSCI533,SOILSCI585') ,
(12, 2, 'COMPSCI533,STAT405') ,
(12, 2, 'COMPSCI533,STAT433') ,
(12, 2, 'COMPSCI533,LIS461') ,
(12, 2, 'COMPSCI533,MATH331') ,
(12, 2, 'COMPSCI533,MATH309') ,
(12, 2, 'COMPSCI533,STAT311') ,
(12, 2, 'COMPSCI533,MATH431') ,
(12, 2, 'COMPSCI533,MATH310') ,
(12, 2, 'COMPSCI533,STAT312') ,
(12, 2, 'COMPSCI533,MATH320') ,
(12, 2, 'COMPSCI533,MATH340') ,
(12, 2, 'COMPSCI533,MATH341') ,
(12, 2, 'COMPSCI533,MATH375') ,
(12, 2, 'COMPSCI559,BMI567') ,
(12, 2, 'COMPSCI559,COMPSCI577') ,
(12, 2, 'COMPSCI559,ECE203') ,
(12, 2, 'COMPSCI559,ECON315') ,
(12, 2, 'COMPSCI559,ECON570') ,
(12, 2, 'COMPSCI559,ECON695') ,
(12, 2, 'COMPSCI559,GEOG378') ,
(12, 2, 'COMPSCI559,GEOG572') ,
(12, 2, 'COMPSCI559,GEOG575') ,
(12, 2, 'COMPSCI559,ISYE323') ,
(12, 2, 'COMPSCI559,ISYE412') ,
(12, 2, 'COMPSCI559,ISYE512') ,
(12, 2, 'COMPSCI559,ISYE612') ,
(12, 2, 'COMPSCI559,INFOSYS322') ,
(12, 2, 'COMPSCI559,LIS407') ,
(12, 2, 'COMPSCI559,LIS440') ,
(12, 2, 'COMPSCI559,LIS464') ,
(12, 2, 'COMPSCI559,LIS501') ,
(12, 2, 'COMPSCI559,LSC460') ,
(12, 2, 'COMPSCI559,LSC660') ,
(12, 2, 'COMPSCI559,MATH331') ,
(12, 2, 'COMPSCI559,SOC351') ,
(12, 2, 'COMPSCI559,CESOC618') ,
(12, 2, 'COMPSCI559,CESOC693') ,
(12, 2, 'COMPSCI559,SOILSCI585') ,
(12, 2, 'COMPSCI559,STAT405') ,
(12, 2, 'COMPSCI559,STAT433') ,
(12, 2, 'COMPSCI559,LIS461') ,
(12, 2, 'COMPSCI559,MATH331') ,
(12, 2, 'COMPSCI559,MATH309') ,
(12, 2, 'COMPSCI559,STAT311') ,
(12, 2, 'COMPSCI559,MATH431') ,
(12, 2, 'COMPSCI559,MATH310') ,
(12, 2, 'COMPSCI559,STAT312') ,
(12, 2, 'COMPSCI559,MATH320') ,
(12, 2, 'COMPSCI559,MATH340') ,
(12, 2, 'COMPSCI559,MATH341') ,
(12, 2, 'COMPSCI559,MATH375') ,
(12, 2, 'BMI567,COMPSCI577') ,
(12, 2, 'BMI567,ECE203') ,
(12, 2, 'BMI567,ECON315') ,
(12, 2, 'BMI567,ECON570') ,
(12, 2, 'BMI567,ECON695') ,
(12, 2, 'BMI567,GEOG378') ,
(12, 2, 'BMI567,GEOG572') ,
(12, 2, 'BMI567,GEOG575') ,
(12, 2, 'BMI567,ISYE323') ,
(12, 2, 'BMI567,ISYE412') ,
(12, 2, 'BMI567,ISYE512') ,
(12, 2, 'BMI567,ISYE612') ,
(12, 2, 'BMI567,INFOSYS322') ,
(12, 2, 'BMI567,LIS407') ,
(12, 2, 'BMI567,LIS440') ,
(12, 2, 'BMI567,LIS464') ,
(12, 2, 'BMI567,LIS501') ,
(12, 2, 'BMI567,LSC460') ,
(12, 2, 'BMI567,LSC660') ,
(12, 2, 'BMI567,MATH331') ,
(12, 2, 'BMI567,SOC351') ,
(12, 2, 'BMI567,CESOC618') ,
(12, 2, 'BMI567,CESOC693') ,
(12, 2, 'BMI567,SOILSCI585') ,
(12, 2, 'BMI567,STAT405') ,
(12, 2, 'BMI567,STAT433') ,
(12, 2, 'BMI567,LIS461') ,
(12, 2, 'BMI567,MATH331') ,
(12, 2, 'BMI567,MATH309') ,
(12, 2, 'BMI567,STAT311') ,
(12, 2, 'BMI567,MATH431') ,
(12, 2, 'BMI567,MATH310') ,
(12, 2, 'BMI567,STAT312') ,
(12, 2, 'BMI567,MATH320') ,
(12, 2, 'BMI567,MATH340') ,
(12, 2, 'BMI567,MATH341') ,
(12, 2, 'BMI567,MATH375') ,
(12, 2, 'COMPSCI577,ECE203') ,
(12, 2, 'COMPSCI577,ECON315') ,
(12, 2, 'COMPSCI577,ECON570') ,
(12, 2, 'COMPSCI577,ECON695') ,
(12, 2, 'COMPSCI577,GEOG378') ,
(12, 2, 'COMPSCI577,GEOG572') ,
(12, 2, 'COMPSCI577,GEOG575') ,
(12, 2, 'COMPSCI577,ISYE323') ,
(12, 2, 'COMPSCI577,ISYE412') ,
(12, 2, 'COMPSCI577,ISYE512') ,
(12, 2, 'COMPSCI577,ISYE612') ,
(12, 2, 'COMPSCI577,INFOSYS322') ,
(12, 2, 'COMPSCI577,LIS407') ,
(12, 2, 'COMPSCI577,LIS440') ,
(12, 2, 'COMPSCI577,LIS464') ,
(12, 2, 'COMPSCI577,LIS501') ,
(12, 2, 'COMPSCI577,LSC460') ,
(12, 2, 'COMPSCI577,LSC660') ,
(12, 2, 'COMPSCI577,MATH331') ,
(12, 2, 'COMPSCI577,SOC351') ,
(12, 2, 'COMPSCI577,CESOC618') ,
(12, 2, 'COMPSCI577,CESOC693') ,
(12, 2, 'COMPSCI577,SOILSCI585') ,
(12, 2, 'COMPSCI577,STAT405') ,
(12, 2, 'COMPSCI577,STAT433') ,
(12, 2, 'COMPSCI577,LIS461') ,
(12, 2, 'COMPSCI577,MATH331') ,
(12, 2, 'COMPSCI577,MATH309') ,
(12, 2, 'COMPSCI577,STAT311') ,
(12, 2, 'COMPSCI577,MATH431') ,
(12, 2, 'COMPSCI577,MATH310') ,
(12, 2, 'COMPSCI577,STAT312') ,
(12, 2, 'COMPSCI577,MATH320') ,
(12, 2, 'COMPSCI577,MATH340') ,
(12, 2, 'COMPSCI577,MATH341') ,
(12, 2, 'COMPSCI577,MATH375') ,
(12, 2, 'ECE203,ECON315') ,
(12, 2, 'ECE203,ECON570') ,
(12, 2, 'ECE203,ECON695') ,
(12, 2, 'ECE203,GEOG378') ,
(12, 2, 'ECE203,GEOG572') ,
(12, 2, 'ECE203,GEOG575') ,
(12, 2, 'ECE203,ISYE323') ,
(12, 2, 'ECE203,ISYE412') ,
(12, 2, 'ECE203,ISYE512') ,
(12, 2, 'ECE203,ISYE612') ,
(12, 2, 'ECE203,INFOSYS322') ,
(12, 2, 'ECE203,LIS407') ,
(12, 2, 'ECE203,LIS440') ,
(12, 2, 'ECE203,LIS464') ,
(12, 2, 'ECE203,LIS501') ,
(12, 2, 'ECE203,LSC460') ,
(12, 2, 'ECE203,LSC660') ,
(12, 2, 'ECE203,MATH331') ,
(12, 2, 'ECE203,SOC351') ,
(12, 2, 'ECE203,CESOC618') ,
(12, 2, 'ECE203,CESOC693') ,
(12, 2, 'ECE203,SOILSCI585') ,
(12, 2, 'ECE203,STAT405') ,
(12, 2, 'ECE203,STAT433') ,
(12, 2, 'ECE203,LIS461') ,
(12, 2, 'ECE203,MATH331') ,
(12, 2, 'ECE203,MATH309') ,
(12, 2, 'ECE203,STAT311') ,
(12, 2, 'ECE203,MATH431') ,
(12, 2, 'ECE203,MATH310') ,
(12, 2, 'ECE203,STAT312') ,
(12, 2, 'ECE203,MATH320') ,
(12, 2, 'ECE203,MATH340') ,
(12, 2, 'ECE203,MATH341') ,
(12, 2, 'ECE203,MATH375') ,
(12, 2, 'ECON315,ECON570') ,
(12, 2, 'ECON315,ECON695') ,
(12, 2, 'ECON315,GEOG378') ,
(12, 2, 'ECON315,GEOG572') ,
(12, 2, 'ECON315,GEOG575') ,
(12, 2, 'ECON315,ISYE323') ,
(12, 2, 'ECON315,ISYE412') ,
(12, 2, 'ECON315,ISYE512') ,
(12, 2, 'ECON315,ISYE612') ,
(12, 2, 'ECON315,INFOSYS322') ,
(12, 2, 'ECON315,LIS407') ,
(12, 2, 'ECON315,LIS440') ,
(12, 2, 'ECON315,LIS464') ,
(12, 2, 'ECON315,LIS501') ,
(12, 2, 'ECON315,LSC460') ,
(12, 2, 'ECON315,LSC660') ,
(12, 2, 'ECON315,MATH331') ,
(12, 2, 'ECON315,SOC351') ,
(12, 2, 'ECON315,CESOC618') ,
(12, 2, 'ECON315,CESOC693') ,
(12, 2, 'ECON315,SOILSCI585') ,
(12, 2, 'ECON315,STAT405') ,
(12, 2, 'ECON315,STAT433') ,
(12, 2, 'ECON315,LIS461') ,
(12, 2, 'ECON315,MATH331') ,
(12, 2, 'ECON315,MATH309') ,
(12, 2, 'ECON315,STAT311') ,
(12, 2, 'ECON315,MATH431') ,
(12, 2, 'ECON315,MATH310') ,
(12, 2, 'ECON315,STAT312') ,
(12, 2, 'ECON315,MATH320') ,
(12, 2, 'ECON315,MATH340') ,
(12, 2, 'ECON315,MATH341') ,
(12, 2, 'ECON315,MATH375') ,
(12, 2, 'ECON570,ECON695') ,
(12, 2, 'ECON570,GEOG378') ,
(12, 2, 'ECON570,GEOG572') ,
(12, 2, 'ECON570,GEOG575') ,
(12, 2, 'ECON570,ISYE323') ,
(12, 2, 'ECON570,ISYE412') ,
(12, 2, 'ECON570,ISYE512') ,
(12, 2, 'ECON570,ISYE612') ,
(12, 2, 'ECON570,INFOSYS322') ,
(12, 2, 'ECON570,LIS407') ,
(12, 2, 'ECON570,LIS440') ,
(12, 2, 'ECON570,LIS464') ,
(12, 2, 'ECON570,LIS501') ,
(12, 2, 'ECON570,LSC460') ,
(12, 2, 'ECON570,LSC660') ,
(12, 2, 'ECON570,MATH331') ,
(12, 2, 'ECON570,SOC351') ,
(12, 2, 'ECON570,CESOC618') ,
(12, 2, 'ECON570,CESOC693') ,
(12, 2, 'ECON570,SOILSCI585') ,
(12, 2, 'ECON570,STAT405') ,
(12, 2, 'ECON570,STAT433') ,
(12, 2, 'ECON570,LIS461') ,
(12, 2, 'ECON570,MATH331') ,
(12, 2, 'ECON570,MATH309') ,
(12, 2, 'ECON570,STAT311') ,
(12, 2, 'ECON570,MATH431') ,
(12, 2, 'ECON570,MATH310') ,
(12, 2, 'ECON570,STAT312') ,
(12, 2, 'ECON570,MATH320') ,
(12, 2, 'ECON570,MATH340') ,
(12, 2, 'ECON570,MATH341') ,
(12, 2, 'ECON570,MATH375') ,
(12, 2, 'ECON695,GEOG378') ,
(12, 2, 'ECON695,GEOG572') ,
(12, 2, 'ECON695,GEOG575') ,
(12, 2, 'ECON695,ISYE323') ,
(12, 2, 'ECON695,ISYE412') ,
(12, 2, 'ECON695,ISYE512') ,
(12, 2, 'ECON695,ISYE612') ,
(12, 2, 'ECON695,INFOSYS322') ,
(12, 2, 'ECON695,LIS407') ,
(12, 2, 'ECON695,LIS440') ,
(12, 2, 'ECON695,LIS464') ,
(12, 2, 'ECON695,LIS501') ,
(12, 2, 'ECON695,LSC460') ,
(12, 2, 'ECON695,LSC660') ,
(12, 2, 'ECON695,MATH331') ,
(12, 2, 'ECON695,SOC351') ,
(12, 2, 'ECON695,CESOC618') ,
(12, 2, 'ECON695,CESOC693') ,
(12, 2, 'ECON695,SOILSCI585') ,
(12, 2, 'ECON695,STAT405') ,
(12, 2, 'ECON695,STAT433') ,
(12, 2, 'ECON695,LIS461') ,
(12, 2, 'ECON695,MATH331') ,
(12, 2, 'ECON695,MATH309') ,
(12, 2, 'ECON695,STAT311') ,
(12, 2, 'ECON695,MATH431') ,
(12, 2, 'ECON695,MATH310') ,
(12, 2, 'ECON695,STAT312') ,
(12, 2, 'ECON695,MATH320') ,
(12, 2, 'ECON695,MATH340') ,
(12, 2, 'ECON695,MATH341') ,
(12, 2, 'ECON695,MATH375') ,
(12, 2, 'GEOG378,GEOG572') ,
(12, 2, 'GEOG378,GEOG575') ,
(12, 2, 'GEOG378,ISYE323') ,
(12, 2, 'GEOG378,ISYE412') ,
(12, 2, 'GEOG378,ISYE512') ,
(12, 2, 'GEOG378,ISYE612') ,
(12, 2, 'GEOG378,INFOSYS322') ,
(12, 2, 'GEOG378,LIS407') ,
(12, 2, 'GEOG378,LIS440') ,
(12, 2, 'GEOG378,LIS464') ,
(12, 2, 'GEOG378,LIS501') ,
(12, 2, 'GEOG378,LSC460') ,
(12, 2, 'GEOG378,LSC660') ,
(12, 2, 'GEOG378,MATH331') ,
(12, 2, 'GEOG378,SOC351') ,
(12, 2, 'GEOG378,CESOC618') ,
(12, 2, 'GEOG378,CESOC693') ,
(12, 2, 'GEOG378,SOILSCI585') ,
(12, 2, 'GEOG378,STAT405') ,
(12, 2, 'GEOG378,STAT433') ,
(12, 2, 'GEOG378,LIS461') ,
(12, 2, 'GEOG378,MATH331') ,
(12, 2, 'GEOG378,MATH309') ,
(12, 2, 'GEOG378,STAT311') ,
(12, 2, 'GEOG378,MATH431') ,
(12, 2, 'GEOG378,MATH310') ,
(12, 2, 'GEOG378,STAT312') ,
(12, 2, 'GEOG378,MATH320') ,
(12, 2, 'GEOG378,MATH340') ,
(12, 2, 'GEOG378,MATH341') ,
(12, 2, 'GEOG378,MATH375') ,
(12, 2, 'GEOG572,GEOG575') ,
(12, 2, 'GEOG572,ISYE323') ,
(12, 2, 'GEOG572,ISYE412') ,
(12, 2, 'GEOG572,ISYE512') ,
(12, 2, 'GEOG572,ISYE612') ,
(12, 2, 'GEOG572,INFOSYS322') ,
(12, 2, 'GEOG572,LIS407') ,
(12, 2, 'GEOG572,LIS440') ,
(12, 2, 'GEOG572,LIS464') ,
(12, 2, 'GEOG572,LIS501') ,
(12, 2, 'GEOG572,LSC460') ,
(12, 2, 'GEOG572,LSC660') ,
(12, 2, 'GEOG572,MATH331') ,
(12, 2, 'GEOG572,SOC351') ,
(12, 2, 'GEOG572,CESOC618') ,
(12, 2, 'GEOG572,CESOC693') ,
(12, 2, 'GEOG572,SOILSCI585') ,
(12, 2, 'GEOG572,STAT405') ,
(12, 2, 'GEOG572,STAT433') ,
(12, 2, 'GEOG572,LIS461') ,
(12, 2, 'GEOG572,MATH331') ,
(12, 2, 'GEOG572,MATH309') ,
(12, 2, 'GEOG572,STAT311') ,
(12, 2, 'GEOG572,MATH431') ,
(12, 2, 'GEOG572,MATH310') ,
(12, 2, 'GEOG572,STAT312') ,
(12, 2, 'GEOG572,MATH320') ,
(12, 2, 'GEOG572,MATH340') ,
(12, 2, 'GEOG572,MATH341') ,
(12, 2, 'GEOG572,MATH375') ,
(12, 2, 'GEOG575,ISYE323') ,
(12, 2, 'GEOG575,ISYE412') ,
(12, 2, 'GEOG575,ISYE512') ,
(12, 2, 'GEOG575,ISYE612') ,
(12, 2, 'GEOG575,INFOSYS322') ,
(12, 2, 'GEOG575,LIS407') ,
(12, 2, 'GEOG575,LIS440') ,
(12, 2, 'GEOG575,LIS464') ,
(12, 2, 'GEOG575,LIS501') ,
(12, 2, 'GEOG575,LSC460') ,
(12, 2, 'GEOG575,LSC660') ,
(12, 2, 'GEOG575,MATH331') ,
(12, 2, 'GEOG575,SOC351') ,
(12, 2, 'GEOG575,CESOC618') ,
(12, 2, 'GEOG575,CESOC693') ,
(12, 2, 'GEOG575,SOILSCI585') ,
(12, 2, 'GEOG575,STAT405') ,
(12, 2, 'GEOG575,STAT433') ,
(12, 2, 'GEOG575,LIS461') ,
(12, 2, 'GEOG575,MATH331') ,
(12, 2, 'GEOG575,MATH309') ,
(12, 2, 'GEOG575,STAT311') ,
(12, 2, 'GEOG575,MATH431') ,
(12, 2, 'GEOG575,MATH310') ,
(12, 2, 'GEOG575,STAT312') ,
(12, 2, 'GEOG575,MATH320') ,
(12, 2, 'GEOG575,MATH340') ,
(12, 2, 'GEOG575,MATH341') ,
(12, 2, 'GEOG575,MATH375') ,
(12, 2, 'ISYE323,ISYE412') ,
(12, 2, 'ISYE323,ISYE512') ,
(12, 2, 'ISYE323,ISYE612') ,
(12, 2, 'ISYE323,INFOSYS322') ,
(12, 2, 'ISYE323,LIS407') ,
(12, 2, 'ISYE323,LIS440') ,
(12, 2, 'ISYE323,LIS464') ,
(12, 2, 'ISYE323,LIS501') ,
(12, 2, 'ISYE323,LSC460') ,
(12, 2, 'ISYE323,LSC660') ,
(12, 2, 'ISYE323,MATH331') ,
(12, 2, 'ISYE323,SOC351') ,
(12, 2, 'ISYE323,CESOC618') ,
(12, 2, 'ISYE323,CESOC693') ,
(12, 2, 'ISYE323,SOILSCI585') ,
(12, 2, 'ISYE323,STAT405') ,
(12, 2, 'ISYE323,STAT433') ,
(12, 2, 'ISYE323,LIS461') ,
(12, 2, 'ISYE323,MATH331') ,
(12, 2, 'ISYE323,MATH309') ,
(12, 2, 'ISYE323,STAT311') ,
(12, 2, 'ISYE323,MATH431') ,
(12, 2, 'ISYE323,MATH310') ,
(12, 2, 'ISYE323,STAT312') ,
(12, 2, 'ISYE323,MATH320') ,
(12, 2, 'ISYE323,MATH340') ,
(12, 2, 'ISYE323,MATH341') ,
(12, 2, 'ISYE323,MATH375') ,
(12, 2, 'ISYE412,ISYE512') ,
(12, 2, 'ISYE412,ISYE612') ,
(12, 2, 'ISYE412,INFOSYS322') ,
(12, 2, 'ISYE412,LIS407') ,
(12, 2, 'ISYE412,LIS440') ,
(12, 2, 'ISYE412,LIS464') ,
(12, 2, 'ISYE412,LIS501') ,
(12, 2, 'ISYE412,LSC460') ,
(12, 2, 'ISYE412,LSC660') ,
(12, 2, 'ISYE412,MATH331') ,
(12, 2, 'ISYE412,SOC351') ,
(12, 2, 'ISYE412,CESOC618') ,
(12, 2, 'ISYE412,CESOC693') ,
(12, 2, 'ISYE412,SOILSCI585') ,
(12, 2, 'ISYE412,STAT405') ,
(12, 2, 'ISYE412,STAT433') ,
(12, 2, 'ISYE412,LIS461') ,
(12, 2, 'ISYE412,MATH331') ,
(12, 2, 'ISYE412,MATH309') ,
(12, 2, 'ISYE412,STAT311') ,
(12, 2, 'ISYE412,MATH431') ,
(12, 2, 'ISYE412,MATH310') ,
(12, 2, 'ISYE412,STAT312') ,
(12, 2, 'ISYE412,MATH320') ,
(12, 2, 'ISYE412,MATH340') ,
(12, 2, 'ISYE412,MATH341') ,
(12, 2, 'ISYE412,MATH375') ,
(12, 2, 'ISYE512,ISYE612') ,
(12, 2, 'ISYE512,INFOSYS322') ,
(12, 2, 'ISYE512,LIS407') ,
(12, 2, 'ISYE512,LIS440') ,
(12, 2, 'ISYE512,LIS464') ,
(12, 2, 'ISYE512,LIS501') ,
(12, 2, 'ISYE512,LSC460') ,
(12, 2, 'ISYE512,LSC660') ,
(12, 2, 'ISYE512,MATH331') ,
(12, 2, 'ISYE512,SOC351') ,
(12, 2, 'ISYE512,CESOC618') ,
(12, 2, 'ISYE512,CESOC693') ,
(12, 2, 'ISYE512,SOILSCI585') ,
(12, 2, 'ISYE512,STAT405') ,
(12, 2, 'ISYE512,STAT433') ,
(12, 2, 'ISYE512,LIS461') ,
(12, 2, 'ISYE512,MATH331') ,
(12, 2, 'ISYE512,MATH309') ,
(12, 2, 'ISYE512,STAT311') ,
(12, 2, 'ISYE512,MATH431') ,
(12, 2, 'ISYE512,MATH310') ,
(12, 2, 'ISYE512,STAT312') ,
(12, 2, 'ISYE512,MATH320') ,
(12, 2, 'ISYE512,MATH340') ,
(12, 2, 'ISYE512,MATH341') ,
(12, 2, 'ISYE512,MATH375') ,
(12, 2, 'ISYE612,INFOSYS322') ,
(12, 2, 'ISYE612,LIS407') ,
(12, 2, 'ISYE612,LIS440') ,
(12, 2, 'ISYE612,LIS464') ,
(12, 2, 'ISYE612,LIS501') ,
(12, 2, 'ISYE612,LSC460') ,
(12, 2, 'ISYE612,LSC660') ,
(12, 2, 'ISYE612,MATH331') ,
(12, 2, 'ISYE612,SOC351') ,
(12, 2, 'ISYE612,CESOC618') ,
(12, 2, 'ISYE612,CESOC693') ,
(12, 2, 'ISYE612,SOILSCI585') ,
(12, 2, 'ISYE612,STAT405') ,
(12, 2, 'ISYE612,STAT433') ,
(12, 2, 'ISYE612,LIS461') ,
(12, 2, 'ISYE612,MATH331') ,
(12, 2, 'ISYE612,MATH309') ,
(12, 2, 'ISYE612,STAT311') ,
(12, 2, 'ISYE612,MATH431') ,
(12, 2, 'ISYE612,MATH310') ,
(12, 2, 'ISYE612,STAT312') ,
(12, 2, 'ISYE612,MATH320') ,
(12, 2, 'ISYE612,MATH340') ,
(12, 2, 'ISYE612,MATH341') ,
(12, 2, 'ISYE612,MATH375') ,
(12, 2, 'INFOSYS322,LIS407') ,
(12, 2, 'INFOSYS322,LIS440') ,
(12, 2, 'INFOSYS322,LIS464') ,
(12, 2, 'INFOSYS322,LIS501') ,
(12, 2, 'INFOSYS322,LSC460') ,
(12, 2, 'INFOSYS322,LSC660') ,
(12, 2, 'INFOSYS322,MATH331') ,
(12, 2, 'INFOSYS322,SOC351') ,
(12, 2, 'INFOSYS322,CESOC618') ,
(12, 2, 'INFOSYS322,CESOC693') ,
(12, 2, 'INFOSYS322,SOILSCI585') ,
(12, 2, 'INFOSYS322,STAT405') ,
(12, 2, 'INFOSYS322,STAT433') ,
(12, 2, 'INFOSYS322,LIS461') ,
(12, 2, 'INFOSYS322,MATH331') ,
(12, 2, 'INFOSYS322,MATH309') ,
(12, 2, 'INFOSYS322,STAT311') ,
(12, 2, 'INFOSYS322,MATH431') ,
(12, 2, 'INFOSYS322,MATH310') ,
(12, 2, 'INFOSYS322,STAT312') ,
(12, 2, 'INFOSYS322,MATH320') ,
(12, 2, 'INFOSYS322,MATH340') ,
(12, 2, 'INFOSYS322,MATH341') ,
(12, 2, 'INFOSYS322,MATH375') ,
(12, 2, 'LIS407,LIS440') ,
(12, 2, 'LIS407,LIS464') ,
(12, 2, 'LIS407,LIS501') ,
(12, 2, 'LIS407,LSC460') ,
(12, 2, 'LIS407,LSC660') ,
(12, 2, 'LIS407,MATH331') ,
(12, 2, 'LIS407,SOC351') ,
(12, 2, 'LIS407,CESOC618') ,
(12, 2, 'LIS407,CESOC693') ,
(12, 2, 'LIS407,SOILSCI585') ,
(12, 2, 'LIS407,STAT405') ,
(12, 2, 'LIS407,STAT433') ,
(12, 2, 'LIS407,LIS461') ,
(12, 2, 'LIS407,MATH331') ,
(12, 2, 'LIS407,MATH309') ,
(12, 2, 'LIS407,STAT311') ,
(12, 2, 'LIS407,MATH431') ,
(12, 2, 'LIS407,MATH310') ,
(12, 2, 'LIS407,STAT312') ,
(12, 2, 'LIS407,MATH320') ,
(12, 2, 'LIS407,MATH340') ,
(12, 2, 'LIS407,MATH341') ,
(12, 2, 'LIS407,MATH375') ,
(12, 2, 'LIS440,LIS464') ,
(12, 2, 'LIS440,LIS501') ,
(12, 2, 'LIS440,LSC460') ,
(12, 2, 'LIS440,LSC660') ,
(12, 2, 'LIS440,MATH331') ,
(12, 2, 'LIS440,SOC351') ,
(12, 2, 'LIS440,CESOC618') ,
(12, 2, 'LIS440,CESOC693') ,
(12, 2, 'LIS440,SOILSCI585') ,
(12, 2, 'LIS440,STAT405') ,
(12, 2, 'LIS440,STAT433') ,
(12, 2, 'LIS440,LIS461') ,
(12, 2, 'LIS440,MATH331') ,
(12, 2, 'LIS440,MATH309') ,
(12, 2, 'LIS440,STAT311') ,
(12, 2, 'LIS440,MATH431') ,
(12, 2, 'LIS440,MATH310') ,
(12, 2, 'LIS440,STAT312') ,
(12, 2, 'LIS440,MATH320') ,
(12, 2, 'LIS440,MATH340') ,
(12, 2, 'LIS440,MATH341') ,
(12, 2, 'LIS440,MATH375') ,
(12, 2, 'LIS464,LIS501') ,
(12, 2, 'LIS464,LSC460') ,
(12, 2, 'LIS464,LSC660') ,
(12, 2, 'LIS464,MATH331') ,
(12, 2, 'LIS464,SOC351') ,
(12, 2, 'LIS464,CESOC618') ,
(12, 2, 'LIS464,CESOC693') ,
(12, 2, 'LIS464,SOILSCI585') ,
(12, 2, 'LIS464,STAT405') ,
(12, 2, 'LIS464,STAT433') ,
(12, 2, 'LIS464,LIS461') ,
(12, 2, 'LIS464,MATH331') ,
(12, 2, 'LIS464,MATH309') ,
(12, 2, 'LIS464,STAT311') ,
(12, 2, 'LIS464,MATH431') ,
(12, 2, 'LIS464,MATH310') ,
(12, 2, 'LIS464,STAT312') ,
(12, 2, 'LIS464,MATH320') ,
(12, 2, 'LIS464,MATH340') ,
(12, 2, 'LIS464,MATH341') ,
(12, 2, 'LIS464,MATH375') ,
(12, 2, 'LIS501,LSC460') ,
(12, 2, 'LIS501,LSC660') ,
(12, 2, 'LIS501,MATH331') ,
(12, 2, 'LIS501,SOC351') ,
(12, 2, 'LIS501,CESOC618') ,
(12, 2, 'LIS501,CESOC693') ,
(12, 2, 'LIS501,SOILSCI585') ,
(12, 2, 'LIS501,STAT405') ,
(12, 2, 'LIS501,STAT433') ,
(12, 2, 'LIS501,LIS461') ,
(12, 2, 'LIS501,MATH331') ,
(12, 2, 'LIS501,MATH309') ,
(12, 2, 'LIS501,STAT311') ,
(12, 2, 'LIS501,MATH431') ,
(12, 2, 'LIS501,MATH310') ,
(12, 2, 'LIS501,STAT312') ,
(12, 2, 'LIS501,MATH320') ,
(12, 2, 'LIS501,MATH340') ,
(12, 2, 'LIS501,MATH341') ,
(12, 2, 'LIS501,MATH375') ,
(12, 2, 'LSC460,LSC660') ,
(12, 2, 'LSC460,MATH331') ,
(12, 2, 'LSC460,SOC351') ,
(12, 2, 'LSC460,CESOC618') ,
(12, 2, 'LSC460,CESOC693') ,
(12, 2, 'LSC460,SOILSCI585') ,
(12, 2, 'LSC460,STAT405') ,
(12, 2, 'LSC460,STAT433') ,
(12, 2, 'LSC460,LIS461') ,
(12, 2, 'LSC460,MATH331') ,
(12, 2, 'LSC460,MATH309') ,
(12, 2, 'LSC460,STAT311') ,
(12, 2, 'LSC460,MATH431') ,
(12, 2, 'LSC460,MATH310') ,
(12, 2, 'LSC460,STAT312') ,
(12, 2, 'LSC460,MATH320') ,
(12, 2, 'LSC460,MATH340') ,
(12, 2, 'LSC460,MATH341') ,
(12, 2, 'LSC460,MATH375') ,
(12, 2, 'LSC660,MATH331') ,
(12, 2, 'LSC660,SOC351') ,
(12, 2, 'LSC660,CESOC618') ,
(12, 2, 'LSC660,CESOC693') ,
(12, 2, 'LSC660,SOILSCI585') ,
(12, 2, 'LSC660,STAT405') ,
(12, 2, 'LSC660,STAT433') ,
(12, 2, 'LSC660,LIS461') ,
(12, 2, 'LSC660,MATH331') ,
(12, 2, 'LSC660,MATH309') ,
(12, 2, 'LSC660,STAT311') ,
(12, 2, 'LSC660,MATH431') ,
(12, 2, 'LSC660,MATH310') ,
(12, 2, 'LSC660,STAT312') ,
(12, 2, 'LSC660,MATH320') ,
(12, 2, 'LSC660,MATH340') ,
(12, 2, 'LSC660,MATH341') ,
(12, 2, 'LSC660,MATH375') ,
(12, 2, 'MATH331,SOC351') ,
(12, 2, 'MATH331,CESOC618') ,
(12, 2, 'MATH331,CESOC693') ,
(12, 2, 'MATH331,SOILSCI585') ,
(12, 2, 'MATH331,STAT405') ,
(12, 2, 'MATH331,STAT433') ,
(12, 2, 'MATH331,LIS461') ,
(12, 2, 'MATH331,MATH331') ,
(12, 2, 'MATH331,MATH309') ,
(12, 2, 'MATH331,STAT311') ,
(12, 2, 'MATH331,MATH431') ,
(12, 2, 'MATH331,MATH310') ,
(12, 2, 'MATH331,STAT312') ,
(12, 2, 'MATH331,MATH320') ,
(12, 2, 'MATH331,MATH340') ,
(12, 2, 'MATH331,MATH341') ,
(12, 2, 'MATH331,MATH375') ,
(12, 2, 'SOC351,CESOC618') ,
(12, 2, 'SOC351,CESOC693') ,
(12, 2, 'SOC351,SOILSCI585') ,
(12, 2, 'SOC351,STAT405') ,
(12, 2, 'SOC351,STAT433') ,
(12, 2, 'SOC351,LIS461') ,
(12, 2, 'SOC351,MATH331') ,
(12, 2, 'SOC351,MATH309') ,
(12, 2, 'SOC351,STAT311') ,
(12, 2, 'SOC351,MATH431') ,
(12, 2, 'SOC351,MATH310') ,
(12, 2, 'SOC351,STAT312') ,
(12, 2, 'SOC351,MATH320') ,
(12, 2, 'SOC351,MATH340') ,
(12, 2, 'SOC351,MATH341') ,
(12, 2, 'SOC351,MATH375') ,
(12, 2, 'CESOC618,CESOC693') ,
(12, 2, 'CESOC618,SOILSCI585') ,
(12, 2, 'CESOC618,STAT405') ,
(12, 2, 'CESOC618,STAT433') ,
(12, 2, 'CESOC618,LIS461') ,
(12, 2, 'CESOC618,MATH331') ,
(12, 2, 'CESOC618,MATH309') ,
(12, 2, 'CESOC618,STAT311') ,
(12, 2, 'CESOC618,MATH431') ,
(12, 2, 'CESOC618,MATH310') ,
(12, 2, 'CESOC618,STAT312') ,
(12, 2, 'CESOC618,MATH320') ,
(12, 2, 'CESOC618,MATH340') ,
(12, 2, 'CESOC618,MATH341') ,
(12, 2, 'CESOC618,MATH375') ,
(12, 2, 'CESOC693,SOILSCI585') ,
(12, 2, 'CESOC693,STAT405') ,
(12, 2, 'CESOC693,STAT433') ,
(12, 2, 'CESOC693,LIS461') ,
(12, 2, 'CESOC693,MATH331') ,
(12, 2, 'CESOC693,MATH309') ,
(12, 2, 'CESOC693,STAT311') ,
(12, 2, 'CESOC693,MATH431') ,
(12, 2, 'CESOC693,MATH310') ,
(12, 2, 'CESOC693,STAT312') ,
(12, 2, 'CESOC693,MATH320') ,
(12, 2, 'CESOC693,MATH340') ,
(12, 2, 'CESOC693,MATH341') ,
(12, 2, 'CESOC693,MATH375') ,
(12, 2, 'SOILSCI585,STAT405') ,
(12, 2, 'SOILSCI585,STAT433') ,
(12, 2, 'SOILSCI585,LIS461') ,
(12, 2, 'SOILSCI585,MATH331') ,
(12, 2, 'SOILSCI585,MATH309') ,
(12, 2, 'SOILSCI585,STAT311') ,
(12, 2, 'SOILSCI585,MATH431') ,
(12, 2, 'SOILSCI585,MATH310') ,
(12, 2, 'SOILSCI585,STAT312') ,
(12, 2, 'SOILSCI585,MATH320') ,
(12, 2, 'SOILSCI585,MATH340') ,
(12, 2, 'SOILSCI585,MATH341') ,
(12, 2, 'SOILSCI585,MATH375') ,
(12, 2, 'STAT405,STAT433') ,
(12, 2, 'STAT405,LIS461') ,
(12, 2, 'STAT405,MATH331') ,
(12, 2, 'STAT405,MATH309') ,
(12, 2, 'STAT405,STAT311') ,
(12, 2, 'STAT405,MATH431') ,
(12, 2, 'STAT405,MATH310') ,
(12, 2, 'STAT405,STAT312') ,
(12, 2, 'STAT405,MATH320') ,
(12, 2, 'STAT405,MATH340') ,
(12, 2, 'STAT405,MATH341') ,
(12, 2, 'STAT405,MATH375') ,
(12, 2, 'STAT433,LIS461') ,
(12, 2, 'STAT433,MATH331') ,
(12, 2, 'STAT433,MATH309') ,
(12, 2, 'STAT433,STAT311') ,
(12, 2, 'STAT433,MATH431') ,
(12, 2, 'STAT433,MATH310') ,
(12, 2, 'STAT433,STAT312') ,
(12, 2, 'STAT433,MATH320') ,
(12, 2, 'STAT433,MATH340') ,
(12, 2, 'STAT433,MATH341') ,
(12, 2, 'STAT433,MATH375') ,
(12, 2, 'LIS461,MATH331') ,
(12, 2, 'LIS461,MATH309') ,
(12, 2, 'LIS461,STAT311') ,
(12, 2, 'LIS461,MATH431') ,
(12, 2, 'LIS461,MATH310') ,
(12, 2, 'LIS461,STAT312') ,
(12, 2, 'LIS461,MATH320') ,
(12, 2, 'LIS461,MATH340') ,
(12, 2, 'LIS461,MATH341') ,
(12, 2, 'LIS461,MATH375') ,
(12, 2, 'MATH331,MATH309') ,
(12, 2, 'MATH331,STAT311') ,
(12, 2, 'MATH331,MATH431') ,
(12, 2, 'MATH331,MATH310') ,
(12, 2, 'MATH331,STAT312') ,
(12, 2, 'MATH331,MATH320') ,
(12, 2, 'MATH331,MATH340') ,
(12, 2, 'MATH331,MATH341') ,
(12, 2, 'MATH331,MATH375') ,
(12, 2, 'MATH309,STAT311') ,
(12, 2, 'MATH309,MATH431') ,
(12, 2, 'MATH309,MATH310') ,
(12, 2, 'MATH309,STAT312') ,
(12, 2, 'MATH309,MATH320') ,
(12, 2, 'MATH309,MATH340') ,
(12, 2, 'MATH309,MATH341') ,
(12, 2, 'MATH309,MATH375') ,
(12, 2, 'STAT311,MATH431') ,
(12, 2, 'STAT311,MATH310') ,
(12, 2, 'STAT311,STAT312') ,
(12, 2, 'STAT311,MATH320') ,
(12, 2, 'STAT311,MATH340') ,
(12, 2, 'STAT311,MATH341') ,
(12, 2, 'STAT311,MATH375') ,
(12, 2, 'MATH431,MATH310') ,
(12, 2, 'MATH431,STAT312') ,
(12, 2, 'MATH431,MATH320') ,
(12, 2, 'MATH431,MATH340') ,
(12, 2, 'MATH431,MATH341') ,
(12, 2, 'MATH431,MATH375') ,
(12, 2, 'MATH310,STAT312') ,
(12, 2, 'MATH310,MATH320') ,
(12, 2, 'MATH310,MATH340') ,
(12, 2, 'MATH310,MATH341') ,
(12, 2, 'MATH310,MATH375') ,
(12, 2, 'STAT312,MATH320') ,
(12, 2, 'STAT312,MATH340') ,
(12, 2, 'STAT312,MATH341') ,
(12, 2, 'STAT312,MATH375') ,
(12, 2, 'MATH320,MATH340') ,
(12, 2, 'MATH320,MATH341') ,
(12, 2, 'MATH320,MATH375') ,
(12, 2, 'MATH340,MATH341') ,
(12, 2, 'MATH340,MATH375') ,
(12, 2, 'MATH341,MATH375') ,]

In [27]:
ds_g = sequence_graph.VariantGraph(ds_ref)
for var in ds_variants:
    ds_g.add_complex_variant(*var)
    
ds_g.reorder_topological()
# ds_g.plot()
major_graph["data-science-bs"] = ds_g

In [28]:
with requests.get("https://guide.wisc.edu/undergraduate/letters-science/statistics/data-science-certificate/#requirementstext") as r:
    soup = BeautifulSoup(r.text, 'html.parser')

    courses = []

    for element in soup.find("div", {"id":"requirementstextcontainer"}).find_all("a", {"class":"bubblelink code"}):
        course = element.text.encode('ascii', 'ignore').decode('ascii')
        if re.findall(r"[A-Z]+[0-9]+", course):
            dep,num = re.findall(r"(\D+)([0-9]+)", course)[0]
            codes = sorted(dep.split("/"))[0]+num
            codes = re.sub(r"\W", "", codes)
            courses.append(codes)
            
courses

major_alphabet["data-science-certificate"] = {}
for i in range(len(courses)):
    if courses[i] not in major_alphabet["data-science-certificate"]:
        major_alphabet["data-science-certificate"][courses[i]] = i

In [29]:
dscert_ref = "COMPSCI220,STAT240,LIS461,BIOCORE382,BIOCORE384"
dscert_l = dscert_ref.split(",")
dscert_break = [(dscert_l[i], i+1) for i in range(len(dscert_l))]
dscert_break

[('COMPSCI220', 1),
 ('STAT240', 2),
 ('LIS461', 3),
 ('BIOCORE382', 4),
 ('BIOCORE384', 5)]

In [30]:
# for k in [   ]:
#         if i != k:
#             # j = (4,2,",".join((k,i)) )
#             print(j, ",")

# for i in [
# ]:
#     j = (12,2,i) 
#     print(j, ",")

In [31]:
dscert_variants = [
    (1, 2, 'COMPSCI220,ECE204') ,
(1, 2, 'STAT240,ECE204') ,
(1, 2, 'COMPSCI320,STAT240') ,
(1, 2, 'COMPSCI320,ECE204') ,
(1, 2, 'STAT240,ECE204') ,
    (3,1,'ECE570'),
    (4, 2, 'BIOCORE382,BIOCORE486') ,
(4, 2, 'BIOCORE382,COMPSCI320') ,
(4, 2, 'BIOCORE382,COMPSCI532') ,
(4, 2, 'BIOCORE382,COMPSCI544') ,
(4, 2, 'BIOCORE382,COMPSCI565') ,
(4, 2, 'BIOCORE382,BMI576') ,
(4, 2, 'BIOCORE382,ECON315') ,
(4, 2, 'BIOCORE382,ECON400') ,
(4, 2, 'BIOCORE382,ECON410') ,
(4, 2, 'BIOCORE382,ECON460') ,
(4, 2, 'BIOCORE382,ECON570') ,
(4, 2, 'BIOCORE382,ECON695') ,
(4, 2, 'BIOCORE382,EDPSYCH551') ,
(4, 2, 'BIOCORE382,FINANCE310') ,
(4, 2, 'BIOCORE382,GENBUS656') ,
(4, 2, 'BIOCORE382,GEOG378') ,
(4, 2, 'BIOCORE382,GEOG573') ,
(4, 2, 'BIOCORE382,GEOG574') ,
(4, 2, 'BIOCORE382,GEOG579') ,
(4, 2, 'BIOCORE382,ISYE412') ,
(4, 2, 'BIOCORE382,ISYE521') ,
(4, 2, 'BIOCORE382,MATH444') ,
(4, 2, 'BIOCORE382,MATH535') ,
(4, 2, 'BIOCORE382,PHYSICS361') ,
(4, 2, 'BIOCORE382,SOC362') ,
(4, 2, 'BIOCORE382,SOILSCI585') ,
(4, 2, 'BIOCORE382,STAT340') ,
(4, 2, 'BIOCORE382,STAT405') ,
(4, 2, 'BIOCORE382,STAT436') ,
(4, 2, 'BIOCORE382,COMPSCI471') ,
(4, 2, 'BIOCORE382,AAE421') ,
(4, 2, 'BIOCORE382,COMPSCI524') ,
(4, 2, 'BIOCORE382,GENBUS307') ,
(4, 2, 'BIOCORE382,INFOSYS322') ,
(4, 2, 'BIOCORE382,LIS440') ,
(4, 2, 'BIOCORE382,LSC460') ,
(4, 2, 'BIOCORE382,LSC660') ,
(4, 2, 'BIOCORE382,SOC351') ,
(4, 2, 'BIOCORE382,CESOC618') ,
(4, 2, 'BIOCORE382,COMPSCI320') ,
(4, 2, 'BIOCORE384,BIOCORE382') ,
(4, 2, 'BIOCORE384,BIOCORE486') ,
(4, 2, 'BIOCORE384,COMPSCI320') ,
(4, 2, 'BIOCORE384,COMPSCI532') ,
(4, 2, 'BIOCORE384,COMPSCI544') ,
(4, 2, 'BIOCORE384,COMPSCI565') ,
(4, 2, 'BIOCORE384,BMI576') ,
(4, 2, 'BIOCORE384,ECON315') ,
(4, 2, 'BIOCORE384,ECON400') ,
(4, 2, 'BIOCORE384,ECON410') ,
(4, 2, 'BIOCORE384,ECON460') ,
(4, 2, 'BIOCORE384,ECON570') ,
(4, 2, 'BIOCORE384,ECON695') ,
(4, 2, 'BIOCORE384,EDPSYCH551') ,
(4, 2, 'BIOCORE384,FINANCE310') ,
(4, 2, 'BIOCORE384,GENBUS656') ,
(4, 2, 'BIOCORE384,GEOG378') ,
(4, 2, 'BIOCORE384,GEOG573') ,
(4, 2, 'BIOCORE384,GEOG574') ,
(4, 2, 'BIOCORE384,GEOG579') ,
(4, 2, 'BIOCORE384,ISYE412') ,
(4, 2, 'BIOCORE384,ISYE521') ,
(4, 2, 'BIOCORE384,MATH444') ,
(4, 2, 'BIOCORE384,MATH535') ,
(4, 2, 'BIOCORE384,PHYSICS361') ,
(4, 2, 'BIOCORE384,SOC362') ,
(4, 2, 'BIOCORE384,SOILSCI585') ,
(4, 2, 'BIOCORE384,STAT340') ,
(4, 2, 'BIOCORE384,STAT405') ,
(4, 2, 'BIOCORE384,STAT436') ,
(4, 2, 'BIOCORE384,COMPSCI471') ,
(4, 2, 'BIOCORE384,AAE421') ,
(4, 2, 'BIOCORE384,COMPSCI524') ,
(4, 2, 'BIOCORE384,GENBUS307') ,
(4, 2, 'BIOCORE384,INFOSYS322') ,
(4, 2, 'BIOCORE384,LIS440') ,
(4, 2, 'BIOCORE384,LSC460') ,
(4, 2, 'BIOCORE384,LSC660') ,
(4, 2, 'BIOCORE384,SOC351') ,
(4, 2, 'BIOCORE384,CESOC618') ,
(4, 2, 'BIOCORE384,COMPSCI320') ,
(4, 2, 'BIOCORE486,BIOCORE382') ,
(4, 2, 'BIOCORE486,BIOCORE384') ,
(4, 2, 'BIOCORE486,COMPSCI320') ,
(4, 2, 'BIOCORE486,COMPSCI532') ,
(4, 2, 'BIOCORE486,COMPSCI544') ,
(4, 2, 'BIOCORE486,COMPSCI565') ,
(4, 2, 'BIOCORE486,BMI576') ,
(4, 2, 'BIOCORE486,ECON315') ,
(4, 2, 'BIOCORE486,ECON400') ,
(4, 2, 'BIOCORE486,ECON410') ,
(4, 2, 'BIOCORE486,ECON460') ,
(4, 2, 'BIOCORE486,ECON570') ,
(4, 2, 'BIOCORE486,ECON695') ,
(4, 2, 'BIOCORE486,EDPSYCH551') ,
(4, 2, 'BIOCORE486,FINANCE310') ,
(4, 2, 'BIOCORE486,GENBUS656') ,
(4, 2, 'BIOCORE486,GEOG378') ,
(4, 2, 'BIOCORE486,GEOG573') ,
(4, 2, 'BIOCORE486,GEOG574') ,
(4, 2, 'BIOCORE486,GEOG579') ,
(4, 2, 'BIOCORE486,ISYE412') ,
(4, 2, 'BIOCORE486,ISYE521') ,
(4, 2, 'BIOCORE486,MATH444') ,
(4, 2, 'BIOCORE486,MATH535') ,
(4, 2, 'BIOCORE486,PHYSICS361') ,
(4, 2, 'BIOCORE486,SOC362') ,
(4, 2, 'BIOCORE486,SOILSCI585') ,
(4, 2, 'BIOCORE486,STAT340') ,
(4, 2, 'BIOCORE486,STAT405') ,
(4, 2, 'BIOCORE486,STAT436') ,
(4, 2, 'BIOCORE486,COMPSCI471') ,
(4, 2, 'BIOCORE486,AAE421') ,
(4, 2, 'BIOCORE486,COMPSCI524') ,
(4, 2, 'BIOCORE486,GENBUS307') ,
(4, 2, 'BIOCORE486,INFOSYS322') ,
(4, 2, 'BIOCORE486,LIS440') ,
(4, 2, 'BIOCORE486,LSC460') ,
(4, 2, 'BIOCORE486,LSC660') ,
(4, 2, 'BIOCORE486,SOC351') ,
(4, 2, 'BIOCORE486,CESOC618') ,
(4, 2, 'BIOCORE486,COMPSCI320') ,
(4, 2, 'COMPSCI320,BIOCORE382') ,
(4, 2, 'COMPSCI320,BIOCORE384') ,
(4, 2, 'COMPSCI320,BIOCORE486') ,
(4, 2, 'COMPSCI320,COMPSCI532') ,
(4, 2, 'COMPSCI320,COMPSCI544') ,
(4, 2, 'COMPSCI320,COMPSCI565') ,
(4, 2, 'COMPSCI320,BMI576') ,
(4, 2, 'COMPSCI320,ECON315') ,
(4, 2, 'COMPSCI320,ECON400') ,
(4, 2, 'COMPSCI320,ECON410') ,
(4, 2, 'COMPSCI320,ECON460') ,
(4, 2, 'COMPSCI320,ECON570') ,
(4, 2, 'COMPSCI320,ECON695') ,
(4, 2, 'COMPSCI320,EDPSYCH551') ,
(4, 2, 'COMPSCI320,FINANCE310') ,
(4, 2, 'COMPSCI320,GENBUS656') ,
(4, 2, 'COMPSCI320,GEOG378') ,
(4, 2, 'COMPSCI320,GEOG573') ,
(4, 2, 'COMPSCI320,GEOG574') ,
(4, 2, 'COMPSCI320,GEOG579') ,
(4, 2, 'COMPSCI320,ISYE412') ,
(4, 2, 'COMPSCI320,ISYE521') ,
(4, 2, 'COMPSCI320,MATH444') ,
(4, 2, 'COMPSCI320,MATH535') ,
(4, 2, 'COMPSCI320,PHYSICS361') ,
(4, 2, 'COMPSCI320,SOC362') ,
(4, 2, 'COMPSCI320,SOILSCI585') ,
(4, 2, 'COMPSCI320,STAT340') ,
(4, 2, 'COMPSCI320,STAT405') ,
(4, 2, 'COMPSCI320,STAT436') ,
(4, 2, 'COMPSCI320,COMPSCI471') ,
(4, 2, 'COMPSCI320,AAE421') ,
(4, 2, 'COMPSCI320,COMPSCI524') ,
(4, 2, 'COMPSCI320,GENBUS307') ,
(4, 2, 'COMPSCI320,INFOSYS322') ,
(4, 2, 'COMPSCI320,LIS440') ,
(4, 2, 'COMPSCI320,LSC460') ,
(4, 2, 'COMPSCI320,LSC660') ,
(4, 2, 'COMPSCI320,SOC351') ,
(4, 2, 'COMPSCI320,CESOC618') ,
(4, 2, 'COMPSCI532,BIOCORE382') ,
(4, 2, 'COMPSCI532,BIOCORE384') ,
(4, 2, 'COMPSCI532,BIOCORE486') ,
(4, 2, 'COMPSCI532,COMPSCI320') ,
(4, 2, 'COMPSCI532,COMPSCI544') ,
(4, 2, 'COMPSCI532,COMPSCI565') ,
(4, 2, 'COMPSCI532,BMI576') ,
(4, 2, 'COMPSCI532,ECON315') ,
(4, 2, 'COMPSCI532,ECON400') ,
(4, 2, 'COMPSCI532,ECON410') ,
(4, 2, 'COMPSCI532,ECON460') ,
(4, 2, 'COMPSCI532,ECON570') ,
(4, 2, 'COMPSCI532,ECON695') ,
(4, 2, 'COMPSCI532,EDPSYCH551') ,
(4, 2, 'COMPSCI532,FINANCE310') ,
(4, 2, 'COMPSCI532,GENBUS656') ,
(4, 2, 'COMPSCI532,GEOG378') ,
(4, 2, 'COMPSCI532,GEOG573') ,
(4, 2, 'COMPSCI532,GEOG574') ,
(4, 2, 'COMPSCI532,GEOG579') ,
(4, 2, 'COMPSCI532,ISYE412') ,
(4, 2, 'COMPSCI532,ISYE521') ,
(4, 2, 'COMPSCI532,MATH444') ,
(4, 2, 'COMPSCI532,MATH535') ,
(4, 2, 'COMPSCI532,PHYSICS361') ,
(4, 2, 'COMPSCI532,SOC362') ,
(4, 2, 'COMPSCI532,SOILSCI585') ,
(4, 2, 'COMPSCI532,STAT340') ,
(4, 2, 'COMPSCI532,STAT405') ,
(4, 2, 'COMPSCI532,STAT436') ,
(4, 2, 'COMPSCI532,COMPSCI471') ,
(4, 2, 'COMPSCI532,AAE421') ,
(4, 2, 'COMPSCI532,COMPSCI524') ,
(4, 2, 'COMPSCI532,GENBUS307') ,
(4, 2, 'COMPSCI532,INFOSYS322') ,
(4, 2, 'COMPSCI532,LIS440') ,
(4, 2, 'COMPSCI532,LSC460') ,
(4, 2, 'COMPSCI532,LSC660') ,
(4, 2, 'COMPSCI532,SOC351') ,
(4, 2, 'COMPSCI532,CESOC618') ,
(4, 2, 'COMPSCI532,COMPSCI320') ,
(4, 2, 'COMPSCI544,BIOCORE382') ,
(4, 2, 'COMPSCI544,BIOCORE384') ,
(4, 2, 'COMPSCI544,BIOCORE486') ,
(4, 2, 'COMPSCI544,COMPSCI320') ,
(4, 2, 'COMPSCI544,COMPSCI532') ,
(4, 2, 'COMPSCI544,COMPSCI565') ,
(4, 2, 'COMPSCI544,BMI576') ,
(4, 2, 'COMPSCI544,ECON315') ,
(4, 2, 'COMPSCI544,ECON400') ,
(4, 2, 'COMPSCI544,ECON410') ,
(4, 2, 'COMPSCI544,ECON460') ,
(4, 2, 'COMPSCI544,ECON570') ,
(4, 2, 'COMPSCI544,ECON695') ,
(4, 2, 'COMPSCI544,EDPSYCH551') ,
(4, 2, 'COMPSCI544,FINANCE310') ,
(4, 2, 'COMPSCI544,GENBUS656') ,
(4, 2, 'COMPSCI544,GEOG378') ,
(4, 2, 'COMPSCI544,GEOG573') ,
(4, 2, 'COMPSCI544,GEOG574') ,
(4, 2, 'COMPSCI544,GEOG579') ,
(4, 2, 'COMPSCI544,ISYE412') ,
(4, 2, 'COMPSCI544,ISYE521') ,
(4, 2, 'COMPSCI544,MATH444') ,
(4, 2, 'COMPSCI544,MATH535') ,
(4, 2, 'COMPSCI544,PHYSICS361') ,
(4, 2, 'COMPSCI544,SOC362') ,
(4, 2, 'COMPSCI544,SOILSCI585') ,
(4, 2, 'COMPSCI544,STAT340') ,
(4, 2, 'COMPSCI544,STAT405') ,
(4, 2, 'COMPSCI544,STAT436') ,
(4, 2, 'COMPSCI544,COMPSCI471') ,
(4, 2, 'COMPSCI544,AAE421') ,
(4, 2, 'COMPSCI544,COMPSCI524') ,
(4, 2, 'COMPSCI544,GENBUS307') ,
(4, 2, 'COMPSCI544,INFOSYS322') ,
(4, 2, 'COMPSCI544,LIS440') ,
(4, 2, 'COMPSCI544,LSC460') ,
(4, 2, 'COMPSCI544,LSC660') ,
(4, 2, 'COMPSCI544,SOC351') ,
(4, 2, 'COMPSCI544,CESOC618') ,
(4, 2, 'COMPSCI544,COMPSCI320') ,
(4, 2, 'COMPSCI565,BIOCORE382') ,
(4, 2, 'COMPSCI565,BIOCORE384') ,
(4, 2, 'COMPSCI565,BIOCORE486') ,
(4, 2, 'COMPSCI565,COMPSCI320') ,
(4, 2, 'COMPSCI565,COMPSCI532') ,
(4, 2, 'COMPSCI565,COMPSCI544') ,
(4, 2, 'COMPSCI565,BMI576') ,
(4, 2, 'COMPSCI565,ECON315') ,
(4, 2, 'COMPSCI565,ECON400') ,
(4, 2, 'COMPSCI565,ECON410') ,
(4, 2, 'COMPSCI565,ECON460') ,
(4, 2, 'COMPSCI565,ECON570') ,
(4, 2, 'COMPSCI565,ECON695') ,
(4, 2, 'COMPSCI565,EDPSYCH551') ,
(4, 2, 'COMPSCI565,FINANCE310') ,
(4, 2, 'COMPSCI565,GENBUS656') ,
(4, 2, 'COMPSCI565,GEOG378') ,
(4, 2, 'COMPSCI565,GEOG573') ,
(4, 2, 'COMPSCI565,GEOG574') ,
(4, 2, 'COMPSCI565,GEOG579') ,
(4, 2, 'COMPSCI565,ISYE412') ,
(4, 2, 'COMPSCI565,ISYE521') ,
(4, 2, 'COMPSCI565,MATH444') ,
(4, 2, 'COMPSCI565,MATH535') ,
(4, 2, 'COMPSCI565,PHYSICS361') ,
(4, 2, 'COMPSCI565,SOC362') ,
(4, 2, 'COMPSCI565,SOILSCI585') ,
(4, 2, 'COMPSCI565,STAT340') ,
(4, 2, 'COMPSCI565,STAT405') ,
(4, 2, 'COMPSCI565,STAT436') ,
(4, 2, 'COMPSCI565,COMPSCI471') ,
(4, 2, 'COMPSCI565,AAE421') ,
(4, 2, 'COMPSCI565,COMPSCI524') ,
(4, 2, 'COMPSCI565,GENBUS307') ,
(4, 2, 'COMPSCI565,INFOSYS322') ,
(4, 2, 'COMPSCI565,LIS440') ,
(4, 2, 'COMPSCI565,LSC460') ,
(4, 2, 'COMPSCI565,LSC660') ,
(4, 2, 'COMPSCI565,SOC351') ,
(4, 2, 'COMPSCI565,CESOC618') ,
(4, 2, 'COMPSCI565,COMPSCI320') ,
(4, 2, 'BMI576,BIOCORE382') ,
(4, 2, 'BMI576,BIOCORE384') ,
(4, 2, 'BMI576,BIOCORE486') ,
(4, 2, 'BMI576,COMPSCI320') ,
(4, 2, 'BMI576,COMPSCI532') ,
(4, 2, 'BMI576,COMPSCI544') ,
(4, 2, 'BMI576,COMPSCI565') ,
(4, 2, 'BMI576,ECON315') ,
(4, 2, 'BMI576,ECON400') ,
(4, 2, 'BMI576,ECON410') ,
(4, 2, 'BMI576,ECON460') ,
(4, 2, 'BMI576,ECON570') ,
(4, 2, 'BMI576,ECON695') ,
(4, 2, 'BMI576,EDPSYCH551') ,
(4, 2, 'BMI576,FINANCE310') ,
(4, 2, 'BMI576,GENBUS656') ,
(4, 2, 'BMI576,GEOG378') ,
(4, 2, 'BMI576,GEOG573') ,
(4, 2, 'BMI576,GEOG574') ,
(4, 2, 'BMI576,GEOG579') ,
(4, 2, 'BMI576,ISYE412') ,
(4, 2, 'BMI576,ISYE521') ,
(4, 2, 'BMI576,MATH444') ,
(4, 2, 'BMI576,MATH535') ,
(4, 2, 'BMI576,PHYSICS361') ,
(4, 2, 'BMI576,SOC362') ,
(4, 2, 'BMI576,SOILSCI585') ,
(4, 2, 'BMI576,STAT340') ,
(4, 2, 'BMI576,STAT405') ,
(4, 2, 'BMI576,STAT436') ,
(4, 2, 'BMI576,COMPSCI471') ,
(4, 2, 'BMI576,AAE421') ,
(4, 2, 'BMI576,COMPSCI524') ,
(4, 2, 'BMI576,GENBUS307') ,
(4, 2, 'BMI576,INFOSYS322') ,
(4, 2, 'BMI576,LIS440') ,
(4, 2, 'BMI576,LSC460') ,
(4, 2, 'BMI576,LSC660') ,
(4, 2, 'BMI576,SOC351') ,
(4, 2, 'BMI576,CESOC618') ,
(4, 2, 'BMI576,COMPSCI320') ,
(4, 2, 'ECON315,BIOCORE382') ,
(4, 2, 'ECON315,BIOCORE384') ,
(4, 2, 'ECON315,BIOCORE486') ,
(4, 2, 'ECON315,COMPSCI320') ,
(4, 2, 'ECON315,COMPSCI532') ,
(4, 2, 'ECON315,COMPSCI544') ,
(4, 2, 'ECON315,COMPSCI565') ,
(4, 2, 'ECON315,BMI576') ,
(4, 2, 'ECON315,ECON400') ,
(4, 2, 'ECON315,ECON410') ,
(4, 2, 'ECON315,ECON460') ,
(4, 2, 'ECON315,ECON570') ,
(4, 2, 'ECON315,ECON695') ,
(4, 2, 'ECON315,EDPSYCH551') ,
(4, 2, 'ECON315,FINANCE310') ,
(4, 2, 'ECON315,GENBUS656') ,
(4, 2, 'ECON315,GEOG378') ,
(4, 2, 'ECON315,GEOG573') ,
(4, 2, 'ECON315,GEOG574') ,
(4, 2, 'ECON315,GEOG579') ,
(4, 2, 'ECON315,ISYE412') ,
(4, 2, 'ECON315,ISYE521') ,
(4, 2, 'ECON315,MATH444') ,
(4, 2, 'ECON315,MATH535') ,
(4, 2, 'ECON315,PHYSICS361') ,
(4, 2, 'ECON315,SOC362') ,
(4, 2, 'ECON315,SOILSCI585') ,
(4, 2, 'ECON315,STAT340') ,
(4, 2, 'ECON315,STAT405') ,
(4, 2, 'ECON315,STAT436') ,
(4, 2, 'ECON315,COMPSCI471') ,
(4, 2, 'ECON315,AAE421') ,
(4, 2, 'ECON315,COMPSCI524') ,
(4, 2, 'ECON315,GENBUS307') ,
(4, 2, 'ECON315,INFOSYS322') ,
(4, 2, 'ECON315,LIS440') ,
(4, 2, 'ECON315,LSC460') ,
(4, 2, 'ECON315,LSC660') ,
(4, 2, 'ECON315,SOC351') ,
(4, 2, 'ECON315,CESOC618') ,
(4, 2, 'ECON315,COMPSCI320') ,
(4, 2, 'ECON400,BIOCORE382') ,
(4, 2, 'ECON400,BIOCORE384') ,
(4, 2, 'ECON400,BIOCORE486') ,
(4, 2, 'ECON400,COMPSCI320') ,
(4, 2, 'ECON400,COMPSCI532') ,
(4, 2, 'ECON400,COMPSCI544') ,
(4, 2, 'ECON400,COMPSCI565') ,
(4, 2, 'ECON400,BMI576') ,
(4, 2, 'ECON400,ECON315') ,
(4, 2, 'ECON400,ECON410') ,
(4, 2, 'ECON400,ECON460') ,
(4, 2, 'ECON400,ECON570') ,
(4, 2, 'ECON400,ECON695') ,
(4, 2, 'ECON400,EDPSYCH551') ,
(4, 2, 'ECON400,FINANCE310') ,
(4, 2, 'ECON400,GENBUS656') ,
(4, 2, 'ECON400,GEOG378') ,
(4, 2, 'ECON400,GEOG573') ,
(4, 2, 'ECON400,GEOG574') ,
(4, 2, 'ECON400,GEOG579') ,
(4, 2, 'ECON400,ISYE412') ,
(4, 2, 'ECON400,ISYE521') ,
(4, 2, 'ECON400,MATH444') ,
(4, 2, 'ECON400,MATH535') ,
(4, 2, 'ECON400,PHYSICS361') ,
(4, 2, 'ECON400,SOC362') ,
(4, 2, 'ECON400,SOILSCI585') ,
(4, 2, 'ECON400,STAT340') ,
(4, 2, 'ECON400,STAT405') ,
(4, 2, 'ECON400,STAT436') ,
(4, 2, 'ECON400,COMPSCI471') ,
(4, 2, 'ECON400,AAE421') ,
(4, 2, 'ECON400,COMPSCI524') ,
(4, 2, 'ECON400,GENBUS307') ,
(4, 2, 'ECON400,INFOSYS322') ,
(4, 2, 'ECON400,LIS440') ,
(4, 2, 'ECON400,LSC460') ,
(4, 2, 'ECON400,LSC660') ,
(4, 2, 'ECON400,SOC351') ,
(4, 2, 'ECON400,CESOC618') ,
(4, 2, 'ECON400,COMPSCI320') ,
(4, 2, 'ECON410,BIOCORE382') ,
(4, 2, 'ECON410,BIOCORE384') ,
(4, 2, 'ECON410,BIOCORE486') ,
(4, 2, 'ECON410,COMPSCI320') ,
(4, 2, 'ECON410,COMPSCI532') ,
(4, 2, 'ECON410,COMPSCI544') ,
(4, 2, 'ECON410,COMPSCI565') ,
(4, 2, 'ECON410,BMI576') ,
(4, 2, 'ECON410,ECON315') ,
(4, 2, 'ECON410,ECON400') ,
(4, 2, 'ECON410,ECON460') ,
(4, 2, 'ECON410,ECON570') ,
(4, 2, 'ECON410,ECON695') ,
(4, 2, 'ECON410,EDPSYCH551') ,
(4, 2, 'ECON410,FINANCE310') ,
(4, 2, 'ECON410,GENBUS656') ,
(4, 2, 'ECON410,GEOG378') ,
(4, 2, 'ECON410,GEOG573') ,
(4, 2, 'ECON410,GEOG574') ,
(4, 2, 'ECON410,GEOG579') ,
(4, 2, 'ECON410,ISYE412') ,
(4, 2, 'ECON410,ISYE521') ,
(4, 2, 'ECON410,MATH444') ,
(4, 2, 'ECON410,MATH535') ,
(4, 2, 'ECON410,PHYSICS361') ,
(4, 2, 'ECON410,SOC362') ,
(4, 2, 'ECON410,SOILSCI585') ,
(4, 2, 'ECON410,STAT340') ,
(4, 2, 'ECON410,STAT405') ,
(4, 2, 'ECON410,STAT436') ,
(4, 2, 'ECON410,COMPSCI471') ,
(4, 2, 'ECON410,AAE421') ,
(4, 2, 'ECON410,COMPSCI524') ,
(4, 2, 'ECON410,GENBUS307') ,
(4, 2, 'ECON410,INFOSYS322') ,
(4, 2, 'ECON410,LIS440') ,
(4, 2, 'ECON410,LSC460') ,
(4, 2, 'ECON410,LSC660') ,
(4, 2, 'ECON410,SOC351') ,
(4, 2, 'ECON410,CESOC618') ,
(4, 2, 'ECON410,COMPSCI320') ,
(4, 2, 'ECON460,BIOCORE382') ,
(4, 2, 'ECON460,BIOCORE384') ,
(4, 2, 'ECON460,BIOCORE486') ,
(4, 2, 'ECON460,COMPSCI320') ,
(4, 2, 'ECON460,COMPSCI532') ,
(4, 2, 'ECON460,COMPSCI544') ,
(4, 2, 'ECON460,COMPSCI565') ,
(4, 2, 'ECON460,BMI576') ,
(4, 2, 'ECON460,ECON315') ,
(4, 2, 'ECON460,ECON400') ,
(4, 2, 'ECON460,ECON410') ,
(4, 2, 'ECON460,ECON570') ,
(4, 2, 'ECON460,ECON695') ,
(4, 2, 'ECON460,EDPSYCH551') ,
(4, 2, 'ECON460,FINANCE310') ,
(4, 2, 'ECON460,GENBUS656') ,
(4, 2, 'ECON460,GEOG378') ,
(4, 2, 'ECON460,GEOG573') ,
(4, 2, 'ECON460,GEOG574') ,
(4, 2, 'ECON460,GEOG579') ,
(4, 2, 'ECON460,ISYE412') ,
(4, 2, 'ECON460,ISYE521') ,
(4, 2, 'ECON460,MATH444') ,
(4, 2, 'ECON460,MATH535') ,
(4, 2, 'ECON460,PHYSICS361') ,
(4, 2, 'ECON460,SOC362') ,
(4, 2, 'ECON460,SOILSCI585') ,
(4, 2, 'ECON460,STAT340') ,
(4, 2, 'ECON460,STAT405') ,
(4, 2, 'ECON460,STAT436') ,
(4, 2, 'ECON460,COMPSCI471') ,
(4, 2, 'ECON460,AAE421') ,
(4, 2, 'ECON460,COMPSCI524') ,
(4, 2, 'ECON460,GENBUS307') ,
(4, 2, 'ECON460,INFOSYS322') ,
(4, 2, 'ECON460,LIS440') ,
(4, 2, 'ECON460,LSC460') ,
(4, 2, 'ECON460,LSC660') ,
(4, 2, 'ECON460,SOC351') ,
(4, 2, 'ECON460,CESOC618') ,
(4, 2, 'ECON460,COMPSCI320') ,
(4, 2, 'ECON570,BIOCORE382') ,
(4, 2, 'ECON570,BIOCORE384') ,
(4, 2, 'ECON570,BIOCORE486') ,
(4, 2, 'ECON570,COMPSCI320') ,
(4, 2, 'ECON570,COMPSCI532') ,
(4, 2, 'ECON570,COMPSCI544') ,
(4, 2, 'ECON570,COMPSCI565') ,
(4, 2, 'ECON570,BMI576') ,
(4, 2, 'ECON570,ECON315') ,
(4, 2, 'ECON570,ECON400') ,
(4, 2, 'ECON570,ECON410') ,
(4, 2, 'ECON570,ECON460') ,
(4, 2, 'ECON570,ECON695') ,
(4, 2, 'ECON570,EDPSYCH551') ,
(4, 2, 'ECON570,FINANCE310') ,
(4, 2, 'ECON570,GENBUS656') ,
(4, 2, 'ECON570,GEOG378') ,
(4, 2, 'ECON570,GEOG573') ,
(4, 2, 'ECON570,GEOG574') ,
(4, 2, 'ECON570,GEOG579') ,
(4, 2, 'ECON570,ISYE412') ,
(4, 2, 'ECON570,ISYE521') ,
(4, 2, 'ECON570,MATH444') ,
(4, 2, 'ECON570,MATH535') ,
(4, 2, 'ECON570,PHYSICS361') ,
(4, 2, 'ECON570,SOC362') ,
(4, 2, 'ECON570,SOILSCI585') ,
(4, 2, 'ECON570,STAT340') ,
(4, 2, 'ECON570,STAT405') ,
(4, 2, 'ECON570,STAT436') ,
(4, 2, 'ECON570,COMPSCI471') ,
(4, 2, 'ECON570,AAE421') ,
(4, 2, 'ECON570,COMPSCI524') ,
(4, 2, 'ECON570,GENBUS307') ,
(4, 2, 'ECON570,INFOSYS322') ,
(4, 2, 'ECON570,LIS440') ,
(4, 2, 'ECON570,LSC460') ,
(4, 2, 'ECON570,LSC660') ,
(4, 2, 'ECON570,SOC351') ,
(4, 2, 'ECON570,CESOC618') ,
(4, 2, 'ECON570,COMPSCI320') ,
(4, 2, 'ECON695,BIOCORE382') ,
(4, 2, 'ECON695,BIOCORE384') ,
(4, 2, 'ECON695,BIOCORE486') ,
(4, 2, 'ECON695,COMPSCI320') ,
(4, 2, 'ECON695,COMPSCI532') ,
(4, 2, 'ECON695,COMPSCI544') ,
(4, 2, 'ECON695,COMPSCI565') ,
(4, 2, 'ECON695,BMI576') ,
(4, 2, 'ECON695,ECON315') ,
(4, 2, 'ECON695,ECON400') ,
(4, 2, 'ECON695,ECON410') ,
(4, 2, 'ECON695,ECON460') ,
(4, 2, 'ECON695,ECON570') ,
(4, 2, 'ECON695,EDPSYCH551') ,
(4, 2, 'ECON695,FINANCE310') ,
(4, 2, 'ECON695,GENBUS656') ,
(4, 2, 'ECON695,GEOG378') ,
(4, 2, 'ECON695,GEOG573') ,
(4, 2, 'ECON695,GEOG574') ,
(4, 2, 'ECON695,GEOG579') ,
(4, 2, 'ECON695,ISYE412') ,
(4, 2, 'ECON695,ISYE521') ,
(4, 2, 'ECON695,MATH444') ,
(4, 2, 'ECON695,MATH535') ,
(4, 2, 'ECON695,PHYSICS361') ,
(4, 2, 'ECON695,SOC362') ,
(4, 2, 'ECON695,SOILSCI585') ,
(4, 2, 'ECON695,STAT340') ,
(4, 2, 'ECON695,STAT405') ,
(4, 2, 'ECON695,STAT436') ,
(4, 2, 'ECON695,COMPSCI471') ,
(4, 2, 'ECON695,AAE421') ,
(4, 2, 'ECON695,COMPSCI524') ,
(4, 2, 'ECON695,GENBUS307') ,
(4, 2, 'ECON695,INFOSYS322') ,
(4, 2, 'ECON695,LIS440') ,
(4, 2, 'ECON695,LSC460') ,
(4, 2, 'ECON695,LSC660') ,
(4, 2, 'ECON695,SOC351') ,
(4, 2, 'ECON695,CESOC618') ,
(4, 2, 'ECON695,COMPSCI320') ,
(4, 2, 'EDPSYCH551,BIOCORE382') ,
(4, 2, 'EDPSYCH551,BIOCORE384') ,
(4, 2, 'EDPSYCH551,BIOCORE486') ,
(4, 2, 'EDPSYCH551,COMPSCI320') ,
(4, 2, 'EDPSYCH551,COMPSCI532') ,
(4, 2, 'EDPSYCH551,COMPSCI544') ,
(4, 2, 'EDPSYCH551,COMPSCI565') ,
(4, 2, 'EDPSYCH551,BMI576') ,
(4, 2, 'EDPSYCH551,ECON315') ,
(4, 2, 'EDPSYCH551,ECON400') ,
(4, 2, 'EDPSYCH551,ECON410') ,
(4, 2, 'EDPSYCH551,ECON460') ,
(4, 2, 'EDPSYCH551,ECON570') ,
(4, 2, 'EDPSYCH551,ECON695') ,
(4, 2, 'EDPSYCH551,FINANCE310') ,
(4, 2, 'EDPSYCH551,GENBUS656') ,
(4, 2, 'EDPSYCH551,GEOG378') ,
(4, 2, 'EDPSYCH551,GEOG573') ,
(4, 2, 'EDPSYCH551,GEOG574') ,
(4, 2, 'EDPSYCH551,GEOG579') ,
(4, 2, 'EDPSYCH551,ISYE412') ,
(4, 2, 'EDPSYCH551,ISYE521') ,
(4, 2, 'EDPSYCH551,MATH444') ,
(4, 2, 'EDPSYCH551,MATH535') ,
(4, 2, 'EDPSYCH551,PHYSICS361') ,
(4, 2, 'EDPSYCH551,SOC362') ,
(4, 2, 'EDPSYCH551,SOILSCI585') ,
(4, 2, 'EDPSYCH551,STAT340') ,
(4, 2, 'EDPSYCH551,STAT405') ,
(4, 2, 'EDPSYCH551,STAT436') ,
(4, 2, 'EDPSYCH551,COMPSCI471') ,
(4, 2, 'EDPSYCH551,AAE421') ,
(4, 2, 'EDPSYCH551,COMPSCI524') ,
(4, 2, 'EDPSYCH551,GENBUS307') ,
(4, 2, 'EDPSYCH551,INFOSYS322') ,
(4, 2, 'EDPSYCH551,LIS440') ,
(4, 2, 'EDPSYCH551,LSC460') ,
(4, 2, 'EDPSYCH551,LSC660') ,
(4, 2, 'EDPSYCH551,SOC351') ,
(4, 2, 'EDPSYCH551,CESOC618') ,
(4, 2, 'EDPSYCH551,COMPSCI320') ,
(4, 2, 'FINANCE310,BIOCORE382') ,
(4, 2, 'FINANCE310,BIOCORE384') ,
(4, 2, 'FINANCE310,BIOCORE486') ,
(4, 2, 'FINANCE310,COMPSCI320') ,
(4, 2, 'FINANCE310,COMPSCI532') ,
(4, 2, 'FINANCE310,COMPSCI544') ,
(4, 2, 'FINANCE310,COMPSCI565') ,
(4, 2, 'FINANCE310,BMI576') ,
(4, 2, 'FINANCE310,ECON315') ,
(4, 2, 'FINANCE310,ECON400') ,
(4, 2, 'FINANCE310,ECON410') ,
(4, 2, 'FINANCE310,ECON460') ,
(4, 2, 'FINANCE310,ECON570') ,
(4, 2, 'FINANCE310,ECON695') ,
(4, 2, 'FINANCE310,EDPSYCH551') ,
(4, 2, 'FINANCE310,GENBUS656') ,
(4, 2, 'FINANCE310,GEOG378') ,
(4, 2, 'FINANCE310,GEOG573') ,
(4, 2, 'FINANCE310,GEOG574') ,
(4, 2, 'FINANCE310,GEOG579') ,
(4, 2, 'FINANCE310,ISYE412') ,
(4, 2, 'FINANCE310,ISYE521') ,
(4, 2, 'FINANCE310,MATH444') ,
(4, 2, 'FINANCE310,MATH535') ,
(4, 2, 'FINANCE310,PHYSICS361') ,
(4, 2, 'FINANCE310,SOC362') ,
(4, 2, 'FINANCE310,SOILSCI585') ,
(4, 2, 'FINANCE310,STAT340') ,
(4, 2, 'FINANCE310,STAT405') ,
(4, 2, 'FINANCE310,STAT436') ,
(4, 2, 'FINANCE310,COMPSCI471') ,
(4, 2, 'FINANCE310,AAE421') ,
(4, 2, 'FINANCE310,COMPSCI524') ,
(4, 2, 'FINANCE310,GENBUS307') ,
(4, 2, 'FINANCE310,INFOSYS322') ,
(4, 2, 'FINANCE310,LIS440') ,
(4, 2, 'FINANCE310,LSC460') ,
(4, 2, 'FINANCE310,LSC660') ,
(4, 2, 'FINANCE310,SOC351') ,
(4, 2, 'FINANCE310,CESOC618') ,
(4, 2, 'FINANCE310,COMPSCI320') ,
(4, 2, 'GENBUS656,BIOCORE382') ,
(4, 2, 'GENBUS656,BIOCORE384') ,
(4, 2, 'GENBUS656,BIOCORE486') ,
(4, 2, 'GENBUS656,COMPSCI320') ,
(4, 2, 'GENBUS656,COMPSCI532') ,
(4, 2, 'GENBUS656,COMPSCI544') ,
(4, 2, 'GENBUS656,COMPSCI565') ,
(4, 2, 'GENBUS656,BMI576') ,
(4, 2, 'GENBUS656,ECON315') ,
(4, 2, 'GENBUS656,ECON400') ,
(4, 2, 'GENBUS656,ECON410') ,
(4, 2, 'GENBUS656,ECON460') ,
(4, 2, 'GENBUS656,ECON570') ,
(4, 2, 'GENBUS656,ECON695') ,
(4, 2, 'GENBUS656,EDPSYCH551') ,
(4, 2, 'GENBUS656,FINANCE310') ,
(4, 2, 'GENBUS656,GEOG378') ,
(4, 2, 'GENBUS656,GEOG573') ,
(4, 2, 'GENBUS656,GEOG574') ,
(4, 2, 'GENBUS656,GEOG579') ,
(4, 2, 'GENBUS656,ISYE412') ,
(4, 2, 'GENBUS656,ISYE521') ,
(4, 2, 'GENBUS656,MATH444') ,
(4, 2, 'GENBUS656,MATH535') ,
(4, 2, 'GENBUS656,PHYSICS361') ,
(4, 2, 'GENBUS656,SOC362') ,
(4, 2, 'GENBUS656,SOILSCI585') ,
(4, 2, 'GENBUS656,STAT340') ,
(4, 2, 'GENBUS656,STAT405') ,
(4, 2, 'GENBUS656,STAT436') ,
(4, 2, 'GENBUS656,COMPSCI471') ,
(4, 2, 'GENBUS656,AAE421') ,
(4, 2, 'GENBUS656,COMPSCI524') ,
(4, 2, 'GENBUS656,GENBUS307') ,
(4, 2, 'GENBUS656,INFOSYS322') ,
(4, 2, 'GENBUS656,LIS440') ,
(4, 2, 'GENBUS656,LSC460') ,
(4, 2, 'GENBUS656,LSC660') ,
(4, 2, 'GENBUS656,SOC351') ,
(4, 2, 'GENBUS656,CESOC618') ,
(4, 2, 'GENBUS656,COMPSCI320') ,
(4, 2, 'GEOG378,BIOCORE382') ,
(4, 2, 'GEOG378,BIOCORE384') ,
(4, 2, 'GEOG378,BIOCORE486') ,
(4, 2, 'GEOG378,COMPSCI320') ,
(4, 2, 'GEOG378,COMPSCI532') ,
(4, 2, 'GEOG378,COMPSCI544') ,
(4, 2, 'GEOG378,COMPSCI565') ,
(4, 2, 'GEOG378,BMI576') ,
(4, 2, 'GEOG378,ECON315') ,
(4, 2, 'GEOG378,ECON400') ,
(4, 2, 'GEOG378,ECON410') ,
(4, 2, 'GEOG378,ECON460') ,
(4, 2, 'GEOG378,ECON570') ,
(4, 2, 'GEOG378,ECON695') ,
(4, 2, 'GEOG378,EDPSYCH551') ,
(4, 2, 'GEOG378,FINANCE310') ,
(4, 2, 'GEOG378,GENBUS656') ,
(4, 2, 'GEOG378,GEOG573') ,
(4, 2, 'GEOG378,GEOG574') ,
(4, 2, 'GEOG378,GEOG579') ,
(4, 2, 'GEOG378,ISYE412') ,
(4, 2, 'GEOG378,ISYE521') ,
(4, 2, 'GEOG378,MATH444') ,
(4, 2, 'GEOG378,MATH535') ,
(4, 2, 'GEOG378,PHYSICS361') ,
(4, 2, 'GEOG378,SOC362') ,
(4, 2, 'GEOG378,SOILSCI585') ,
(4, 2, 'GEOG378,STAT340') ,
(4, 2, 'GEOG378,STAT405') ,
(4, 2, 'GEOG378,STAT436') ,
(4, 2, 'GEOG378,COMPSCI471') ,
(4, 2, 'GEOG378,AAE421') ,
(4, 2, 'GEOG378,COMPSCI524') ,
(4, 2, 'GEOG378,GENBUS307') ,
(4, 2, 'GEOG378,INFOSYS322') ,
(4, 2, 'GEOG378,LIS440') ,
(4, 2, 'GEOG378,LSC460') ,
(4, 2, 'GEOG378,LSC660') ,
(4, 2, 'GEOG378,SOC351') ,
(4, 2, 'GEOG378,CESOC618') ,
(4, 2, 'GEOG378,COMPSCI320') ,
(4, 2, 'GEOG573,BIOCORE382') ,
(4, 2, 'GEOG573,BIOCORE384') ,
(4, 2, 'GEOG573,BIOCORE486') ,
(4, 2, 'GEOG573,COMPSCI320') ,
(4, 2, 'GEOG573,COMPSCI532') ,
(4, 2, 'GEOG573,COMPSCI544') ,
(4, 2, 'GEOG573,COMPSCI565') ,
(4, 2, 'GEOG573,BMI576') ,
(4, 2, 'GEOG573,ECON315') ,
(4, 2, 'GEOG573,ECON400') ,
(4, 2, 'GEOG573,ECON410') ,
(4, 2, 'GEOG573,ECON460') ,
(4, 2, 'GEOG573,ECON570') ,
(4, 2, 'GEOG573,ECON695') ,
(4, 2, 'GEOG573,EDPSYCH551') ,
(4, 2, 'GEOG573,FINANCE310') ,
(4, 2, 'GEOG573,GENBUS656') ,
(4, 2, 'GEOG573,GEOG378') ,
(4, 2, 'GEOG573,GEOG574') ,
(4, 2, 'GEOG573,GEOG579') ,
(4, 2, 'GEOG573,ISYE412') ,
(4, 2, 'GEOG573,ISYE521') ,
(4, 2, 'GEOG573,MATH444') ,
(4, 2, 'GEOG573,MATH535') ,
(4, 2, 'GEOG573,PHYSICS361') ,
(4, 2, 'GEOG573,SOC362') ,
(4, 2, 'GEOG573,SOILSCI585') ,
(4, 2, 'GEOG573,STAT340') ,
(4, 2, 'GEOG573,STAT405') ,
(4, 2, 'GEOG573,STAT436') ,
(4, 2, 'GEOG573,COMPSCI471') ,
(4, 2, 'GEOG573,AAE421') ,
(4, 2, 'GEOG573,COMPSCI524') ,
(4, 2, 'GEOG573,GENBUS307') ,
(4, 2, 'GEOG573,INFOSYS322') ,
(4, 2, 'GEOG573,LIS440') ,
(4, 2, 'GEOG573,LSC460') ,
(4, 2, 'GEOG573,LSC660') ,
(4, 2, 'GEOG573,SOC351') ,
(4, 2, 'GEOG573,CESOC618') ,
(4, 2, 'GEOG573,COMPSCI320') ,
(4, 2, 'GEOG574,BIOCORE382') ,
(4, 2, 'GEOG574,BIOCORE384') ,
(4, 2, 'GEOG574,BIOCORE486') ,
(4, 2, 'GEOG574,COMPSCI320') ,
(4, 2, 'GEOG574,COMPSCI532') ,
(4, 2, 'GEOG574,COMPSCI544') ,
(4, 2, 'GEOG574,COMPSCI565') ,
(4, 2, 'GEOG574,BMI576') ,
(4, 2, 'GEOG574,ECON315') ,
(4, 2, 'GEOG574,ECON400') ,
(4, 2, 'GEOG574,ECON410') ,
(4, 2, 'GEOG574,ECON460') ,
(4, 2, 'GEOG574,ECON570') ,
(4, 2, 'GEOG574,ECON695') ,
(4, 2, 'GEOG574,EDPSYCH551') ,
(4, 2, 'GEOG574,FINANCE310') ,
(4, 2, 'GEOG574,GENBUS656') ,
(4, 2, 'GEOG574,GEOG378') ,
(4, 2, 'GEOG574,GEOG573') ,
(4, 2, 'GEOG574,GEOG579') ,
(4, 2, 'GEOG574,ISYE412') ,
(4, 2, 'GEOG574,ISYE521') ,
(4, 2, 'GEOG574,MATH444') ,
(4, 2, 'GEOG574,MATH535') ,
(4, 2, 'GEOG574,PHYSICS361') ,
(4, 2, 'GEOG574,SOC362') ,
(4, 2, 'GEOG574,SOILSCI585') ,
(4, 2, 'GEOG574,STAT340') ,
(4, 2, 'GEOG574,STAT405') ,
(4, 2, 'GEOG574,STAT436') ,
(4, 2, 'GEOG574,COMPSCI471') ,
(4, 2, 'GEOG574,AAE421') ,
(4, 2, 'GEOG574,COMPSCI524') ,
(4, 2, 'GEOG574,GENBUS307') ,
(4, 2, 'GEOG574,INFOSYS322') ,
(4, 2, 'GEOG574,LIS440') ,
(4, 2, 'GEOG574,LSC460') ,
(4, 2, 'GEOG574,LSC660') ,
(4, 2, 'GEOG574,SOC351') ,
(4, 2, 'GEOG574,CESOC618') ,
(4, 2, 'GEOG574,COMPSCI320') ,
(4, 2, 'GEOG579,BIOCORE382') ,
(4, 2, 'GEOG579,BIOCORE384') ,
(4, 2, 'GEOG579,BIOCORE486') ,
(4, 2, 'GEOG579,COMPSCI320') ,
(4, 2, 'GEOG579,COMPSCI532') ,
(4, 2, 'GEOG579,COMPSCI544') ,
(4, 2, 'GEOG579,COMPSCI565') ,
(4, 2, 'GEOG579,BMI576') ,
(4, 2, 'GEOG579,ECON315') ,
(4, 2, 'GEOG579,ECON400') ,
(4, 2, 'GEOG579,ECON410') ,
(4, 2, 'GEOG579,ECON460') ,
(4, 2, 'GEOG579,ECON570') ,
(4, 2, 'GEOG579,ECON695') ,
(4, 2, 'GEOG579,EDPSYCH551') ,
(4, 2, 'GEOG579,FINANCE310') ,
(4, 2, 'GEOG579,GENBUS656') ,
(4, 2, 'GEOG579,GEOG378') ,
(4, 2, 'GEOG579,GEOG573') ,
(4, 2, 'GEOG579,GEOG574') ,
(4, 2, 'GEOG579,ISYE412') ,
(4, 2, 'GEOG579,ISYE521') ,
(4, 2, 'GEOG579,MATH444') ,
(4, 2, 'GEOG579,MATH535') ,
(4, 2, 'GEOG579,PHYSICS361') ,
(4, 2, 'GEOG579,SOC362') ,
(4, 2, 'GEOG579,SOILSCI585') ,
(4, 2, 'GEOG579,STAT340') ,
(4, 2, 'GEOG579,STAT405') ,
(4, 2, 'GEOG579,STAT436') ,
(4, 2, 'GEOG579,COMPSCI471') ,
(4, 2, 'GEOG579,AAE421') ,
(4, 2, 'GEOG579,COMPSCI524') ,
(4, 2, 'GEOG579,GENBUS307') ,
(4, 2, 'GEOG579,INFOSYS322') ,
(4, 2, 'GEOG579,LIS440') ,
(4, 2, 'GEOG579,LSC460') ,
(4, 2, 'GEOG579,LSC660') ,
(4, 2, 'GEOG579,SOC351') ,
(4, 2, 'GEOG579,CESOC618') ,
(4, 2, 'GEOG579,COMPSCI320') ,
(4, 2, 'ISYE412,BIOCORE382') ,
(4, 2, 'ISYE412,BIOCORE384') ,
(4, 2, 'ISYE412,BIOCORE486') ,
(4, 2, 'ISYE412,COMPSCI320') ,
(4, 2, 'ISYE412,COMPSCI532') ,
(4, 2, 'ISYE412,COMPSCI544') ,
(4, 2, 'ISYE412,COMPSCI565') ,
(4, 2, 'ISYE412,BMI576') ,
(4, 2, 'ISYE412,ECON315') ,
(4, 2, 'ISYE412,ECON400') ,
(4, 2, 'ISYE412,ECON410') ,
(4, 2, 'ISYE412,ECON460') ,
(4, 2, 'ISYE412,ECON570') ,
(4, 2, 'ISYE412,ECON695') ,
(4, 2, 'ISYE412,EDPSYCH551') ,
(4, 2, 'ISYE412,FINANCE310') ,
(4, 2, 'ISYE412,GENBUS656') ,
(4, 2, 'ISYE412,GEOG378') ,
(4, 2, 'ISYE412,GEOG573') ,
(4, 2, 'ISYE412,GEOG574') ,
(4, 2, 'ISYE412,GEOG579') ,
(4, 2, 'ISYE412,ISYE521') ,
(4, 2, 'ISYE412,MATH444') ,
(4, 2, 'ISYE412,MATH535') ,
(4, 2, 'ISYE412,PHYSICS361') ,
(4, 2, 'ISYE412,SOC362') ,
(4, 2, 'ISYE412,SOILSCI585') ,
(4, 2, 'ISYE412,STAT340') ,
(4, 2, 'ISYE412,STAT405') ,
(4, 2, 'ISYE412,STAT436') ,
(4, 2, 'ISYE412,COMPSCI471') ,
(4, 2, 'ISYE412,AAE421') ,
(4, 2, 'ISYE412,COMPSCI524') ,
(4, 2, 'ISYE412,GENBUS307') ,
(4, 2, 'ISYE412,INFOSYS322') ,
(4, 2, 'ISYE412,LIS440') ,
(4, 2, 'ISYE412,LSC460') ,
(4, 2, 'ISYE412,LSC660') ,
(4, 2, 'ISYE412,SOC351') ,
(4, 2, 'ISYE412,CESOC618') ,
(4, 2, 'ISYE412,COMPSCI320') ,
(4, 2, 'ISYE521,BIOCORE382') ,
(4, 2, 'ISYE521,BIOCORE384') ,
(4, 2, 'ISYE521,BIOCORE486') ,
(4, 2, 'ISYE521,COMPSCI320') ,
(4, 2, 'ISYE521,COMPSCI532') ,
(4, 2, 'ISYE521,COMPSCI544') ,
(4, 2, 'ISYE521,COMPSCI565') ,
(4, 2, 'ISYE521,BMI576') ,
(4, 2, 'ISYE521,ECON315') ,
(4, 2, 'ISYE521,ECON400') ,
(4, 2, 'ISYE521,ECON410') ,
(4, 2, 'ISYE521,ECON460') ,
(4, 2, 'ISYE521,ECON570') ,
(4, 2, 'ISYE521,ECON695') ,
(4, 2, 'ISYE521,EDPSYCH551') ,
(4, 2, 'ISYE521,FINANCE310') ,
(4, 2, 'ISYE521,GENBUS656') ,
(4, 2, 'ISYE521,GEOG378') ,
(4, 2, 'ISYE521,GEOG573') ,
(4, 2, 'ISYE521,GEOG574') ,
(4, 2, 'ISYE521,GEOG579') ,
(4, 2, 'ISYE521,ISYE412') ,
(4, 2, 'ISYE521,MATH444') ,
(4, 2, 'ISYE521,MATH535') ,
(4, 2, 'ISYE521,PHYSICS361') ,
(4, 2, 'ISYE521,SOC362') ,
(4, 2, 'ISYE521,SOILSCI585') ,
(4, 2, 'ISYE521,STAT340') ,
(4, 2, 'ISYE521,STAT405') ,
(4, 2, 'ISYE521,STAT436') ,
(4, 2, 'ISYE521,COMPSCI471') ,
(4, 2, 'ISYE521,AAE421') ,
(4, 2, 'ISYE521,COMPSCI524') ,
(4, 2, 'ISYE521,GENBUS307') ,
(4, 2, 'ISYE521,INFOSYS322') ,
(4, 2, 'ISYE521,LIS440') ,
(4, 2, 'ISYE521,LSC460') ,
(4, 2, 'ISYE521,LSC660') ,
(4, 2, 'ISYE521,SOC351') ,
(4, 2, 'ISYE521,CESOC618') ,
(4, 2, 'ISYE521,COMPSCI320') ,
(4, 2, 'MATH444,BIOCORE382') ,
(4, 2, 'MATH444,BIOCORE384') ,
(4, 2, 'MATH444,BIOCORE486') ,
(4, 2, 'MATH444,COMPSCI320') ,
(4, 2, 'MATH444,COMPSCI532') ,
(4, 2, 'MATH444,COMPSCI544') ,
(4, 2, 'MATH444,COMPSCI565') ,
(4, 2, 'MATH444,BMI576') ,
(4, 2, 'MATH444,ECON315') ,
(4, 2, 'MATH444,ECON400') ,
(4, 2, 'MATH444,ECON410') ,
(4, 2, 'MATH444,ECON460') ,
(4, 2, 'MATH444,ECON570') ,
(4, 2, 'MATH444,ECON695') ,
(4, 2, 'MATH444,EDPSYCH551') ,
(4, 2, 'MATH444,FINANCE310') ,
(4, 2, 'MATH444,GENBUS656') ,
(4, 2, 'MATH444,GEOG378') ,
(4, 2, 'MATH444,GEOG573') ,
(4, 2, 'MATH444,GEOG574') ,
(4, 2, 'MATH444,GEOG579') ,
(4, 2, 'MATH444,ISYE412') ,
(4, 2, 'MATH444,ISYE521') ,
(4, 2, 'MATH444,MATH535') ,
(4, 2, 'MATH444,PHYSICS361') ,
(4, 2, 'MATH444,SOC362') ,
(4, 2, 'MATH444,SOILSCI585') ,
(4, 2, 'MATH444,STAT340') ,
(4, 2, 'MATH444,STAT405') ,
(4, 2, 'MATH444,STAT436') ,
(4, 2, 'MATH444,COMPSCI471') ,
(4, 2, 'MATH444,AAE421') ,
(4, 2, 'MATH444,COMPSCI524') ,
(4, 2, 'MATH444,GENBUS307') ,
(4, 2, 'MATH444,INFOSYS322') ,
(4, 2, 'MATH444,LIS440') ,
(4, 2, 'MATH444,LSC460') ,
(4, 2, 'MATH444,LSC660') ,
(4, 2, 'MATH444,SOC351') ,
(4, 2, 'MATH444,CESOC618') ,
(4, 2, 'MATH444,COMPSCI320') ,
(4, 2, 'MATH535,BIOCORE382') ,
(4, 2, 'MATH535,BIOCORE384') ,
(4, 2, 'MATH535,BIOCORE486') ,
(4, 2, 'MATH535,COMPSCI320') ,
(4, 2, 'MATH535,COMPSCI532') ,
(4, 2, 'MATH535,COMPSCI544') ,
(4, 2, 'MATH535,COMPSCI565') ,
(4, 2, 'MATH535,BMI576') ,
(4, 2, 'MATH535,ECON315') ,
(4, 2, 'MATH535,ECON400') ,
(4, 2, 'MATH535,ECON410') ,
(4, 2, 'MATH535,ECON460') ,
(4, 2, 'MATH535,ECON570') ,
(4, 2, 'MATH535,ECON695') ,
(4, 2, 'MATH535,EDPSYCH551') ,
(4, 2, 'MATH535,FINANCE310') ,
(4, 2, 'MATH535,GENBUS656') ,
(4, 2, 'MATH535,GEOG378') ,
(4, 2, 'MATH535,GEOG573') ,
(4, 2, 'MATH535,GEOG574') ,
(4, 2, 'MATH535,GEOG579') ,
(4, 2, 'MATH535,ISYE412') ,
(4, 2, 'MATH535,ISYE521') ,
(4, 2, 'MATH535,MATH444') ,
(4, 2, 'MATH535,PHYSICS361') ,
(4, 2, 'MATH535,SOC362') ,
(4, 2, 'MATH535,SOILSCI585') ,
(4, 2, 'MATH535,STAT340') ,
(4, 2, 'MATH535,STAT405') ,
(4, 2, 'MATH535,STAT436') ,
(4, 2, 'MATH535,COMPSCI471') ,
(4, 2, 'MATH535,AAE421') ,
(4, 2, 'MATH535,COMPSCI524') ,
(4, 2, 'MATH535,GENBUS307') ,
(4, 2, 'MATH535,INFOSYS322') ,
(4, 2, 'MATH535,LIS440') ,
(4, 2, 'MATH535,LSC460') ,
(4, 2, 'MATH535,LSC660') ,
(4, 2, 'MATH535,SOC351') ,
(4, 2, 'MATH535,CESOC618') ,
(4, 2, 'MATH535,COMPSCI320') ,
(4, 2, 'PHYSICS361,BIOCORE382') ,
(4, 2, 'PHYSICS361,BIOCORE384') ,
(4, 2, 'PHYSICS361,BIOCORE486') ,
(4, 2, 'PHYSICS361,COMPSCI320') ,
(4, 2, 'PHYSICS361,COMPSCI532') ,
(4, 2, 'PHYSICS361,COMPSCI544') ,
(4, 2, 'PHYSICS361,COMPSCI565') ,
(4, 2, 'PHYSICS361,BMI576') ,
(4, 2, 'PHYSICS361,ECON315') ,
(4, 2, 'PHYSICS361,ECON400') ,
(4, 2, 'PHYSICS361,ECON410') ,
(4, 2, 'PHYSICS361,ECON460') ,
(4, 2, 'PHYSICS361,ECON570') ,
(4, 2, 'PHYSICS361,ECON695') ,
(4, 2, 'PHYSICS361,EDPSYCH551') ,
(4, 2, 'PHYSICS361,FINANCE310') ,
(4, 2, 'PHYSICS361,GENBUS656') ,
(4, 2, 'PHYSICS361,GEOG378') ,
(4, 2, 'PHYSICS361,GEOG573') ,
(4, 2, 'PHYSICS361,GEOG574') ,
(4, 2, 'PHYSICS361,GEOG579') ,
(4, 2, 'PHYSICS361,ISYE412') ,
(4, 2, 'PHYSICS361,ISYE521') ,
(4, 2, 'PHYSICS361,MATH444') ,
(4, 2, 'PHYSICS361,MATH535') ,
(4, 2, 'PHYSICS361,SOC362') ,
(4, 2, 'PHYSICS361,SOILSCI585') ,
(4, 2, 'PHYSICS361,STAT340') ,
(4, 2, 'PHYSICS361,STAT405') ,
(4, 2, 'PHYSICS361,STAT436') ,
(4, 2, 'PHYSICS361,COMPSCI471') ,
(4, 2, 'PHYSICS361,AAE421') ,
(4, 2, 'PHYSICS361,COMPSCI524') ,
(4, 2, 'PHYSICS361,GENBUS307') ,
(4, 2, 'PHYSICS361,INFOSYS322') ,
(4, 2, 'PHYSICS361,LIS440') ,
(4, 2, 'PHYSICS361,LSC460') ,
(4, 2, 'PHYSICS361,LSC660') ,
(4, 2, 'PHYSICS361,SOC351') ,
(4, 2, 'PHYSICS361,CESOC618') ,
(4, 2, 'PHYSICS361,COMPSCI320') ,
(4, 2, 'SOC362,BIOCORE382') ,
(4, 2, 'SOC362,BIOCORE384') ,
(4, 2, 'SOC362,BIOCORE486') ,
(4, 2, 'SOC362,COMPSCI320') ,
(4, 2, 'SOC362,COMPSCI532') ,
(4, 2, 'SOC362,COMPSCI544') ,
(4, 2, 'SOC362,COMPSCI565') ,
(4, 2, 'SOC362,BMI576') ,
(4, 2, 'SOC362,ECON315') ,
(4, 2, 'SOC362,ECON400') ,
(4, 2, 'SOC362,ECON410') ,
(4, 2, 'SOC362,ECON460') ,
(4, 2, 'SOC362,ECON570') ,
(4, 2, 'SOC362,ECON695') ,
(4, 2, 'SOC362,EDPSYCH551') ,
(4, 2, 'SOC362,FINANCE310') ,
(4, 2, 'SOC362,GENBUS656') ,
(4, 2, 'SOC362,GEOG378') ,
(4, 2, 'SOC362,GEOG573') ,
(4, 2, 'SOC362,GEOG574') ,
(4, 2, 'SOC362,GEOG579') ,
(4, 2, 'SOC362,ISYE412') ,
(4, 2, 'SOC362,ISYE521') ,
(4, 2, 'SOC362,MATH444') ,
(4, 2, 'SOC362,MATH535') ,
(4, 2, 'SOC362,PHYSICS361') ,
(4, 2, 'SOC362,SOILSCI585') ,
(4, 2, 'SOC362,STAT340') ,
(4, 2, 'SOC362,STAT405') ,
(4, 2, 'SOC362,STAT436') ,
(4, 2, 'SOC362,COMPSCI471') ,
(4, 2, 'SOC362,AAE421') ,
(4, 2, 'SOC362,COMPSCI524') ,
(4, 2, 'SOC362,GENBUS307') ,
(4, 2, 'SOC362,INFOSYS322') ,
(4, 2, 'SOC362,LIS440') ,
(4, 2, 'SOC362,LSC460') ,
(4, 2, 'SOC362,LSC660') ,
(4, 2, 'SOC362,SOC351') ,
(4, 2, 'SOC362,CESOC618') ,
(4, 2, 'SOC362,COMPSCI320') ,
(4, 2, 'SOILSCI585,BIOCORE382') ,
(4, 2, 'SOILSCI585,BIOCORE384') ,
(4, 2, 'SOILSCI585,BIOCORE486') ,
(4, 2, 'SOILSCI585,COMPSCI320') ,
(4, 2, 'SOILSCI585,COMPSCI532') ,
(4, 2, 'SOILSCI585,COMPSCI544') ,
(4, 2, 'SOILSCI585,COMPSCI565') ,
(4, 2, 'SOILSCI585,BMI576') ,
(4, 2, 'SOILSCI585,ECON315') ,
(4, 2, 'SOILSCI585,ECON400') ,
(4, 2, 'SOILSCI585,ECON410') ,
(4, 2, 'SOILSCI585,ECON460') ,
(4, 2, 'SOILSCI585,ECON570') ,
(4, 2, 'SOILSCI585,ECON695') ,
(4, 2, 'SOILSCI585,EDPSYCH551') ,
(4, 2, 'SOILSCI585,FINANCE310') ,
(4, 2, 'SOILSCI585,GENBUS656') ,
(4, 2, 'SOILSCI585,GEOG378') ,
(4, 2, 'SOILSCI585,GEOG573') ,
(4, 2, 'SOILSCI585,GEOG574') ,
(4, 2, 'SOILSCI585,GEOG579') ,
(4, 2, 'SOILSCI585,ISYE412') ,
(4, 2, 'SOILSCI585,ISYE521') ,
(4, 2, 'SOILSCI585,MATH444') ,
(4, 2, 'SOILSCI585,MATH535') ,
(4, 2, 'SOILSCI585,PHYSICS361') ,
(4, 2, 'SOILSCI585,SOC362') ,
(4, 2, 'SOILSCI585,STAT340') ,
(4, 2, 'SOILSCI585,STAT405') ,
(4, 2, 'SOILSCI585,STAT436') ,
(4, 2, 'SOILSCI585,COMPSCI471') ,
(4, 2, 'SOILSCI585,AAE421') ,
(4, 2, 'SOILSCI585,COMPSCI524') ,
(4, 2, 'SOILSCI585,GENBUS307') ,
(4, 2, 'SOILSCI585,INFOSYS322') ,
(4, 2, 'SOILSCI585,LIS440') ,
(4, 2, 'SOILSCI585,LSC460') ,
(4, 2, 'SOILSCI585,LSC660') ,
(4, 2, 'SOILSCI585,SOC351') ,
(4, 2, 'SOILSCI585,CESOC618') ,
(4, 2, 'SOILSCI585,COMPSCI320') ,
(4, 2, 'STAT340,BIOCORE382') ,
(4, 2, 'STAT340,BIOCORE384') ,
(4, 2, 'STAT340,BIOCORE486') ,
(4, 2, 'STAT340,COMPSCI320') ,
(4, 2, 'STAT340,COMPSCI532') ,
(4, 2, 'STAT340,COMPSCI544') ,
(4, 2, 'STAT340,COMPSCI565') ,
(4, 2, 'STAT340,BMI576') ,
(4, 2, 'STAT340,ECON315') ,
(4, 2, 'STAT340,ECON400') ,
(4, 2, 'STAT340,ECON410') ,
(4, 2, 'STAT340,ECON460') ,
(4, 2, 'STAT340,ECON570') ,
(4, 2, 'STAT340,ECON695') ,
(4, 2, 'STAT340,EDPSYCH551') ,
(4, 2, 'STAT340,FINANCE310') ,
(4, 2, 'STAT340,GENBUS656') ,
(4, 2, 'STAT340,GEOG378') ,
(4, 2, 'STAT340,GEOG573') ,
(4, 2, 'STAT340,GEOG574') ,
(4, 2, 'STAT340,GEOG579') ,
(4, 2, 'STAT340,ISYE412') ,
(4, 2, 'STAT340,ISYE521') ,
(4, 2, 'STAT340,MATH444') ,
(4, 2, 'STAT340,MATH535') ,
(4, 2, 'STAT340,PHYSICS361') ,
(4, 2, 'STAT340,SOC362') ,
(4, 2, 'STAT340,SOILSCI585') ,
(4, 2, 'STAT340,STAT405') ,
(4, 2, 'STAT340,STAT436') ,
(4, 2, 'STAT340,COMPSCI471') ,
(4, 2, 'STAT340,AAE421') ,
(4, 2, 'STAT340,COMPSCI524') ,
(4, 2, 'STAT340,GENBUS307') ,
(4, 2, 'STAT340,INFOSYS322') ,
(4, 2, 'STAT340,LIS440') ,
(4, 2, 'STAT340,LSC460') ,
(4, 2, 'STAT340,LSC660') ,
(4, 2, 'STAT340,SOC351') ,
(4, 2, 'STAT340,CESOC618') ,
(4, 2, 'STAT340,COMPSCI320') ,
(4, 2, 'STAT405,BIOCORE382') ,
(4, 2, 'STAT405,BIOCORE384') ,
(4, 2, 'STAT405,BIOCORE486') ,
(4, 2, 'STAT405,COMPSCI320') ,
(4, 2, 'STAT405,COMPSCI532') ,
(4, 2, 'STAT405,COMPSCI544') ,
(4, 2, 'STAT405,COMPSCI565') ,
(4, 2, 'STAT405,BMI576') ,
(4, 2, 'STAT405,ECON315') ,
(4, 2, 'STAT405,ECON400') ,
(4, 2, 'STAT405,ECON410') ,
(4, 2, 'STAT405,ECON460') ,
(4, 2, 'STAT405,ECON570') ,
(4, 2, 'STAT405,ECON695') ,
(4, 2, 'STAT405,EDPSYCH551') ,
(4, 2, 'STAT405,FINANCE310') ,
(4, 2, 'STAT405,GENBUS656') ,
(4, 2, 'STAT405,GEOG378') ,
(4, 2, 'STAT405,GEOG573') ,
(4, 2, 'STAT405,GEOG574') ,
(4, 2, 'STAT405,GEOG579') ,
(4, 2, 'STAT405,ISYE412') ,
(4, 2, 'STAT405,ISYE521') ,
(4, 2, 'STAT405,MATH444') ,
(4, 2, 'STAT405,MATH535') ,
(4, 2, 'STAT405,PHYSICS361') ,
(4, 2, 'STAT405,SOC362') ,
(4, 2, 'STAT405,SOILSCI585') ,
(4, 2, 'STAT405,STAT340') ,
(4, 2, 'STAT405,STAT436') ,
(4, 2, 'STAT405,COMPSCI471') ,
(4, 2, 'STAT405,AAE421') ,
(4, 2, 'STAT405,COMPSCI524') ,
(4, 2, 'STAT405,GENBUS307') ,
(4, 2, 'STAT405,INFOSYS322') ,
(4, 2, 'STAT405,LIS440') ,
(4, 2, 'STAT405,LSC460') ,
(4, 2, 'STAT405,LSC660') ,
(4, 2, 'STAT405,SOC351') ,
(4, 2, 'STAT405,CESOC618') ,
(4, 2, 'STAT405,COMPSCI320') ,
(4, 2, 'STAT436,BIOCORE382') ,
(4, 2, 'STAT436,BIOCORE384') ,
(4, 2, 'STAT436,BIOCORE486') ,
(4, 2, 'STAT436,COMPSCI320') ,
(4, 2, 'STAT436,COMPSCI532') ,
(4, 2, 'STAT436,COMPSCI544') ,
(4, 2, 'STAT436,COMPSCI565') ,
(4, 2, 'STAT436,BMI576') ,
(4, 2, 'STAT436,ECON315') ,
(4, 2, 'STAT436,ECON400') ,
(4, 2, 'STAT436,ECON410') ,
(4, 2, 'STAT436,ECON460') ,
(4, 2, 'STAT436,ECON570') ,
(4, 2, 'STAT436,ECON695') ,
(4, 2, 'STAT436,EDPSYCH551') ,
(4, 2, 'STAT436,FINANCE310') ,
(4, 2, 'STAT436,GENBUS656') ,
(4, 2, 'STAT436,GEOG378') ,
(4, 2, 'STAT436,GEOG573') ,
(4, 2, 'STAT436,GEOG574') ,
(4, 2, 'STAT436,GEOG579') ,
(4, 2, 'STAT436,ISYE412') ,
(4, 2, 'STAT436,ISYE521') ,
(4, 2, 'STAT436,MATH444') ,
(4, 2, 'STAT436,MATH535') ,
(4, 2, 'STAT436,PHYSICS361') ,
(4, 2, 'STAT436,SOC362') ,
(4, 2, 'STAT436,SOILSCI585') ,
(4, 2, 'STAT436,STAT340') ,
(4, 2, 'STAT436,STAT405') ,
(4, 2, 'STAT436,COMPSCI471') ,
(4, 2, 'STAT436,AAE421') ,
(4, 2, 'STAT436,COMPSCI524') ,
(4, 2, 'STAT436,GENBUS307') ,
(4, 2, 'STAT436,INFOSYS322') ,
(4, 2, 'STAT436,LIS440') ,
(4, 2, 'STAT436,LSC460') ,
(4, 2, 'STAT436,LSC660') ,
(4, 2, 'STAT436,SOC351') ,
(4, 2, 'STAT436,CESOC618') ,
(4, 2, 'STAT436,COMPSCI320') ,
(4, 2, 'COMPSCI471,BIOCORE382') ,
(4, 2, 'COMPSCI471,BIOCORE384') ,
(4, 2, 'COMPSCI471,BIOCORE486') ,
(4, 2, 'COMPSCI471,COMPSCI320') ,
(4, 2, 'COMPSCI471,COMPSCI532') ,
(4, 2, 'COMPSCI471,COMPSCI544') ,
(4, 2, 'COMPSCI471,COMPSCI565') ,
(4, 2, 'COMPSCI471,BMI576') ,
(4, 2, 'COMPSCI471,ECON315') ,
(4, 2, 'COMPSCI471,ECON400') ,
(4, 2, 'COMPSCI471,ECON410') ,
(4, 2, 'COMPSCI471,ECON460') ,
(4, 2, 'COMPSCI471,ECON570') ,
(4, 2, 'COMPSCI471,ECON695') ,
(4, 2, 'COMPSCI471,EDPSYCH551') ,
(4, 2, 'COMPSCI471,FINANCE310') ,
(4, 2, 'COMPSCI471,GENBUS656') ,
(4, 2, 'COMPSCI471,GEOG378') ,
(4, 2, 'COMPSCI471,GEOG573') ,
(4, 2, 'COMPSCI471,GEOG574') ,
(4, 2, 'COMPSCI471,GEOG579') ,
(4, 2, 'COMPSCI471,ISYE412') ,
(4, 2, 'COMPSCI471,ISYE521') ,
(4, 2, 'COMPSCI471,MATH444') ,
(4, 2, 'COMPSCI471,MATH535') ,
(4, 2, 'COMPSCI471,PHYSICS361') ,
(4, 2, 'COMPSCI471,SOC362') ,
(4, 2, 'COMPSCI471,SOILSCI585') ,
(4, 2, 'COMPSCI471,STAT340') ,
(4, 2, 'COMPSCI471,STAT405') ,
(4, 2, 'COMPSCI471,STAT436') ,
(4, 2, 'COMPSCI471,AAE421') ,
(4, 2, 'COMPSCI471,COMPSCI524') ,
(4, 2, 'COMPSCI471,GENBUS307') ,
(4, 2, 'COMPSCI471,INFOSYS322') ,
(4, 2, 'COMPSCI471,LIS440') ,
(4, 2, 'COMPSCI471,LSC460') ,
(4, 2, 'COMPSCI471,LSC660') ,
(4, 2, 'COMPSCI471,SOC351') ,
(4, 2, 'COMPSCI471,CESOC618') ,
(4, 2, 'COMPSCI471,COMPSCI320') ,]

In [32]:
dscert_break

[('COMPSCI220', 1),
 ('STAT240', 2),
 ('LIS461', 3),
 ('BIOCORE382', 4),
 ('BIOCORE384', 5)]

In [33]:
dscert_g = sequence_graph.VariantGraph(dscert_ref)
for var in dscert_variants:
    dscert_g.add_complex_variant(*var)
    
dscert_g.reorder_topological()
# dscert_g.plot()
major_graph["data-science-certificate"] = dscert_g

In [34]:
all_classes = set()
for key in major_alphabet:
    all_classes.update(list(major_alphabet[key].keys()))

In [35]:
DNA = all_classes
basic_dna_submatrix = submatrix.match_mismatch_matrix(1, -1, DNA)

In [36]:
major_graph

{'astronomy-physics-bs': <sequence_graph.VariantGraph at 0x7f80cb5cc400>,
 'dairy-science-bs': <sequence_graph.VariantGraph at 0x7f80ca92b3d0>,
 'genetics-genomics-bs': <sequence_graph.VariantGraph at 0x7f80c8ff05e0>,
 'data-science-bs': <sequence_graph.VariantGraph at 0x7f80c8fa9900>,
 'data-science-certificate': <sequence_graph.VariantGraph at 0x7f80cb479d50>}

In [41]:
align_graphs(major_graph['data-science-certificate'], major_graph['genetics-genomics-bs'], basic_dna_submatrix, 0)

(3,
 [('COMPSCI220', '-'),
  ('-', 'MATH221'),
  ('STAT240', 'STAT240'),
  ('LIS461', '-'),
  ('-', 'CHEM103'),
  ('-', 'CHEM104'),
  ('-', 'CHEM341'),
  ('-', 'PHYSICS103'),
  ('-', 'PHYSICS104'),
  ('-', 'BIOCORE381'),
  ('-', 'BIOCORE383'),
  ('BIOCORE382', 'BIOCORE382'),
  ('BIOCORE384', 'BIOCORE384'),
  ('-', 'BIOCHEM501'),
  ('-', 'GENETICS467'),
  ('-', 'GENETICS468'),
  ('-', 'GENETICS545'),
  ('-', 'BIOLOGY522'),
  ('-', 'GENETICS520'),
  ('-', 'BIOLOGY522'),
  ('-', 'GENETICS525'),
  ('-', 'BIOCHEM508'),
  ('-', 'BIOCHEM560')])

In [42]:
align_graphs(major_graph['data-science-certificate'], major_graph['data-science-bs'], basic_dna_submatrix, 0)

(4,
 [('COMPSCI220', '-'),
  ('-', 'MATH221'),
  ('-', 'MATH222'),
  ('STAT240', 'STAT240'),
  ('-', 'STAT340'),
  ('-', 'COMPSCI220'),
  ('-', 'COMPSCI320'),
  ('LIS461', 'LIS461'),
  ('COMPSCI532', 'COMPSCI532'),
  ('COMPSCI544', 'COMPSCI544'),
  ('-', 'ECON400'),
  ('-', 'MATH320'),
  ('-', 'COMPSCI425'),
  ('-', 'COMPSCI525')])